# Creating a Placement Algorithm

This tutorial demonstrates how we can create a simple placement algorithm on EdgeSimPy.

Let's start by importing the EdgeSimPy modules:

In [ ]:
# EdgeSimPy Import Debugging Script

# Explicit dependency installation

!pip install tqdm
!pip install numpy rich
!pip install rich
!pip install rich --upgrade
!pip install networkx==2.6.2
!pip install matplotlib pandas numpy
!pip install git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

# Python and package information
!python --version
!pip list | grep -E "networkx|edge_sim_py"

# Comprehensive import and debugging script
import sys
import os
import importlib

def print_module_structure(module_name):
    """
    Recursively print the structure of a module
    """
    print(f"\n--- Module Structure for {module_name} ---")
    try:
        # Import the module
        module = importlib.import_module(module_name)

        # Get the module's file path
        module_file = getattr(module, '__file__', 'No __file__ attribute')
        print(f"Module file path: {module_file}")

        # Get the module's directory
        module_dir = os.path.dirname(module_file) if hasattr(module, '__file__') else 'Unknown'
        print(f"Module directory: {module_dir}")

        # List all attributes and their types
        print("\nModule Contents:")
        for attr_name in dir(module):
            try:
                attr = getattr(module, attr_name)
                print(f"  {attr_name}: {type(attr)}")
            except Exception as attr_err:
                print(f"  {attr_name}: Could not retrieve (Error: {attr_err})")

        # List files in the module directory
        if os.path.isdir(module_dir):
            print("\nFiles in module directory:")
            try:
                for item in os.listdir(module_dir):
                    print(f"  {item}")
            except Exception as list_err:
                print(f"  Could not list directory contents: {list_err}")

    except ImportError as e:
        print(f"Could not import {module_name}: {e}")
    except Exception as e:
        print(f"Unexpected error examining {module_name}: {e}")

# Print Python path and sys.path for debugging
print("--- Python Path ---")
print(sys.path)

# Attempt to import and examine EdgeSimPy
print_module_structure('edge_sim_py')

# Attempt alternative import methods
print("\n--- Alternative Import Attempts ---")
import_attempts = [
    'edge_sim_py',
    'edge_sim_py.core',
    'edge_sim_py.components',
    'edge_sim_py.device',
    'edge_sim_py.server'
]

for attempt in import_attempts:
    print(f"\nTrying to import {attempt}")
    try:
        module = importlib.import_module(attempt)
        print(f"Successfully imported {attempt}")
        print(f"Module file: {getattr(module, '__file__', 'No file attribute')}")
    except ImportError as e:
        print(f"Import failed: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# List installed packages with their paths
print("\n--- Installed Packages Paths ---")
for package_name in ['edge_sim_py', 'networkx', 'numpy', 'pandas']:
    try:
        package = importlib.import_module(package_name)
        print(f"{package_name}: {package.__file__}")
    except ImportError:
        print(f"{package_name}: Not found")
    except Exception as e:
        print(f"{package_name}: Error - {e}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires n

## Implementing the Placement Algorithm

In this example, we are going to create a simple placement algorithm that works according to the well-known First-Fit heuristic. In a nutshell, our algorithm will provision each service to the first edge server with available resources to host them.

In [ ]:
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## Running the Simulation

As we're creating a placement algorithm, we must instruct EdgeSimPy that it needs to continue the simulation until all services are provisioned within the infrastructure.

To do so, let's create a simple function that will be used as the simulation's stopping criterion. EdgeSimPy will run that function at the end of each time step, halting the simulation as soon as it returns `True`.

In [ ]:
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0

    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1

    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Google Colab Setup for FCFS Task Processing



In [ ]:
# Google Colab Setup for FCFS Task Processing

# Install required libraries
!pip install numpy

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# List files in the task sets directory
import os
task_sets_dir = '/content/drive/My Drive/FCFS_Task_Sets/'
print("Available task set files:")
for filename in os.listdir(task_sets_dir):
    print(filename)

# Note: After running this, copy the full path of the desired JSON file
# and use it in the main FCFS scheduler script

MessageError: Error: credential propagation was unsuccessful

FCFS Algorithm Logic


In [ ]:
import json
from typing import List, Dict, Any
import logging
import sys
import time

# Configure logging to print to console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)  # Explicitly add console output
    ]
)
logger = logging.getLogger(__name__)

# Print function to ensure output
def print_to_console(*args, **kwargs):
    """
    Wrapper function to ensure printing
    """
    print(*args, **kwargs)
    sys.stdout.flush()

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')
        self.task_class = self.details.get('task_class', 'generic')
        self.cpu_intensity = self.details.get('cpu_intensity', 'medium')

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with enhanced tracking and visualization
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.memory = memory
        self.bandwidth = bandwidth

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)

    def process_queue(self, current_time: float):
        """
        Process tasks in the queue with detailed tracking
        """
        # Process current tasks first
        for task in self.current_tasks[:]:
            processing_result = task.process(self.cpu_rating)

            # Detailed task processing output
            self._log_task_processing(task, processing_result)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

        # If resource has available capacity, move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:  # Limit concurrent tasks
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)

        return len(self.current_tasks)

    def _log_task_processing(self, task: Task, processing_result: Dict):
        """
        Log detailed task processing information
        """
        print_to_console(
            f"Resource {self.id} ({self.type}) - "
            f"Task {task.id} ({task.task_name}): "
            f"Processed {processing_result['processed']:.2f} MI, "
            f"Remaining {processing_result['remaining']:.2f} MI, "
            f"Completion: {processing_result['completion_percentage']:.2f}%"
        )

class AdvancedFCFSScheduler:
    """
    Advanced First-Come-First-Serve Scheduler with Real-Time Visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.task_queue: List[Task] = []
        self.current_time = 0

        # Metrics tracking with enhanced details
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'queued_tasks': 0,
            'task_distribution': {},
            'resource_utilization': {},
            'average_wait_time': 0,
            'max_wait_time': 0
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks from JSON with comprehensive parsing
        """
        print_to_console(f"Attempting to load tasks from: {json_path}")

        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),  # Default 10 MB
                cpu_required=task_dict.get('instructions', 50000),  # Default 50,000 MI
                task_details=task_dict
            )
            task.arrival_time = self.current_time
            tasks.append(task)

        print_to_console(f"Loaded {len(tasks)} tasks from JSON")
        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources with advanced visualization
        """
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin task distribution with visualization
        resource_index = 0
        for task in tasks:
            # Select resource
            resource = self.resources[resource_index]

            # Enqueue task
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1

            # Cycle through resources
            resource_index = (resource_index + 1) % len(self.resources)

        # Update metrics
        self.metrics['task_distribution'] = task_distribution
        self.metrics['queued_tasks'] = sum(len(resource.task_queue) for resource in self.resources)

        # Print initial distribution
        print_to_console("\n--- Initial Task Distribution ---")
        for resource_type, count in task_distribution.items():
            print_to_console(f"{resource_type}: {count} tasks")

        print_to_console("\n--- Resource Queue Lengths ---")
        for i, resource in enumerate(self.resources, 1):
            print_to_console(f"Resource {i} ({resource.type}) Queue Length: {len(resource.task_queue)} tasks")

    def run_simulation(self, max_iterations: int = 1000):
        """
        Run scheduling simulation with real-time visualization
        """
        # Distribute tasks initially
        self.distribute_tasks()

        # Simulation loop with enhanced visualization
        start_time = time.time()
        for iteration in range(max_iterations):
            print_to_console(f"\n--- Iteration {iteration} ---")

            # Process queues for all resources
            completed_in_iteration = 0
            resource_utilization = {}

            for resource in self.resources:
                # Track resource utilization
                initial_completed = len(resource.completed_tasks)
                resource.process_queue(self.current_time)
                completed_this_resource = len(resource.completed_tasks) - initial_completed
                completed_in_iteration += completed_this_resource

                # Calculate resource utilization
                resource_utilization[resource.type] = {
                    'completed_tasks': completed_this_resource,
                    'current_tasks': len(resource.current_tasks),
                    'queue_length': len(resource.task_queue)
                }

            # Update metrics
            self.metrics['completed_tasks'] = sum(
                len(resource.completed_tasks) for resource in self.resources
            )
            self.metrics['resource_utilization'] = resource_utilization

            # Print real-time resource utilization
            print_to_console("\n--- Resource Utilization ---")
            for resource_type, stats in resource_utilization.items():
                print_to_console(
                    f"{resource_type}: "
                    f"Completed: {stats['completed_tasks']}, "
                    f"Current Tasks: {stats['current_tasks']}, "
                    f"Queue Length: {stats['queue_length']}"
                )

            # Check if all tasks are processed
            if self.metrics['completed_tasks'] == self.metrics['total_tasks']:
                print_to_console("\n--- All Tasks Processed! ---")
                break

            # Increment time
            self.current_time += 1

            # Optional: Add a small delay to simulate real-time processing
            time.sleep(0.1)

        # Calculate total processing time
        total_processing_time = time.time() - start_time
        self.metrics['total_processing_time'] = total_processing_time

        # Calculate wait time metrics
        self.calculate_wait_time_metrics()

        return self.metrics

    def calculate_wait_time_metrics(self):
        """
        Calculate comprehensive wait time metrics
        """
        all_completed_tasks = []
        for resource in self.resources:
            all_completed_tasks.extend(resource.completed_tasks)

        if all_completed_tasks:
            wait_times = [task.wait_time for task in all_completed_tasks]
            self.metrics['average_wait_time'] = sum(wait_times) / len(wait_times)
            self.metrics['max_wait_time'] = max(wait_times)

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Explicitly set print to console
    print = print_to_console

    # Create resources matching original configuration
    resources = create_original_resources()

    # Print initial resource details
    print("\n--- Resource Configurations ---")
    for resource in resources:
        print(f"Resource {resource.id} ({resource.type}):")
        print(f"  CPU Rating: {resource.cpu_rating} MI/s")
        print(f"  Memory: {resource.memory} GB")
        print(f"  Bandwidth: {resource.bandwidth} MB/s")

    # Initialize scheduler
    scheduler = AdvancedFCFSScheduler(resources)

    # Run simulation
    metrics = scheduler.run_simulation()

    # Print final detailed metrics
    print("\n--- Final Scheduling Metrics ---")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    # Detailed resource reporting
    print("\n--- Final Resource Status ---")
    for resource in scheduler.resources:
        print(f"\nResource {resource.id} ({resource.type}):")
        print(f"Completed Tasks: {len(resource.completed_tasks)}")
        print(f"Remaining Queue: {len(resource.task_queue)}")

if __name__ == "__main__":
    main()



--- Resource Configurations ---
Resource 1 (Edge_Raspberry_Pi):
  CPU Rating: 80000 MI/s
  Memory: 1 GB
  Bandwidth: 5 MB/s
Resource 2 (Edge_Smartphone):
  CPU Rating: 400000 MI/s
  Memory: 4 GB
  Bandwidth: 20 MB/s
Resource 3 (Cloud_Host):
  CPU Rating: 1000000 MI/s
  Memory: 32 GB
  Bandwidth: 80 MB/s
Attempting to load tasks from: /content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'

Using Rich in FCFS

In [ ]:
import json
import numpy as np
from typing import List, Dict, Any
import logging
import sys
import time
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 task_type: str,
                 data_size: float,     # in GB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.type = task_type
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = f"{task_type}_Task_{task_id}"

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with comprehensive tracking and utilization metrics
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int,     # in MB/s
                 num_cpus: int = 1):  # Number of CPUs, defaulting to 1
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.total_memory = memory
        self.bandwidth = bandwidth

        # CPU configuration
        self.num_cpus = num_cpus
        self.available_cpus = num_cpus

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Utilization tracking
        self.current_cpu_usage = 0
        self.current_memory_usage = 0

        # Additional tracking
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

    def can_process_task(self, task: Task) -> bool:
        """
        Check if the resource can process the given task
        """
        # Check if CPUs are available
        if self.available_cpus <= 0:
            return False

        # Cloud host can process all task types
        if self.type == "Cloud_Host":
            return True

        # Edge nodes (Raspberry Pi and Smartphone) can only process RT2 tasks
        if self.type in ["Edge_Raspberry_Pi", "Edge_Smartphone"]:
            # Explicitly fail RT1 and RT3 tasks on edge resources
            if task.type in ["RT1", "RT3"]:
                logger.warning(f"Task {task.id} of type {task.type} FAILED on {self.type}")
                return False

            # Additional memory check for RT2 tasks
            if task.data_size > self.total_memory:
                logger.warning(f"Task {task.id} requires {task.data_size} GB, exceeding {self.type}'s memory of {self.total_memory} GB")
                return False

        return True

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue if it can be processed
        """
        if self.can_process_task(task):
            task.queue_position = len(self.task_queue)
            task.arrival_time = time.time()
            self.task_queue.append(task)
        else:
            # Mark task as failed
            task.status = 'failed'

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in the queue with detailed tracking and utilization update
        """
        # Reset current usage and task tracking
        self.current_cpu_usage = 0
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

        # Reset available CPUs
        self.available_cpus = self.num_cpus

        # Process current tasks first
        for task in self.current_tasks[:]:
            # Skip if no CPUs available
            if self.available_cpus <= 0:
                break

            # Determine how much CPU can be used for this task
            task_cpu = min(self.cpu_rating, task.remaining_cpu)

            processing_result = task.process(task_cpu)

            # Update CPU usage
            processed_amount = processing_result['processed']
            self.current_cpu_usage += processed_amount

            # Track detailed task information
            task_info = {
                'id': task.id,
                'name': task.task_name,
                'type': task.type,
                'processed': processed_amount,
                'total_required': task.total_cpu_required,
                'completion_percentage': processing_result['completion_percentage']
            }
            self.detailed_task_tracking.append(task_info)

            # Calculate task CPU demand
            task_demand = processed_amount / self.cpu_rating
            self.task_cpu_demands.append(task_demand)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)
                # Free up a CPU
                self.available_cpus += 1

        # Move tasks from queue to current tasks if CPUs are available
        while self.task_queue and self.available_cpus > 0:
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)
            # Use up a CPU
            self.available_cpus -= 1

        # Calculate CPU utilization
        if self.task_cpu_demands:
            cpu_utilization = min(sum(self.task_cpu_demands) * 100, 100)
        else:
            cpu_utilization = 0

        # Estimate memory usage
        self.current_memory_usage = len(self.current_tasks) * (self.total_memory / 10)
        memory_utilization = min((self.current_memory_usage / self.total_memory) * 100, 100)

        # Return detailed resource state with utilization
        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'cpu_utilization': cpu_utilization,
            'memory_utilization': memory_utilization,
            'raw_cpu_usage': self.current_cpu_usage,
            'task_demands': self.task_cpu_demands,
            'detailed_tasks': self.detailed_task_tracking,
            'available_cpus': self.available_cpus
        }

def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Edge Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Edge_{i}",
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=20         # 20 MB/s
            )
        )

    # Create 5 Raspberry Pi Edge Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=5          # 5 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=80         # 80 MB/s
            )
        )

    return resources

class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        self.console = Console()

        # Poisson process parameters
        self.arrival_rate = 0.8  # λ = 0.8 tasks per second

        # Metrics tracking
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'failed_tasks': 0,
            'task_distribution': {},
            'makespan': 0,
            'throughput': 0
        }

    def generate_tasks(self, simulation_time: float = 100) -> List[Task]:
        """
        Generate tasks using Poisson process
        """
        # Task types and their characteristics based on the paper
        task_types = [
            # Read Tasks
            {"type": "RT1", "data_size": 5.0, "cpu_required": 2_000_000},   # CPU-intensive, memory-intensive
            {"type": "RT2", "data_size": 0.2, "cpu_required": 4_000_000},   # CPU-intensive, memory-light
            {"type": "RT3", "data_size": 5.0, "cpu_required": 200_000},     # CPU-light, memory-intensive
            {"type": "RT4", "data_size": 0.5, "cpu_required": 500_000}      # CPU-light, memory-light
        ]

        # Generate task arrival times using Poisson process
        num_tasks = np.random.poisson(self.arrival_rate * simulation_time)

        tasks = []
        for i in range(num_tasks):
            # Randomly select task type
            task_type_data = np.random.choice(task_types)

            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )

            # Set arrival time
            task.arrival_time = np.random.uniform(0, simulation_time)

            tasks.append(task)

        # Sort tasks by arrival time
        return sorted(tasks, key=lambda x: x.arrival_time)

    def distribute_tasks(self, total_tasks: int = 1500) -> List[Task]:
        """
        Generate and distribute a fixed number of tasks
        """
        # Task types and their characteristics based on the paper
        task_types = [
            # Read Tasks
            {"type": "RT1", "data_size": 5.0, "cpu_required": 2_000_000},   # CPU-intensive, memory-intensive
            {"type": "RT2", "data_size": 0.2, "cpu_required": 4_000_000},   # CPU-intensive, memory-light
            {"type": "RT3", "data_size": 5.0, "cpu_required": 200_000},     # CPU-light, memory-intensive
            {"type": "RT4", "data_size": 0.5, "cpu_required": 500_000}      # CPU-light, memory-light
        ]

        # Generate tasks
        tasks = []
        for i in range(total_tasks):
            # Randomly select task type
            task_type_data = np.random.choice(task_types)

            task = Task(
                task_id=i+1,
                task_type=task_type_data['type'],
                data_size=task_type_data['data_size'],
                cpu_required=task_type_data['cpu_required']
            )

            # Set arrival time with uniform distribution
            task.arrival_time = np.random.uniform(0, 100)  # Distribute over 100 seconds

            tasks.append(task)

        # Sort tasks by arrival time
        tasks.sort(key=lambda x: x.arrival_time)

        # Update metrics
        self.metrics['total_tasks'] = total_tasks

        # Explicitly select edge, raspberry, and cloud resources
        edge_resources = [r for r in self.resources if r.type.startswith("Edge_")]
        raspberry_resources = [r for r in self.resources if r.type.startswith("Raspberry_")]
        cloud_resources = [r for r in self.resources if r.type.startswith("Cloud_")]

        # Define resource order as specified in the paper
        # Order: Smartphone, Raspberry Pi, Cloud
        resource_order = edge_resources + raspberry_resources + cloud_resources

        # Track task distribution and failures
        task_distribution = {resource.type: 0 for resource in self.resources}
        failed_tasks_by_resource = {resource.type: 0 for resource in self.resources}

        # Circular resource selection
        resource_index = 0
        num_resources = len(resource_order)

        for task in tasks:
            # Select resource in the specified order
            resource = resource_order[resource_index]

            # Attempt to enqueue task
            if resource.can_process_task(task):
                resource.enqueue_task(task)
                task_distribution[resource.type] += 1
            else:
                # Increment failed tasks for the specific resource type
                failed_tasks_by_resource[resource.type] += 1
                self.metrics['failed_tasks'] += 1

            # Move to next resource in circular manner
            resource_index = (resource_index + 1) % num_resources

        self.metrics['task_distribution'] = task_distribution

        # Print distribution table
        distribution_table = Table(title="Task Distribution")
        distribution_table.add_column("Resource", style="cyan")
        distribution_table.add_column("Tasks", style="magenta")
        distribution_table.add_column("Failed Tasks", style="red")

        for resource_type in task_distribution:
            distribution_table.add_row(
                resource_type,
                str(task_distribution[resource_type]),
                str(failed_tasks_by_resource[resource_type])
            )

        self.console.print(distribution_table)

        return tasks
    def run_simulation(self, total_tasks: int = 1500, max_iterations: int = 10000):
            """
            Run simulation with a fixed number of tasks
            """
            # Record start time
            self.start_time = time.time()

            # Distribute tasks
            tasks = self.distribute_tasks(total_tasks)

            # Prepare layout for live visualization
            layout = Layout()
            layout.split_row(
                Layout(name="resource1"),
                Layout(name="resource2"),
                Layout(name="resource3")
            )

            # Live visualization
            with Live(layout, console=self.console, refresh_per_second=10) as live:
                for iteration in range(max_iterations):
                    # Calculate current simulation time
                    current_simulation_time = time.time() - self.start_time

                    # Process tasks on each resource
                    for i, resource in enumerate(self.resources, 1):
                        # Process resource queue
                        resource_status = resource.process_queue(current_simulation_time)

                        # Update layout with resource-specific panel
                        layout[f"resource{i}"].update(
                            self._create_resource_panel(resource, resource_status)
                        )

                    # Update live display
                    live.update(layout)

                    # Track completed and failed tasks across all resources
                    total_processed_tasks = sum(
                        len(resource.completed_tasks) for resource in self.resources
                    ) + self.metrics['failed_tasks']

                    # Check if all tasks are processed (completed or failed)
                    if total_processed_tasks >= total_tasks:
                        logger.info(f"Simulation completed in {iteration} iterations")
                        break

                    time.sleep(0.1)

            # Calculate final metrics
            completed_tasks = sum(len(resource.completed_tasks) for resource in self.resources)
            self.metrics['completed_tasks'] = completed_tasks
            self.metrics['makespan'] = time.time() - self.start_time

            return self.metrics
    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with utilization metrics
        """
        # Create table for resource details
        table = Table(show_header=False)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['cpu_utilization']:.2f}% "
            f"({status['raw_cpu_usage']:.2f}/{resource.cpu_rating} MI/s)"
        )

        # Detailed task tracking
        if status['detailed_tasks']:
            table.add_row("\n[bold]Current Tasks[/bold]")
            for task in status['detailed_tasks']:
                table.add_row(                f"[blue]Task {task['id']} ({task['type']}):[/blue] "
                f"Processed {task['processed']:.2f}/{task['total_required']} MI "
                f"({task['completion_percentage']:.2f}%)"
                )

        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['memory_utilization']:.2f}% "
            f"({resource.current_memory_usage:.2f}/{resource.total_memory} GB)"
        )

        # Task processing status
        table.add_row("\n[bold]Task Processing[/bold]")
        table.add_row(f"[green]Completed Tasks:[/green] {status['completed_tasks']}")
        table.add_row(f"[yellow]Current Tasks:[/yellow] {status['current_tasks']}")
        table.add_row(f"[red]Queue Length:[/red] {status['queue_length']}")

        # Create panel with resource-specific styling
        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )

def create_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    """
    Main simulation entry point
    """
    # Create resources
    resources = create_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    # Run simulation
    try:
        # Run simulation for 1500 tasks
        metrics = scheduler.run_simulation(total_tasks=1500)

        # Print final metrics
        print("\n--- Simulation Metrics ---")
        print(f"Total Tasks Generated: {metrics['total_tasks']}")
        print(f"Completed Tasks: {metrics['completed_tasks']}")
        print(f"Failed Tasks: {metrics['failed_tasks']}")
        print("\nTask Distribution:")
        for resource_type, count in metrics['task_distribution'].items():
            print(f"{resource_type}: {count}")

        print("\nPerformance Metrics:")
        print(f"Makespan: {metrics['makespan']:.2f} seconds")

    except Exception as e:
        logger.error(f"Simulation failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


             Task Distribution              
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Resource          ┃ Tasks ┃ Failed Tasks ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ Edge_Raspberry_Pi │ 253   │ 247          │
│ Edge_Smartphone   │ 265   │ 235          │
│ Cloud_Host        │ 500   │ 0            │
└───────────────────┴───────┴──────────────┘

Output()

KeyboardInterrupt: 

With CPU and Utilization output included

In [ ]:
import json
from typing import List, Dict, Any
import logging
import sys
import time
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
from rich.progress import Progress, SpinnerColumn, TextColumn, BarColumn, TaskProgressColumn

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

class Task:
    """
    Detailed task representation with advanced tracking
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

        # Queuing attributes
        self.wait_time = 0
        self.queue_position = None

        # Additional metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')
        self.task_class = self.details.get('task_class', 'generic')
        self.cpu_intensity = self.details.get('cpu_intensity', 'medium')

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with comprehensive tracking and utilization metrics
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.total_memory = memory
        self.bandwidth = bandwidth

        # Task management
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Utilization tracking
        self.current_cpu_usage = 0
        self.current_memory_usage = 0

        # Additional tracking for more nuanced CPU utilization
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in the queue with detailed tracking and utilization update
        """
        # Reset current usage and task tracking
        self.current_cpu_usage = 0
        self.task_cpu_demands = []
        self.detailed_task_tracking = []

        # Calculate available CPU for this time step
        available_cpu = self.cpu_rating

        # Process current tasks first
        for task in self.current_tasks[:]:
            # Determine how much CPU can be used for this task
            task_cpu = min(available_cpu, task.remaining_cpu)

            processing_result = task.process(task_cpu)

            # Update CPU usage and available CPU
            processed_amount = processing_result['processed']
            self.current_cpu_usage += processed_amount
            available_cpu -= processed_amount

            # Track detailed task information
            task_info = {
                'id': task.id,
                'name': task.task_name,
                'processed': processed_amount,
                'total_required': task.total_cpu_required,
                'completion_percentage': processing_result['completion_percentage']
            }
            self.detailed_task_tracking.append(task_info)

            # Calculate task CPU demand
            task_demand = processed_amount / self.cpu_rating
            self.task_cpu_demands.append(task_demand)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

            # Stop processing if no CPU left
            if available_cpu <= 0:
                break

        # Calculate CPU utilization
        # Use sum of task CPU demands to get a more dynamic representation
        if self.task_cpu_demands:
            cpu_utilization = min(sum(self.task_cpu_demands) * 100, 100)
        else:
            cpu_utilization = 0

        # Estimate memory usage (simple model: each current task uses some memory)
        self.current_memory_usage = len(self.current_tasks) * (self.total_memory / 10)
        memory_utilization = min((self.current_memory_usage / self.total_memory) * 100, 100)

        # If resource has available capacity, move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:  # Limit concurrent tasks
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time
            next_task.wait_time = current_time - next_task.arrival_time

            self.current_tasks.append(next_task)

        # Return detailed resource state with utilization
        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'cpu_utilization': cpu_utilization,
            'memory_utilization': memory_utilization,
            'raw_cpu_usage': self.current_cpu_usage,
            'task_demands': self.task_cpu_demands,
            'detailed_tasks': self.detailed_task_tracking
        }

class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        self.console = Console()

        # Metrics tracking
        self.metrics = {
            'total_tasks': 0,
            'task_distribution': {},
            'resource_status': {}
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks from JSON
        """
        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),
                cpu_required=task_dict.get('instructions', 50000),
                task_details=task_dict
            )
            task.arrival_time = self.current_time
            tasks.append(task)

        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources
        """
        # Load tasks
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin distribution
        resource_index = 0
        for task in tasks:
            resource = self.resources[resource_index]
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1
            resource_index = (resource_index + 1) % len(self.resources)

        self.metrics['task_distribution'] = task_distribution

        # Print distribution table
        distribution_table = Table(title="Task Distribution")
        distribution_table.add_column("Resource", style="cyan")
        distribution_table.add_column("Tasks", style="magenta")

        for resource_type, count in task_distribution.items():
            distribution_table.add_row(resource_type, str(count))

        self.console.print(distribution_table)

    def run_simulation(self, max_iterations: int = 10000):
        """
        Run simulation with a stopping criterion similar to the provided code
        """
        # Distribute tasks
        self.distribute_tasks()

        # Total number of tasks
        total_tasks = self.metrics['total_tasks']

        # Prepare layout for live visualization
        layout = Layout()
        layout.split_row(
            Layout(name="resource1"),
            Layout(name="resource2"),
            Layout(name="resource3")
        )

        # Live visualization
        with Live(layout, console=self.console, refresh_per_second=10) as live:
            for iteration in range(max_iterations):
                self.current_time += 1

                # Process tasks on each resource
                for i, resource in enumerate(self.resources, 1):
                    # Process resource queue
                    resource_status = resource.process_queue(self.current_time)

                    # Update layout with resource-specific panel
                    layout[f"resource{i}"].update(
                        self._create_resource_panel(resource, resource_status)
                    )

                # Update live display
                live.update(layout)

                # Custom stopping criterion similar to the provided code
                provisioned_tasks = sum(
                    len(resource.completed_tasks) for resource in self.resources
                )

                # Stop when all tasks are provisioned (completed)
                if provisioned_tasks == total_tasks:
                    logger.info(f"Simulation completed in {iteration} iterations")
                    break

                time.sleep(0.1)

        return self.metrics

    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with utilization metrics
        """
        # Create table for resource details
        table = Table(show_header=False)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['cpu_utilization']:.2f}% "
            f"({status['raw_cpu_usage']:.2f}/{resource.cpu_rating} MI/s)"
        )

        # Show individual task demands for more insight
        if status['task_demands']:
            demands_str = ", ".join([f"{d*100:.2f}%" for d in status['task_demands']])
            table.add_row(f"[yellow]Task Demands:[/yellow] {demands_str}")

        # Detailed task tracking
        if status['detailed_tasks']:
            table.add_row("\n[bold]Current Tasks[/bold]")
            for task in status['detailed_tasks']:
                table.add_row(
                    f"[blue]Task {task['id']} ({task['name']}):[/blue] "
                    f"{task['processed']:.2f}/{task['total_required']} MI "
                    f"({task['completion_percentage']:.2f}%)"
                )

        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['memory_utilization']:.2f}% "
            f"({resource.current_memory_usage:.2f}/{resource.total_memory} GB)"
        )

        # Task processing status
        table.add_row("\n[bold]Task Processing[/bold]")
        table.add_row(f"[green]Completed Tasks:[/green] {status['completed_tasks']}")
        table.add_row(f"[yellow]Current Tasks:[/yellow] {status['current_tasks']}")
        table.add_row(f"[red]Queue Length:[/red] {status['queue_length']}")

        # Create panel with resource-specific styling
        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Create resources
    resources = create_original_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    # Run simulation
    metrics = scheduler.run_simulation()

if __name__ == "__main__":
    main()


Mounted at /content/drive

📂 Available log files in EdgeSimPy/logs:
simulation.log


Output()

KeyboardInterrupt: 

Using Round Robin Algorithm


In [ ]:
import json
import numpy as np
from typing import List, Dict, Any, Optional, Tuple  # Added Tuple
import logging
import sys
import time
import random
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
import logging
import threading
from google.colab import drive
import os
from rich import box  # This is the import we need
from datetime import datetime, timedelta
import csv
import matplotlib.pyplot as plt
import collections
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import functools
import copy
import itertools

# Mount Google Drive
drive.mount('/content/drive')

# Define the log directory in Google Drive
log_dir = "/content/drive/My Drive/EdgeSimPy/logs"

# Create the directory if it does not exist
os.makedirs(log_dir, exist_ok=True)

# Generate a log filename with current date and time
log_filename = datetime.now().strftime("simulation_%Y-%m-%d_%H-%M-%S.log")
full_log_path = os.path.join(log_dir, log_filename)

# List existing log files in the directory (optional)
try:
    print("\n📂 Available log files in EdgeSimPy/logs:")
    for filename in os.listdir(log_dir):
        print(filename)
except Exception as e:
    print(f"Error listing log files: {e}")

# Configure logging to write to both console and file
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG to capture more detailed logs
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),  # Output to console
        logging.FileHandler(full_log_path)  # Save logs to dated file in Google Drive
    ]
)

# Create a logger instance
logger = logging.getLogger(__name__)


class Task:
    def __init__(self,
                 task_id: int,
                 task_type: str,
                 input_size: float,    # Input data size in GB
                 output_size: float,   # Output data size in GB
                 cpu_required: float): # Total MI required
        # Basic identification
        self.id = task_id
        self.type = task_type
        # Resource requirements
        self.input_size = input_size
        self.output_size = output_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Timing metrics (CloudSim-style)
        self.arrival_time = datetime.now().timestamp()  # Set arrival time immediately
        self.start_time = None    # Will be set when actual processing begins
        self.completion_time = None
        self.exec_start_time = None
        self.actual_exec_time = 0.0     # Initialize as float
        self.turnaround_time = 0.0      # Initialize as float
        self.waiting_time = 0.0         # Initialize as float

        # Status tracking
        self.status = 'CREATED'
        self.completion_percentage = 0.0
        self.assigned_resource = None

        # Progress tracking
        self.transferred_input = 0.0
        self.transferred_output = 0.0
        self.processed_mi = 0.0

        # Tracking flags
        self._time_initialized = False
        self.last_update_time = None
    def calculate_timing_metrics(self):
        """Calculate final timing metrics when task completes"""
        if self.completion_time and self.arrival_time:
            # Total time from arrival to completion
            self.turnaround_time = self.completion_time - self.arrival_time

            # Time spent in actual processing (excluding transfers)
            if self.exec_start_time:
                self.actual_exec_time = self.completion_time - self.exec_start_time
            else:
                self.actual_exec_time = self.completion_time - self.start_time

            # Waiting time is turnaround time minus execution time
            self.waiting_time = self.turnaround_time - self.actual_exec_time

            # Ensure we don't have negative times
            self.turnaround_time = max(0, self.turnaround_time)
            self.actual_exec_time = max(0, self.actual_exec_time)
            self.waiting_time = max(0, self.waiting_time)
    def estimate_execution_time(self, resource) -> float:
        """
        Calculate execution time considering separate input and output transfers
        """
        # Input transfer time (for read tasks)
        input_mb = self.input_size * 1024
        input_transfer_time = input_mb / resource.total_bandwidth if input_mb > 0 else 0

        # Processing time
        processing_time = self.total_cpu_required / resource.total_cpu_rating

        # Output transfer time (for write tasks)
        output_mb = self.output_size * 1024
        output_transfer_time = output_mb / resource.total_bandwidth if output_mb > 0 else 0

        total_time = input_transfer_time + processing_time + output_transfer_time
        return total_time

    def update_progress(self, resource, current_time: float) -> Dict:
        """
        Update task progress with fixed time step calculations and enhanced timing metrics

        This method handles the complete lifecycle of a task, including:
        - Input data transfer
        - Processing
        - Output data transfer
        - Timing metrics calculation
        """
        # Log initial state for debugging
        logger.info(f"Task {self.id} UPDATE: Current state={self.status}, Time={current_time:.2f}")

        # SECTION 1: Initialize timing tracking
        if not self._time_initialized:
            logger.info(f"Task {self.id} INIT: First update at time {current_time:.2f}")
            self._time_initialized = True
            self.last_update_time = current_time

        # SECTION 2: Calculate time steps for this update
        elapsed = current_time - self.last_update_time  # Time since last update
        steps = max(1, int(elapsed / 1.0))  # Divide into 1-second steps
        time_per_step = elapsed / steps     # Actual time per step

        logger.info(f"Task {self.id} STEP: Time since last update={elapsed:.3f}s, Steps={steps}")

        # Calculate bandwidth in GB/s (convert from Mb/s)
        step_bandwidth = (resource.total_bandwidth / 8.0) / 1024.0

        # SECTION 3: State Machine - Handle different task states
        old_status = self.status  # Track state transitions

        if self.status == 'CREATED':
            # Initial state: Set up task for processing
            self.status = 'READY'
            self.transferred_input = 0.0
            self.transferred_output = 0.0
            self.processed_mi = 0.0
            if not self.start_time:  # Only set start_time if not already set
                self.start_time = current_time  # Record when task starts
                logger.info(f"Task {self.id} TRANSITION: CREATED → READY at {current_time:.2f}")

        elif self.status == 'READY':
            # Determine next phase based on input requirements
            if self.input_size > 0:
                self.status = 'TRANSFERRING_INPUT'
                logger.info(f"Task {self.id} TRANSITION: READY → TRANSFERRING_INPUT (input_size={self.input_size}GB)")
            else:
                self.status = 'PROCESSING'
                self.exec_start_time = current_time  # Record processing start
                logger.info(f"Task {self.id} TRANSITION: READY → PROCESSING (no input transfer needed)")
            self.completion_percentage = 0

        elif self.status == 'TRANSFERRING_INPUT':
            # Calculate input data transfer for this time step
            step_transfer = step_bandwidth * time_per_step * steps
            old_transfer = self.transferred_input
            self.transferred_input = min(self.input_size, self.transferred_input + step_transfer)

            logger.info(f"Task {self.id} TRANSFER: Input {old_transfer:.3f}GB → {self.transferred_input:.3f}GB of {self.input_size:.3f}GB")

            if self.transferred_input >= self.input_size:
                # Input transfer complete, start processing
                self.status = 'PROCESSING'
                self.exec_start_time = current_time  # Record actual processing start time
                self.completion_percentage = 0
                logger.info(f"Task {self.id} TRANSITION: TRANSFERRING_INPUT → PROCESSING at {current_time:.2f}")
            else:
                self.completion_percentage = (self.transferred_input / self.input_size) * 100

        elif self.status == 'PROCESSING':
            # Calculate processing progress
            step_processing = resource.total_cpu_rating * time_per_step * steps
            old_processed = self.processed_mi
            self.processed_mi = min(self.total_cpu_required, self.processed_mi + step_processing)
            self.remaining_cpu = self.total_cpu_required - self.processed_mi

            logger.info(f"Task {self.id} PROCESSING: {old_processed:.0f}MI → {self.processed_mi:.0f}MI of {self.total_cpu_required:.0f}MI")

            if self.processed_mi >= self.total_cpu_required:
                # Processing complete, check if output transfer needed
                if self.output_size > 0:
                    self.status = 'TRANSFERRING_OUTPUT'
                    self.completion_percentage = 0
                    logger.info(f"Task {self.id} TRANSITION: PROCESSING → TRANSFERRING_OUTPUT (output_size={self.output_size}GB)")
                else:
                    # No output transfer needed, task is complete
                    self.status = 'COMPLETED'
                    self.completion_time = current_time
                    self.calculate_timing_metrics()
                    self.completion_percentage = 100
                    logger.info(f"Task {self.id} TRANSITION: PROCESSING → COMPLETED at {current_time:.2f}")
            else:
                self.completion_percentage = (self.processed_mi / self.total_cpu_required) * 100

        elif self.status == 'TRANSFERRING_OUTPUT':
            # Calculate output data transfer for this time step
            step_transfer = step_bandwidth * time_per_step * steps
            old_transfer = self.transferred_output
            self.transferred_output = min(self.output_size, self.transferred_output + step_transfer)

            logger.info(f"Task {self.id} TRANSFER: Output {old_transfer:.3f}GB → {self.transferred_output:.3f}GB of {self.output_size:.3f}GB")

            if self.transferred_output >= self.output_size:
                # Output transfer complete, task is finished
                self.status = 'COMPLETED'
                self.completion_time = current_time
                self.calculate_timing_metrics()
                self.completion_percentage = 100
                logger.info(f"Task {self.id} TRANSITION: TRANSFERRING_OUTPUT → COMPLETED at {current_time:.2f}")
            else:
                self.completion_percentage = (self.transferred_output / self.output_size) * 100

        # Log state transition if it occurred
        if old_status != self.status:
            logger.info(f"Task {self.id} STATE CHANGE: {old_status} → {self.status}")

        # SECTION 4: Update timing and return status
        self.last_update_time = current_time

        # Calculate current execution time
        current_exec_time = current_time - (self.start_time or current_time)

        # Final progress report for this update
        logger.info(f"Task {self.id} PROGRESS: Status={self.status}, Completion={self.completion_percentage:.1f}%, ExecTime={current_exec_time:.2f}s")

        return {
            'status': self.status,
            'progress': self.completion_percentage,
            'exec_time': current_exec_time
        }
    def calculate_final_metrics(self):
        """
        Calculate final timing metrics when task completes
        """
        if self.completion_time and self.arrival_time and self.exec_start_time:
            # Turnaround time: time from arrival to completion
            self.turnaround_time = self.completion_time - self.arrival_time

            # Actual execution time: time spent processing (excluding transfers)
            self.actual_exec_time = self.completion_time - self.exec_start_time

            # Waiting time: turnaround time minus actual execution time
            self.waiting_time = self.turnaround_time - self.actual_exec_time
    def update_execution(self, resource, time_step: float) -> bool:
            """
            Process task as a single unit, tracking overall progress.
            Returns True if task is completed.
            """
            if self.status == 'CREATED':
                self.status = 'RUNNING'
                self.start_time = datetime.now().timestamp()

            elif self.status == 'RUNNING':
                # Calculate progress including both transfer and processing
                progress = (time_step * resource.total_cpu_rating)
                self.remaining_cpu -= progress

                if self.remaining_cpu <= 0:
                    self.status = 'COMPLETED'
                    self.completion_time = datetime.now().timestamp()
                    self.execution_time = self.completion_time - self.start_time
                    return True

            return False
    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Log task processing details
        logger.info(f"Processing task {self.id} on resource. Remaining CPU: {self.remaining_cpu}")

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = datetime.now().timestamp()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }
class Resource:
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # MIPS
                 memory: int,        # GB
                 bandwidth: int):    # Mb/s
        # Resource identification
        self.id = resource_id
        self.type = resource_type

        # Resource capabilities
        self.total_cpu_rating = cpu_rating
        self.total_memory = memory
        self.total_bandwidth = bandwidth

        # Task tracking
        self.task_queue = []
        self.current_task = None
        self.completed_tasks = []
        self.failed_tasks = []
        self.failed_tasks_count = 0

        # Resource utilization tracking
        self.used_cpu = 0
        self.used_memory = 0
        self.current_load = 0.0
        # Add reliability attribute
        # You can assign different reliability values based on resource type
        if resource_type.startswith("Cloud_"):
            self.reliability = 0.95  # High reliability for cloud resources
        elif resource_type.startswith("Smartphone_"):
            self.reliability = 0.8   # Moderate reliability for smartphones
        elif resource_type.startswith("Raspberry_"):
            self.reliability = 0.4   # Lower reliability for Raspberry Pi
        else:
            self.reliability = 0.5   # Default reliability
    def calculate_resource_utilization(self):
        """
        Debug resource utilization
        """
        # First, log the current state
        task_info = "No task"
        if self.current_task:
            task_info = f"Task {self.current_task.id} ({self.current_task.type}) in state {self.current_task.status}"
            # Add extensive task state logging
            logger.info(f"""
            CURRENT TASK STATE:
            Task ID: {self.current_task.id}
            Type: {self.current_task.type}
            Status: {self.current_task.status}
            Progress: {self.current_task.completion_percentage}%
            Input Size: {self.current_task.input_size} GB
            Output Size: {self.current_task.output_size} GB
            Processed MI: {self.current_task.processed_mi} / {self.current_task.total_cpu_required}
            Input Transfer: {self.current_task.transferred_input} / {self.current_task.input_size} GB
            Output Transfer: {self.current_task.transferred_output} / {self.current_task.output_size} GB
            """)

        logger.info(f"Resource {self.id} ({self.type}) - {task_info}")

        # Default values
        cpu_util = 0
        mem_util = 0
        bw_util = 0

        # Super simple utilization based purely on state
        if self.current_task:
            status = self.current_task.status

            # Debug status transitions
            logger.info(f"Current status: {status}")

            if status == 'PROCESSING':
                cpu_util = 100  # Full CPU during processing
                logger.info(f"Setting CPU utilization to 100% for PROCESSING state")

            if status in ['TRANSFERRING_INPUT', 'TRANSFERRING_OUTPUT']:
                bw_util = 100  # Full bandwidth during transfers
                logger.info(f"Setting bandwidth utilization to 100% for {status}")

            # Set memory utilization based on data presence
            if self.current_task.input_size > 0 or self.current_task.output_size > 0:
                mem_util = 50  # Using 50% as a simple indicator
                logger.info(f"Setting memory utilization to 50% due to data presence")

        # Log final values
       # logger.info(f"""
       # FINAL UTILIZATION VALUES:
       #CPU: {cpu_util}%
       # Memory: {mem_util}%
       # Bandwidth: {bw_util}%
       # """)

        return {
            'cpu_utilization': cpu_util,
            'memory_utilization': mem_util,
            'bandwidth_utilization': bw_util,
            'overall_utilization': (cpu_util + mem_util + bw_util) / 3,
            'raw_cpu_usage': (cpu_util / 100.0) * self.total_cpu_rating,
            'raw_memory_usage': (mem_util / 100.0) * self.total_memory * 1024,
            'raw_bandwidth_usage': (bw_util / 100.0) * self.total_bandwidth,
            'active_tasks': 1 if self.current_task else 0,
            'transfer_phase': self.current_task.status if self.current_task else 'NONE'
        }
    def _get_zero_utilization(self):
        """Helper method to return zero utilization state"""
        return {
            'cpu_utilization': 0,
            'memory_utilization': 0,
            'bandwidth_utilization': 0,
            'overall_utilization': 0,
            'raw_cpu_usage': 0,
            'raw_memory_usage': 0,
            'raw_bandwidth_usage': 0,
            'active_tasks': 0,
            'transfer_phase': 'NONE'
        }
    def can_process_task(self, task: Task) -> Tuple[bool, str]:
        """
        Check if resource can process a given task based on task type and resource type.
        Only RT1 and RT3 tasks are restricted from Smartphone and Raspberry Pi resources.
        """
        # Cloud resources can process all tasks
        if self.type.startswith("Cloud_"):
            return True, ""

        # For Smartphone and Raspberry Pi resources
        if self.type.startswith(("Smartphone_", "Raspberry_")):
            return True, ""

        # All other combinations are valid
        return True, ""

    def process_task(self, resource, current_time: float) -> Dict:
        """
        Process task as a single unit, ensuring proper completion.
        """
        status = {
            'processed': False,
            'completed': False,
            'task_id': self.id,
            'progress': self.completion_percentage
        }

        if self.status == 'CREATED':
            self.status = 'RUNNING'
            self.start_time = current_time
            status['processed'] = True

        elif self.status == 'RUNNING':
            # Calculate progress including both transfer and processing
            time_step = current_time - self.start_time
            progress = (time_step * resource.total_cpu_rating)
            self.remaining_cpu = max(0, self.remaining_cpu - progress)

            # Update completion percentage
            self.completion_percentage = min(100, ((self.total_cpu_required - self.remaining_cpu) /
                                                self.total_cpu_required * 100))

            if self.remaining_cpu <= 0:
                self.status = 'COMPLETED'
                self.completion_time = current_time
                self.actual_exec_time = self.completion_time - self.start_time
                status['completed'] = True

            status['processed'] = True
            status['progress'] = self.completion_percentage

        return status
    def enqueue_task(self, task: Task):
        """Add task to queue and update resource utilization"""
        can_process, failure_reason = self.can_process_task(task)

        if can_process:
            task.status = 'READY'
            self.task_queue.append(task)
            logger.info(f"Task {task.id} queued on {self.type}. Queue length: {len(self.task_queue)}")
        else:
            task.status = 'FAILED'
            task.failure_reason = failure_reason
            self.failed_tasks.append(task)
            self.failed_tasks_count += 1
            logger.warning(f"Task {task.id} ({task.type}) failed on {self.type}: {failure_reason}")
    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in queue with fixed progress tracking
        """
        status = {
            'completed_tasks': len(self.completed_tasks),
            'current_task': None,
            'queue_length': len(self.task_queue),
            'resource_utilization': self.calculate_resource_utilization()
        }

        if self.current_task:
            # Update task progress
            progress = self.current_task.update_progress(self, current_time)

            # Update status dictionary
            status['current_task'] = {
                'id': self.current_task.id,
                'type': self.current_task.type,
                'phase': self.current_task.status,
                'progress': progress['progress'],
                'input_size': self.current_task.input_size,
                'output_size': self.current_task.output_size
            }

            # Handle task completion
            if self.current_task.status == 'COMPLETED':
                self.completed_tasks.append(self.current_task)
                logger.info(f"Task {self.current_task.id} completed on {self.type}")
                self.current_task = None

        # Start new task if available
        if not self.current_task and self.task_queue:
            self.current_task = self.task_queue.pop(0)
            # Make sure tasks are in CREATED state before processing
            logger.info(f"Starting Task {self.current_task.id} (status: {self.current_task.status}) - forcing to CREATED")
            self.current_task.status = 'CREATED'

            self.current_task.exec_start_time = current_time
            self.current_task._time_initialized = False  # Reset this flag to trigger initialization

        return status
class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        #self.arrival_rate = 1.0  # Default task arrival rate
        self.console = Console()
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'failed_tasks': 0,
            'globally_failed_tasks': [],  # Store globally unassigned failed tasks
            'makespan': 0,
            'throughput': 0
        }
    def calculate_turnaround_time(self, task, simulation_start_time):
        """
        Calculate turnaround time relative to simulation start time
        """
        if (task.arrival_time is not None and
            task.completion_time is not None):
            # Adjust times relative to simulation start
            arrival_relative = task.arrival_time - simulation_start_time
            completion_relative = task.completion_time - simulation_start_time

            turnaround_time = max(completion_relative - arrival_relative, 0)
            return turnaround_time
        return 0.0

    def calculate_waiting_time(self, task):
        """
        Calculate waiting time
        """
        if (task.turnaround_time is not None and
            task.actual_exec_time is not None):
            waiting_time = max(task.turnaround_time - task.actual_exec_time, 0)
            return waiting_time
        return 0.0

    def calculate_timing_metrics(self, completed_tasks: List[Task], simulation_start_time: float) -> Dict:
        """
        Calculate timing metrics with guaranteed dictionary return
        """
        # Initialize metrics with default values
        metrics = {
            'average_turnaround_time': 0.0,
            'average_waiting_time': 0.0,
            'average_execution_time': 0.0,
            'total_tasks_processed': 0,
            'total_turnaround_time': 0.0,
            'total_waiting_time': 0.0,
            'total_execution_time': 0.0
        }

        if not completed_tasks:
            logger.warning("No completed tasks to process")
            return metrics

        total_turnaround = 0.0
        total_waiting = 0.0
        total_execution = 0.0
        valid_tasks = 0

        logger.info(f"\nProcessing timing metrics for {len(completed_tasks)} completed tasks")

        for task in completed_tasks:
            if task.status == 'COMPLETED':
                # Log timing values for debugging
                logger.debug(f"""
                Task {task.id} timing values:
                - Arrival: {task.arrival_time}
                - Start: {task.start_time}
                - Exec Start: {task.exec_start_time}
                - Completion: {task.completion_time}
                """)

                # Ensure timing values are valid
                if all(x is not None for x in [task.arrival_time, task.start_time,
                                            task.completion_time, task.exec_start_time]):
                    # Recalculate metrics to ensure consistency
                    task.turnaround_time = max(0, task.completion_time - task.arrival_time)
                    task.actual_exec_time = max(0, task.completion_time - task.exec_start_time)
                    task.waiting_time = max(0, task.turnaround_time - task.actual_exec_time)

                    if task.turnaround_time > 0 and task.actual_exec_time > 0:
                        total_turnaround += task.turnaround_time
                        total_waiting += task.waiting_time
                        total_execution += task.actual_exec_time
                        valid_tasks += 1
                        logger.info(f"Task {task.id} metrics valid - turnaround: {task.turnaround_time:.2f}s")
                    else:
                        logger.warning(f"Task {task.id} has zero or negative timing values")
                else:
                    logger.warning(f"Task {task.id} has missing timing values")

        # Update metrics if we have valid tasks
        if valid_tasks > 0:
            metrics.update({
                'average_turnaround_time': total_turnaround / valid_tasks,
                'average_waiting_time': total_waiting / valid_tasks,
                'average_execution_time': total_execution / valid_tasks,
                'total_tasks_processed': valid_tasks,
                'total_turnaround_time': total_turnaround,
                'total_waiting_time': total_waiting,
                'total_execution_time': total_execution
            })

        # Log final summary
        logger.info(f"""
        Timing Metrics Summary:
        - Total Tasks: {len(completed_tasks)}
        - Valid Tasks: {valid_tasks}
        - Average Turnaround: {metrics['average_turnaround_time']:.2f}s
        - Average Waiting: {metrics['average_waiting_time']:.2f}s
        - Average Execution: {metrics['average_execution_time']:.2f}s
        """)

        return metrics
    def calculate_datacenter_utilization(self, start_time: float, end_time: float) -> Dict:
        """
        Calculate utilization for different datacenter types following the paper's integral approach
        """
        # Separate resources by datacenter type
        smartphone_resources = [r for r in self.resources if r.type.startswith('Smartphone_')]
        raspberry_pi_resources = [r for r in self.resources if r.type.startswith('Raspberry_')]
        cloud_resources = [r for r in self.resources if r.type.startswith('Cloud_')]

        def calculate_datacenter_ru(resources: List[Resource]) -> Dict:
            """
            Calculate resource utilization for a specific datacenter type
            """
            # Collect individual resource utilizations
            resource_utilizations = [r.calculate_resource_utilization() for r in resources]

            # Calculate datacenter-wide metrics
            return {
                'total_resources': len(resources),
                'avg_cpu_utilization': np.mean([ru['cpu_utilization'] for ru in resource_utilizations]),
                'avg_memory_utilization': np.mean([ru['memory_utilization'] for ru in resource_utilizations]),
                'avg_bandwidth_utilization': np.mean([ru['bandwidth_utilization'] for ru in resource_utilizations]),
                'overall_utilization': np.mean([ru['overall_utilization'] for ru in resource_utilizations]),
                'active_tasks': sum(ru['active_tasks'] for ru in resource_utilizations),
                'raw_metrics': resource_utilizations
            }

        # Calculate utilization for each datacenter type
        datacenter_utilization = {
            'smartphone_edge': calculate_datacenter_ru(smartphone_resources),
            'raspberry_pi_edge': calculate_datacenter_ru(raspberry_pi_resources),
            'cloud': calculate_datacenter_ru(cloud_resources)
        }

        # Calculate overall datacenter utilization
        datacenter_utilization['overall'] = {
            'total_resources': len(self.resources),
            'total_simulation_time': end_time - start_time,
            'avg_cpu_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_cpu_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_cpu_utilization'],
                datacenter_utilization['cloud']['avg_cpu_utilization']
            ]),
            'avg_memory_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_memory_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_memory_utilization'],
                datacenter_utilization['cloud']['avg_memory_utilization']
            ]),
            'avg_bandwidth_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_bandwidth_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_bandwidth_utilization'],
                datacenter_utilization['cloud']['avg_bandwidth_utilization']
            ]),
            'total_active_tasks': sum(
                datacenter_utilization[dc_type]['active_tasks']
                for dc_type in ['smartphone_edge', 'raspberry_pi_edge', 'cloud']
            )
        }

        return datacenter_utilization

    def generate_tasks(self, total_tasks: int) -> Tuple[List[Task], List[float], List[float]]:
            """
            Generate tasks with Poisson-distributed arrival times and return raw inter-arrival times.

            Args:
                total_tasks (int): The total number of tasks to generate.

            Returns:
                Tuple[List[Task], List[float], List[float]]: A tuple containing:
                    - List of generated Task objects
                    - List of cumulative arrival times
                    - List of raw inter-arrival times
            """


            simulation_duration = 100
            lambda_rate = total_tasks / simulation_duration
            inter_arrival_times = []
            cumulative_times = [0]  # Start with 0 as the first arrival time

            while len(inter_arrival_times) < total_tasks - 1:  # We need one less inter-arrival time than total tasks
                interval = np.random.exponential(1.0 / lambda_rate)
                inter_arrival_times.append(interval)
                cumulative_times.append(cumulative_times[-1] + interval)
            # Use a base time to ensure consistent and positive arrival times
            # In the generate_tasks method
            base_time = datetime.now().timestamp()

            tasks = []
            for i in range(total_tasks):
                task = Task(
                    task_id=i + 1,
                    task_type="placeholder",
                    input_size=0.0,    # Changed from data_size to input_size
                    output_size=0.0,   # Added output_size
                    cpu_required=0.0
                )
                task.arrival_time = base_time + cumulative_times[i]
                tasks.append(task)

            # Write inter-arrival times to a CSV file
            with open('inter_arrival_times.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Inter-arrival Time'])
                for time in inter_arrival_times:
                    writer.writerow([time])

            return tasks, cumulative_times, inter_arrival_times

    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with comprehensive task information.
        """
        table = Table(show_header=False, show_lines=True)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.total_cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.total_bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['resource_utilization']['cpu_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_cpu_usage']:.2f}/{resource.total_cpu_rating} MI/s)"
        )
        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['resource_utilization']['memory_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_memory_usage']:.2f}/{resource.total_memory} GB)"
        )
        table.add_row(
            f"[yellow]Bandwidth Usage:[/yellow] {status['resource_utilization']['bandwidth_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_bandwidth_usage']:.2f}/{resource.total_bandwidth} MB/s)"
        )

        # Task Queue Information
        table.add_row("\n[bold]Task Queue[/bold]")
        table.add_row(f"[yellow]Queued Tasks:[/yellow] {status['queue_length']}")

        # Current Tasks
        table.add_row("\n[bold]Current Tasks[/bold]")
        if status['current_task']:
            current_task = status['current_task']
            table.add_row(
                f"[blue]Task {current_task['id']} ({current_task['type']}):[/blue] "
                f"Phase: {current_task['phase']} Progress: {current_task['progress']}"
            )
        else:
            table.add_row("[dim]No tasks currently processing[/dim]")

        # Failed Tasks
        table.add_row("\n[bold]Failed Tasks[/bold]")
        if resource.failed_tasks:
            for task in resource.failed_tasks:
                table.add_row(
                    f"[red]Task {task.id} ({task.type}):[/red] "
                    f"Reason: {task.failure_reason}"
                )
        else:
            table.add_row("[dim]No failed tasks[/dim]")

        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )
    def verify_and_fix_task_distribution(self, total_tasks: int) -> List[Task]:
        """
        Emergency fix to directly ensure tasks are properly queued to resources
        regardless of distribution algorithm issues.
        """
        logger.info("🚨 EMERGENCY FIX: Verifying task distribution...")

        # Check current queue status
        total_queued = sum(len(resource.task_queue) for resource in self.resources)
        logger.info(f"Current queued tasks: {total_queued} of {total_tasks}")

        # If we have tasks queued, no action needed
        if total_queued >= total_tasks:
            logger.info("✅ Task distribution verified: All tasks are properly queued")

            # Collect all tasks to return
            all_tasks = []
            for resource in self.resources:
                all_tasks.extend(resource.task_queue)
                all_tasks.extend(resource.failed_tasks)

            return all_tasks

        # If we have zero or insufficient tasks queued, perform emergency distribution
        logger.warning(f"⚠️ Task distribution problem detected: Only {total_queued} of {total_tasks} tasks queued")

        # Clear all existing resource queues
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Generate fresh tasks
        logger.info("Generating emergency replacement tasks...")
        tasks, _, _ = self.generate_tasks(total_tasks)

        # Define task types with fixed requirements
        task_types = [
            # Read Tasks
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Distribute tasks using simple round-robin
        distributed_tasks = []
        resource_index = 0

        for i, task_record in enumerate(tasks):
            # Select task type based on index
            task_type = task_types[i % len(task_types)]

            # Create task with proper properties
            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time
            task.status = 'CREATED'  # Ensure correct initial status

            # Assign to resource using round-robin
            resource = self.resources[resource_index]
            resource.task_queue.append(task)

            # Log the assignment
            logger.info(f"Emergency assigned Task {task.id} ({task.type}) to {resource.type}")

            # Advance resource index for round-robin
            resource_index = (resource_index + 1) % len(self.resources)

            # Add to distributed tasks list
            distributed_tasks.append(task)

        # Verify fix worked
        total_queued = sum(len(resource.task_queue) for resource in self.resources)
        logger.info(f"Emergency fix completed: {total_queued} of {total_tasks} tasks now queued")

        # Double check each resource has tasks
        for resource in self.resources:
            logger.info(f"Resource {resource.type} now has {len(resource.task_queue)} tasks queued")

        return distributed_tasks

    def calculate_load_balancing_metrics(self) -> Dict:
        """
        Calculate load balancing metrics based on resource utilization formula:
        1. U(Ri) = L(Ri) / Makespan - Utilization of each resource
        2. Uav = (Σ U(Ri)) / n - Average utilization across all resources

        Returns:
            Dict: Load balancing metrics
        """
        # Start debug logging
        logger.info("===== LOAD BALANCING METRICS CALCULATION STARTED =====")

        # Get makespan from metrics (correct order - define before using)
        # Calculate makespan dynamically
        makespan = max(
            max(
                (task.completion_time or 0)
                for resource in self.resources
                for task in resource.completed_tasks
            ),
            0.001  # Ensure a minimum makespan to avoid division by zero
        )

        # Debug output
        logger.info(f"DEBUG - Makespan value: {makespan}")
        logger.info(f"DEBUG - self.metrics contents: {self.metrics}")

        # Check for valid makespan
        if makespan <= 0:
            logger.warning("Makespan is zero or negative, returning empty metrics")
            return {
                'resource_utilizations': {},
                'average_utilization': 0,
                'load_balance_score': 0,
                'utilization_std_dev': 0,
                'resource_type_metrics': {}
            }

        # Calculate load on each resource
        # L(Ri) = total processing time of all tasks assigned to Ri
        resource_loads = {}
        resource_utilizations = {}

        # Group resources by type
        smartphone_resources = []
        raspberry_pi_resources = []
        cloud_resources = []

        # Check and fix task timing issues
        task_timing_issues = 0
        fixed_task_count = 0

        # Process each resource
        logger.info("Processing individual resource metrics:")

        for resource in self.resources:
            # Group resource by type
            if resource.type.startswith('Smartphone_'):
                smartphone_resources.append(resource)
            elif resource.type.startswith('Raspberry_'):
                raspberry_pi_resources.append(resource)
            elif resource.type.startswith('Cloud_'):
                cloud_resources.append(resource)

            # Fix any task timing issues
            for task in resource.completed_tasks:
                if not hasattr(task, 'actual_exec_time') or task.actual_exec_time is None:
                    task_timing_issues += 1
                    # Try to calculate from raw timestamps
                    if hasattr(task, 'completion_time') and hasattr(task, 'exec_start_time'):
                        if task.completion_time and task.exec_start_time:
                            task.actual_exec_time = max(0.001, task.completion_time - task.exec_start_time)
                            fixed_task_count += 1
                            logger.info(f"Fixed task {task.id} exec_time to {task.actual_exec_time:.4f}s")
                elif task.actual_exec_time <= 0:
                    task_timing_issues += 1
                    # Force a minimum value
                    task.actual_exec_time = 0.001  # 1 millisecond minimum
                    fixed_task_count += 1
                    logger.info(f"Corrected zero/negative exec_time for task {task.id}")

            # Debug task information
            completed_task_count = len(resource.completed_tasks)
            valid_task_count = sum(
                1 for task in resource.completed_tasks
                if hasattr(task, 'actual_exec_time') and task.actual_exec_time is not None and task.actual_exec_time > 0
            )

            logger.info(f"Resource {resource.id} ({resource.type}):")
            logger.info(f"  - Completed tasks: {completed_task_count}")
            logger.info(f"  - Tasks with valid execution time: {valid_task_count}")

            # Log individual task execution times
            task_times_summary = []
            for i, task in enumerate(resource.completed_tasks[:5]):  # Log first 5 tasks for brevity
                task_exec_time = getattr(task, 'actual_exec_time', None)
                task_times_summary.append(f"Task {task.id}: {task_exec_time:.4f}s" if task_exec_time else f"Task {task.id}: None")

            logger.info(f"  - Task times: {', '.join(task_times_summary)}")

            if completed_task_count > 5:
                logger.info(f"  - ... and {completed_task_count - 5} more tasks")

            # Calculate total processing time for all completed tasks
            total_processing_time = sum(
                task.actual_exec_time
                for task in resource.completed_tasks
                if hasattr(task, 'actual_exec_time') and task.actual_exec_time is not None and task.actual_exec_time > 0
            )

            logger.info(f"  - Total processing time: {total_processing_time:.4f} seconds")

            # Calculate utilization: U(Ri) = L(Ri) / Makespan
            utilization = total_processing_time / makespan if makespan > 0 else 0
            logger.info(f"  - Utilization (U(Ri) = L(Ri) / Makespan): {utilization:.4f}")

            resource_loads[resource.id] = total_processing_time
            resource_utilizations[resource.id] = utilization

        # Log task timing fixes
        logger.info(f"Found {task_timing_issues} tasks with timing issues, fixed {fixed_task_count}")

        # Check if we have any valid utilization data
        if not resource_utilizations or all(util == 0 for util in resource_utilizations.values()):
            logger.warning("All resources have zero utilization, metrics will be zero")

        # Calculate average utilization overall: Uav = (Σ U(Ri)) / n
        average_utilization = sum(resource_utilizations.values()) / len(self.resources) if self.resources else 0
        logger.info(f"Average utilization across all resources (Uav): {average_utilization:.4f}")

        # Calculate standard deviation of utilizations (measure of balance)
        utilization_values = list(resource_utilizations.values())
        logger.info(f"All utilization values: {utilization_values}")

        utilization_std_dev = np.std(utilization_values) if utilization_values else 0
        logger.info(f"Standard deviation of utilizations: {utilization_std_dev:.4f}")

        # Load balance score (lower is better - 0 is perfect balance)
        # Using coefficient of variation to normalize
        if average_utilization <= 0.0001:  # Very small threshold
            logger.warning("Average utilization near zero, setting load balance score to 0")
            load_balance_score = 0
        else:
            load_balance_score = utilization_std_dev / average_utilization

        logger.info(f"Load balance score (CV = std_dev / avg): {load_balance_score:.4f}")

        # Calculate metrics per resource type
        def calculate_resource_type_metrics(resources, type_name):
            logger.info(f"\nCalculating metrics for {type_name} resources:")

            if not resources:
                logger.info(f"  No {type_name} resources found")
                return {
                    'average_utilization': 0,
                    'utilization_std_dev': 0,
                    'load_balance_score': 0,
                    'resource_count': 0
                }

            logger.info(f"  Found {len(resources)} {type_name} resources")
            type_utilizations = [resource_utilizations[r.id] for r in resources]
            logger.info(f"  Utilization values: {type_utilizations}")

            type_avg_util = sum(type_utilizations) / len(resources)
            logger.info(f"  Average utilization: {type_avg_util:.4f}")

            type_std_dev = np.std(type_utilizations) if len(type_utilizations) > 1 else 0
            logger.info(f"  Standard deviation: {type_std_dev:.4f}")

            type_balance_score = type_std_dev / type_avg_util if type_avg_util > 0 else 0
            logger.info(f"  Balance score: {type_balance_score:.4f}")

            return {
                'average_utilization': type_avg_util,
                'utilization_std_dev': type_std_dev,
                'load_balance_score': type_balance_score,
                'resource_count': len(resources)
            }

        # Get metrics for each resource type
        logger.info("\nCalculating metrics by resource type:")
        resource_type_metrics = {
            'smartphone': calculate_resource_type_metrics(smartphone_resources, "smartphone"),
            'raspberry_pi': calculate_resource_type_metrics(raspberry_pi_resources, "raspberry_pi"),
            'cloud': calculate_resource_type_metrics(cloud_resources, "cloud")
        }

        # Calculate inter-type balance
        logger.info("\nCalculating inter-type balance metrics:")
        type_avg_utils = [
            resource_type_metrics['smartphone']['average_utilization'],
            resource_type_metrics['raspberry_pi']['average_utilization'],
            resource_type_metrics['cloud']['average_utilization']
        ]
        logger.info(f"  Resource type average utilizations: {type_avg_utils}")

        inter_type_std_dev = np.std(type_avg_utils)
        logger.info(f"  Inter-type standard deviation: {inter_type_std_dev:.4f}")

        inter_type_avg = sum(type_avg_utils) / 3 if sum(type_avg_utils) > 0 else 0
        logger.info(f"  Inter-type average: {inter_type_avg:.4f}")

        inter_type_balance = inter_type_std_dev / inter_type_avg if inter_type_avg > 0 else 0
        logger.info(f"  Inter-type balance score: {inter_type_balance:.4f}")

        resource_type_metrics['inter_type_balance'] = {
            'standard_deviation': inter_type_std_dev,
            'balance_score': inter_type_balance
        }

        logger.info("===== LOAD BALANCING METRICS CALCULATION COMPLETED =====")

        # Force log flush
        for handler in logger.handlers:
            if isinstance(handler, logging.FileHandler):
                handler.flush()

        return {
            'resource_utilizations': resource_utilizations,
            'average_utilization': average_utilization,
            'load_balance_score': load_balance_score,
            'utilization_std_dev': utilization_std_dev,
            'resource_type_metrics': resource_type_metrics
        }

    def _optimized_hybrid_algorithm(self, total_tasks: int) -> List[Task]:
        """
        Optimized hybrid Tabu-Genetic algorithm for smaller task sets (2000-5000 tasks).
        This version is significantly faster while maintaining solution quality.

        Key optimizations:
        1. Faster initialization with smaller population
        2. Reduced generations with early convergence detection
        3. Streamlined Tabu Search focused on high-impact tasks
        4. Simplified fitness function with multi-objective cost calculation
        5. Better progress tracking throughout execution

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        # Log start time and show progress indicator
        start_time = time.time()
        logger.info(f"\n⏳ Starting Optimized Hybrid Algorithm for {total_tasks} tasks...")
        logger.info("Phase 1/5: Task generation and initialization...")

        # Initialize CSV tracking for algorithm progress
        csv_folder = "/content/drive/My Drive/EdgeSimPy/algorithm_tracking"
        os.makedirs(csv_folder, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        ga_csv_path = os.path.join(csv_folder, f'ga_evolution_{timestamp}.csv')
        ts_csv_path = os.path.join(csv_folder, f'tabu_search_{timestamp}.csv')
        solution_csv_path = os.path.join(csv_folder, f'solution_comparison_{timestamp}.csv')

        # Initialize CSV files with headers
        with open(ga_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Generation', 'Best_Fitness', 'Average_Fitness', 'Population_Diversity',
                            'Mutation_Rate', 'Crossover_Rate', 'Tasks_Per_Resource',
                            'Load_Balance_Score', 'Solution_Hash'])

        with open(ts_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Iteration', 'Current_Fitness', 'Best_Fitness', 'Move_Type',
                            'Task_ID', 'Old_Resource', 'New_Resource', 'Tabu_List_Size',
                            'Improvement_Percentage', 'Solution_Hash'])

        with open(solution_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Algorithm_Phase', 'Timestamp', 'Solution_Cost', 'Load_Balance_Score',
                            'Resource_Distribution', 'Task_Type_Distribution', 'Average_Execution_Time',
                            'Worst_Task_Time', 'Best_Task_Time'])

        try:
            # === STEP 1: Generate tasks with batch processing for efficiency ===
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with cached requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                    'reliability': resource.reliability
                } for resource in self.resources
            }

            # Helper methods for tracking metrics
            def _calculate_population_diversity(population):
                """Calculate population diversity as percentage of different assignments"""
                if len(population) <= 1:
                    return 0

                # Take the first solution as reference
                reference = population[0]
                total_assignments = len(reference) * (len(population) - 1)
                different_assignments = 0

                # Compare each solution with the reference
                for solution in population[1:]:
                    for task in reference:
                        if task in solution and reference[task] != solution[task]:
                            different_assignments += 1

                return (different_assignments / total_assignments) * 100 if total_assignments > 0 else 0

            def _get_resource_distribution(solution):
                """Get distribution of tasks per resource"""
                distribution = {resource.type: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if resource:
                        distribution[resource.type] += 1

                return distribution

            def _get_task_type_distribution(solution):
                """Get distribution of task types per resource type"""
                distribution = {}

                for task, resource in solution.items():
                    if resource:
                        resource_type = resource.type.split('_')[0]  # Cloud, Smartphone, or Raspberry
                        task_type = task.type

                        if resource_type not in distribution:
                            distribution[resource_type] = {}

                        if task_type not in distribution[resource_type]:
                            distribution[resource_type][task_type] = 0

                        distribution[resource_type][task_type] += 1

                return distribution
           # === NEW MAKESPAN WEIGHTED SUM FUNCTION ===
            def makespan_weighted_sum_fitness(solution, weights=None):
                """
                Multi-objective weighted sum fitness function focusing on:
                1. Makespan (total completion time)
                2. Average resource utilization

                Args:
                    solution: Task-to-resource mapping
                    weights: Optional weights for different objectives

                Returns:
                    float: Weighted sum fitness value (lower is better)
                """
                if weights is None:
                    # Default weights - fixed for consistent optimization
                    weights = {
                        'makespan': 0.65,
                        'resource_utilization': 0.35
                    }

                # 1. Makespan calculation
                # Track earliest completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task arrival order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        continue

                    # Calculate task execution time
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Task starts at either its arrival time or when the resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + total_task_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                # 2. Average resource utilization calculation
                # Track total processing time for each resource
                resource_total_processing = {resource.id: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if not resource:
                        continue

                    # Calculate processing time for this task
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Add to resource's total processing time
                    resource_total_processing[resource.id] += total_task_time

                # Calculate average utilization across all resources
                if makespan > 0:
                    resource_utilizations = [processing_time / makespan for processing_time in resource_total_processing.values()]
                    avg_resource_utilization = sum(resource_utilizations) / len(self.resources)

                    # Convert to cost: higher utilization should have LOWER cost
                    utilization_cost = 1.0 / (avg_resource_utilization + 0.001)  # Add small constant to avoid division by zero
                else:
                    utilization_cost = 100000  # High penalty for zero makespan (shouldn't happen)

                # Calculate weighted sum of all objectives
                total_cost = (
                    weights['makespan'] * makespan +
                    weights['resource_utilization'] * utilization_cost
                )

                return total_cost

            # Faster task creation with larger batch size
            batch_size = 500
            initial_tasks = []
            resource_loads = {r.id: 0 for r in self.resources}

            # Show progress for task creation
            logger.info(f"Creating {total_tasks} tasks in batches of {batch_size}...")

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Instead of cycling through task types:
                    # task_type = list(TASK_TYPES.keys())[j % len(TASK_TYPES)]

                    # Use random selection:
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # Special handling for very small task counts (like 5)
            if total_tasks <= 10:
                logger.info(f"Small task count ({total_tasks}) detected - using specialized optimization parameters")
                # Smaller population, fewer generations for tiny problems
                population_size = max(5, min(10, total_tasks))
                generations = 3
                tabu_iterations = max(5, min(10, total_tasks))
                tabu_tenure = max(3, min(5, total_tasks))
                max_stagnation = 2
            else:
            # === STEP 2: Adjusted optimization parameters for larger task counts ===
                population_size = min(50, int(total_tasks * 0.1))  # Proportional to total tasks
                generations = min(20, int(total_tasks * 0.05))     # More generations for larger problems
                tabu_iterations = min(100, int(total_tasks * 0.05))  # Increased iterations
                tabu_tenure = min(50, int(total_tasks * 0.02))     # Slightly larger tabu tenure
                max_stagnation = 10    # Increased early stopping patience

            # === Initialize epsilon decay parameters ===
                epsilon_start = 1.0  # Start with full exploration
                epsilon_end = 0.1    # End with minimal exploration
                ga_epsilon_decay_rate = (epsilon_start - epsilon_end) / generations
                ts_epsilon_decay_rate = (epsilon_start - epsilon_end) / tabu_iterations
                current_ga_epsilon = epsilon_start
                current_ts_epsilon = epsilon_start

            # With these adaptive mutation parameters:
            base_mutation_rate = 0.15
            min_mutation_rate = 0.1
            max_mutation_rate = 0.4
            current_mutation_rate = base_mutation_rate
            diversity_threshold_low = 15  # Percentage below which to increase mutation
            diversity_threshold_high = 40  # Percentage above which to decrease mutation
            crossover_rate = 0.8
            elite_count = 2

            tabu_list = collections.deque(maxlen=tabu_tenure)

            # === STEP 3: Define optimized utility functions ===
            def fast_exec_time_estimate(task, resource_id):
                """Simplified execution time estimation for speed"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Basic estimate combining transfer and processing time
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']
                process_time = task.total_cpu_required / resource['cpu_rating']

                # Simple reliability factor
                reliability_factor = 1.0 + (1.0 - resource['reliability']) * 0.5

                # Simplified total time calculation
                return (transfer_time + process_time) * reliability_factor

            def simplified_fitness(solution):
                """
                Multi-objective cost function focusing on:
                1. Task distribution efficiency
                2. Load balancing
                3. Resource type compatibility
                4. Task execution characteristics
                """
                # Calculate total task execution times with penalties
                total_execution_cost = 0
                task_counts = {}
                resource_loads = {}

                # Specific task type and resource compatibility coefficients
                type_resource_penalties = {
                    'WT3': {
                        'Raspberry_': 3.0,  # Heavy penalty for WT3 on Raspberry Pi
                        'Cloud_': 0.5,      # Slight bonus for WT3 on cloud
                        'Smartphone_': 1.5  # Moderate penalty on smartphones
                    },
                    'RT1': {
                        'Cloud_': 0.1,      # Preferred placement
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    },
                    'RT3': {
                        'Cloud_': 0.1,      # Preferred placement
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    }
                }

                # Process each task in the solution
                for task, resource in solution.items():
                    if not resource:
                        # Heavy penalty for unassigned tasks
                        total_execution_cost += 100000
                        continue

                    # Basic execution time estimation
                    exec_time = (task.input_size + task.output_size + task.total_cpu_required / resource.total_cpu_rating)

                    # Adjust cost based on task-resource compatibility
                    penalty_multiplier = type_resource_penalties.get(task.type, {}).get(resource.type.split('_')[0] + '_', 1.0)
                    exec_time *= penalty_multiplier

                    # Track task counts per resource
                    task_counts[resource.id] = task_counts.get(resource.id, 0) + 1
                    resource_loads[resource.id] = resource_loads.get(resource.id, 0) + exec_time

                    total_execution_cost += exec_time

                # Load balancing penalty
                if task_counts:
                    # Standard deviation of task counts
                    count_std = np.std(list(task_counts.values())) * 500
                    # Standard deviation of resource loads
                    load_std = np.std(list(resource_loads.values())) * 1000

                    total_execution_cost += count_std + load_std
                # Add the weighted sum component for makespan optimization (10% of total fitness)
                #weighted_sum_component = makespan_weighted_sum_fitness(solution)
                total_execution_cost = 1.0 * total_execution_cost # + 0.1 * weighted_sum_component
                # Scale down the total cost for manageable values
                total_execution_cost *= 0.5

                return total_execution_cost

            # === STEP 4: Create smart initial solution ===
            logger.info("Phase 2/5: Creating initial population...")

            def create_bandwidth_aware_solution():
                """Generate initial solution with data size awareness"""
                solution = {}
                high_bw_resources = [r for r in self.resources if r.total_bandwidth >= 800]
                med_bw_resources = [r for r in self.resources if 400 <= r.total_bandwidth < 800]
                low_bw_resources = [r for r in self.resources if r.total_bandwidth < 400]

                # Track loads for balancing
                local_loads = {r.id: 0 for r in self.resources}

                # Sort tasks by arrival time
                for task in sorted(initial_tasks, key=lambda t: t.arrival_time):
                    # Assign based on data size needs
                    if task.input_size > 2.0 or task.output_size > 2.0:
                        candidates = high_bw_resources if high_bw_resources else self.resources
                    elif task.input_size > 0.5 or task.output_size > 0.5:
                        candidates = med_bw_resources + high_bw_resources if med_bw_resources or high_bw_resources else self.resources
                    else:
                        candidates = self.resources

                    # Select least loaded resource
                    resource = min(candidates, key=lambda r: local_loads[r.id])
                    solution[task] = resource
                    local_loads[resource.id] += 1

                return solution

            # Create initial population
            population = []
            smart_solution = create_bandwidth_aware_solution()
            # Log the smart solution to verify it contains valid assignments
            logger.info(f"Created smart solution with {len(smart_solution)} assignments")
            population.append(smart_solution)

            # Generate diverse initial population
            for i in range(1, population_size):
                # Create variation of smart solution
                solution = copy.deepcopy(smart_solution)

                # Mutate a percentage of assignments
                mutation_pct = 0.1 + (0.2 * i / population_size)  # More mutation for later solutions
                tasks_to_mutate = random.sample(
                    list(solution.keys()),
                    k=max(1, int(len(solution) * mutation_pct))
                )

                for task in tasks_to_mutate:
                    solution[task] = random.choice(self.resources)

                population.append(solution)

            # === STEP 5: Run Optimized Genetic Algorithm ===
            logger.info("Phase 3/5: Running genetic algorithm optimization...")

            # Evaluate initial population
            fitnesses = [simplified_fitness(sol) for sol in population]

            # Track best solution
            best_idx = fitnesses.index(min(fitnesses))
            best_solution = copy.deepcopy(population[best_idx])
            best_fitness = fitnesses[best_idx]
            best_cost = simplified_fitness(best_solution)

            # Record initial state for the solution comparison CSV
            initial_resource_distribution = json.dumps(_get_resource_distribution(best_solution))
            initial_task_type_distribution = json.dumps(_get_task_type_distribution(best_solution))
            initial_task_times = [fast_exec_time_estimate(task, resource.id) for task, resource in best_solution.items()]
            avg_time = sum(initial_task_times) / len(initial_task_times) if initial_task_times else 0
            worst_time = max(initial_task_times) if initial_task_times else 0
            best_time = min(initial_task_times) if initial_task_times else 0

            # Calculate initial load balance
            load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id]) for res in self.resources])

            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Initial_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    initial_resource_distribution,  # Resource distribution
                    initial_task_type_distribution,  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Run genetic algorithm with progress updates
            stagnation_counter = 0

            for gen in range(generations):
                # Progress indicator
                current_cost = simplified_fitness(best_solution)
                logger.info(f"Generation {gen+1}/{generations} - Current best cost: {current_cost}")

                # Update epsilon for this generation
                current_ga_epsilon = max(epsilon_end, epsilon_start - gen * ga_epsilon_decay_rate)
                logger.info(f"Current GA epsilon: {current_ga_epsilon:.3f}")

                # Sort by fitness and retain elites
                sorted_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i])
                new_population = [copy.deepcopy(population[i]) for i in sorted_indices[:elite_count]]

                # Fill remaining population with crossover and mutation
                while len(new_population) < population_size:

                    # Use epsilon-greedy selection for parent selection
                    if random.random() < current_ga_epsilon:
                        # Exploration: Random selection
                        parent1_idx = random.randrange(len(population))
                        parent2_idx = random.randrange(len(population))
                    else:
                        # Exploitation: Tournament selection
                        tournament_size = min(3, len(population))
                        parent1_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])
                        parent2_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])

                    parent1 = population[parent1_idx]
                    parent2 = population[parent2_idx]
                    # Adaptive crossover rate based on epsilon
                    effective_crossover_rate = crossover_rate * (1 - current_ga_epsilon * 0.3)

                    # Simplified crossover
                    if random.random() < effective_crossover_rate:
                        child = {}

                        # Create a random crossover point
                        tasks_ordered = sorted(parent1.keys(), key=lambda t: t.arrival_time)
                        crossover_point = random.randint(1, len(tasks_ordered) - 1)

                        # Take first part from parent1, second from parent2
                        for i, task in enumerate(tasks_ordered):
                            if i < crossover_point:
                                child[task] = parent1[task]
                            else:
                                # Make sure task exists in parent2
                                if task in parent2:
                                    child[task] = parent2[task]
                                else:
                                    child[task] = parent1[task]
                    else:
                        # No crossover, just clone parent1
                        child = copy.deepcopy(parent1)

                    # Epsilon-influenced mutation
                    # Higher mutation when epsilon is high (exploration), lower when epsilon is low (exploitation)
                    epsilon_adjusted_mutation_rate = current_mutation_rate * current_ga_epsilon + min_mutation_rate
                    # Simple mutation
                    if random.random() < epsilon_adjusted_mutation_rate:
                        # Adaptive mutation count based on epsilon
                        # More aggressive mutation when epsilon is high
                        mutation_ratio = 0.05 + current_ga_epsilon * 0.15  # 5-20% mutation
                        mutation_count = max(1, int(len(child) * mutation_ratio))
                        tasks_to_mutate = random.sample(list(child.keys()), k=mutation_count)

                        for task in tasks_to_mutate:
                            child[task] = random.choice(self.resources)

                    new_population.append(child)

                # Update population
                population = new_population

                # Calculate fitness - evaluate each solution
                fitnesses = [simplified_fitness(sol) for sol in population]

                # Check for improvement
                current_best_idx = fitnesses.index(min(fitnesses))
                current_best_fitness = fitnesses[current_best_idx]

                if current_best_fitness < best_fitness:
                    best_solution = copy.deepcopy(population[current_best_idx])
                    best_fitness = current_best_fitness
                    best_cost = simplified_fitness(best_solution)
                    logger.info(f"✓ New best solution found! Fitness: {best_fitness}, Cost: {best_cost}")
                    stagnation_counter = 0
                else:
                    stagnation_counter += 1

                # Calculate diversity (percentage of different assignments between solutions)
                diversity = _calculate_population_diversity(population)

                if diversity < diversity_threshold_low:
                    # Low diversity - increase mutation rate to encourage exploration
                    current_mutation_rate = min(max_mutation_rate, current_mutation_rate * 1.5)
                    logger.info(f"Low diversity ({diversity:.1f}%) - increasing mutation rate to {current_mutation_rate:.3f}")
                elif diversity > diversity_threshold_high:
                    # High diversity - decrease mutation rate to encourage exploitation
                    current_mutation_rate = max(min_mutation_rate, current_mutation_rate * 0.8)
                    logger.info(f"High diversity ({diversity:.1f}%) - decreasing mutation rate to {current_mutation_rate:.3f}")

                # Calculate resource distribution
                resource_distribution = _get_resource_distribution(best_solution)

                # Calculate task type distribution
                task_type_distribution = _get_task_type_distribution(best_solution)

                # Calculate task execution times
                task_times = [fast_exec_time_estimate(task, resource.id)
                            for task, resource in best_solution.items()]
                avg_time = sum(task_times) / len(task_times) if task_times else 0
                worst_time = max(task_times) if task_times else 0
                best_time = min(task_times) if task_times else 0

                # Create a hash to track solution differences
                solution_hash = hash(frozenset((task.id, resource.id)
                                            for task, resource in best_solution.items()))

                # Calculate load balance score
                load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id])
                                    for res in self.resources])

                # Write GA progress to CSV
                with open(ga_csv_path, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        gen + 1,  # Generation number
                        best_fitness,  # Best fitness
                        sum(fitnesses) / len(fitnesses),  # Average fitness
                        diversity,  # Population diversity
                        current_mutation_rate,  # Mutation rate
                        crossover_rate,  # Crossover rate
                        json.dumps(resource_distribution),  # Tasks per resource
                        load_balance,  # Load balance score
                        solution_hash  # Solution hash
                    ])

                # Record overall solution at end of each generation
                if gen % 5 == 0 or gen == generations - 1:  # Every 5 generations and the last one
                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"GA_Gen_{gen+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping at generation {gen+1} (no improvement for {max_stagnation} generations)")
                    break

            # Record final GA solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_GA_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # === STEP 6: Apply focused Tabu Search to best solution ===
            logger.info("Phase 4/5: Applying focused Tabu Search with Multiple Neighborhood Structures...")

            # Define neighborhood types
            neighborhood_types = ['high_impact', 'swap_pairs', 'cluster_reassign']

            # Create a copy of the best solution
            current_solution = copy.deepcopy(best_solution)
            current_fitness = simplified_fitness(current_solution)
            tabu_best_solution = copy.deepcopy(current_solution)
            tabu_best_fitness = current_fitness

            # Function to identify high-impact tasks (focuses on tasks with largest execution times)
            def identify_high_impact_tasks(solution, limit=20):
                # Calculate execution times for each task
                task_times = []
                for task, resource in solution.items():
                    if resource:
                        exec_time = fast_exec_time_estimate(task, resource.id)
                        task_times.append((task, exec_time))

                # Sort by execution time and return top tasks
                return [task for task, _ in sorted(task_times, key=lambda x: x[1], reverse=True)[:limit]]

            # Run efficient Tabu Search
            stagnation_counter = 0

            for iteration in range(tabu_iterations):
                # Update epsilon for this iteration
                current_ts_epsilon = max(epsilon_end, epsilon_start - iteration * ts_epsilon_decay_rate)
                # Progress indicator every few iterations
                if iteration % 5 == 0 or iteration == tabu_iterations-1:
                    current_cost = simplified_fitness(tabu_best_solution)
                    logger.info(f"Tabu iteration {iteration+1}/{tabu_iterations} - Current best cost: {current_cost}, Epsilon: {current_ts_epsilon:.3f}")

                # Select neighborhood type based on iteration
                # Use epsilon to determine neighborhood size and move selection strategy
                # When epsilon is high: broader neighborhood exploration
                # When epsilon is low: more focused exploitation

                # Select neighborhood type based on epsilon and iteration
                if random.random() < current_ts_epsilon:
                    # More exploration when epsilon is high - randomize neighborhood
                    current_neighborhood = random.randint(0, len(neighborhood_types)-1)
                else:
                    # More exploitation when epsilon is low - use deterministic pattern
                    current_neighborhood = iteration % len(neighborhood_types)
                neighborhood_type = neighborhood_types[current_neighborhood]
                logger.info(f"Using neighborhood type: {neighborhood_type}")

                # Generate and evaluate a limited set of moves
                best_move = None
                best_move_fitness = float('inf')
                best_move_details = {}

                if neighborhood_type == 'high_impact':
                    # Adjust task limit based on epsilon - higher epsilon means more tasks to consider
                    task_limit_base = max(20, int(len(current_solution) * 0.2))
                    task_limit = int(task_limit_base * (1 + current_ts_epsilon))
                    high_impact_tasks = identify_high_impact_tasks(current_solution, limit=task_limit)

                    # For each high-impact task, try a few resources
                    for task in high_impact_tasks:
                        current_resource = current_solution[task]

                        # Epsilon-adjusted resource sampling
                        resource_sample_size = min(int(3 + current_ts_epsilon * 7), len(self.resources))
                        sampled_resources = random.sample(self.resources, resource_sample_size)

                        for resource in sampled_resources:
                            # Check if the move is tabu
                            is_tabu = (task.id, resource.id) in tabu_list
                            # Skip current assignment
                            if resource == current_resource:
                                continue

                            # Evaluate the move
                            current_solution[task] = resource
                            move_fitness = simplified_fitness(current_solution)

                            # *** ASPIRATION CRITERION ***
                            # Allow tabu moves if they lead to the best solution found so far
                            aspiration_satisfied = move_fitness < tabu_best_fitness
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                            if (not is_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and move_fitness < best_move_fitness:
                                best_move = (task, resource)
                                best_move_fitness = move_fitness
                                best_move_details = {
                                    'task_id': task.id,
                                    'task_type': task.type,
                                    'old_resource': current_resource.type,
                                    'new_resource': resource.type,
                                    'fitness_before': current_fitness,
                                    'fitness_after': move_fitness,
                                    'move_type': 'high_impact',
                                    'aspiration_used': is_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                                }

                                # Log if aspiration criterion was used
                                if is_tabu and aspiration_satisfied:
                                    logger.info(f"Aspiration criterion applied: Tabu move allowed for task {task.id} as it improves best solution")

                            # Restore current solution
                            current_solution[task] = current_resource

                elif neighborhood_type == 'swap_pairs':
                    # Swap pairs of tasks between resources
                    # Adjust candidate count based on epsilon
                    all_tasks = list(current_solution.keys())
                    base_candidate_count = min(40, len(all_tasks))
                    candidate_count = min(len(all_tasks), int(base_candidate_count * (1 + current_ts_epsilon)))
                    candidate_tasks = random.sample(all_tasks, candidate_count)
                    # Try swapping pairs
                    for i in range(candidate_count // 2):
                        if i * 2 + 1 >= candidate_count:
                            break

                        task1 = candidate_tasks[i * 2]
                        task2 = candidate_tasks[i * 2 + 1]

                        resource1 = current_solution[task1]
                        resource2 = current_solution[task2]

                        # Skip if tasks are on the same resource
                        if resource1 == resource2:
                            continue

                        # Check if swap is tabu (both directions)
                        swap_tabu = (task1.id, resource2.id) in tabu_list or (task2.id, resource1.id) in tabu_list

                        # Try the swap
                        current_solution[task1] = resource2
                        current_solution[task2] = resource1

                        # Evaluate
                        swap_fitness = simplified_fitness(current_solution)

                        # Aspiration criterion
                        aspiration_satisfied = swap_fitness < tabu_best_fitness

                        # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                        if (not swap_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and swap_fitness < best_move_fitness:
                            best_move = ('swap', task1, task2, resource1, resource2)
                            best_move_fitness = swap_fitness
                            best_move_details = {
                                'task1_id': task1.id,
                                'task2_id': task2.id,
                                'resource1': resource1.type,
                                'resource2': resource2.type,
                                'fitness_before': current_fitness,
                                'fitness_after': swap_fitness,
                                'move_type': 'swap_pairs',
                                'aspiration_used': swap_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                            }

                            # Log if aspiration criterion was used
                            if swap_tabu and aspiration_satisfied:
                                logger.info(f"Aspiration criterion applied: Tabu swap allowed as it improves best solution")

                        # Restore current solution
                        current_solution[task1] = resource1
                        current_solution[task2] = resource2

                elif neighborhood_type == 'cluster_reassign':
                    # Reassign clusters of related tasks
                    # Group tasks by type
                    task_clusters = {}
                    for task in current_solution:
                        if task.type not in task_clusters:
                            task_clusters[task.type] = []
                        task_clusters[task.type].append(task)

                    # For each task type, try reassigning a subset of tasks
                    for task_type, tasks in task_clusters.items():
                        # Skip very large clusters
                        if len(tasks) > 30:
                            subset_base_size = min(10, len(tasks))
                            subset_size = min(len(tasks), int(subset_base_size * (1 + current_ts_epsilon)))
                            tasks_subset = random.sample(tasks, subset_size)
                        else:
                            tasks_subset = tasks

                        # Try each target resource
                        for target_resource in self.resources:
                            # Store original assignments
                            original_assignments = {task: current_solution[task] for task in tasks_subset}
                            cluster_tabu = False

                            # Check if any reassignment would be tabu
                            for task in tasks_subset:
                                if (task.id, target_resource.id) in tabu_list:
                                    cluster_tabu = True
                                    break

                            # Reassign all tasks in subset to target resource
                            for task in tasks_subset:
                                current_solution[task] = target_resource

                            # Evaluate the cluster reassignment
                            cluster_fitness = simplified_fitness(current_solution)

                            # Aspiration criterion
                            aspiration_satisfied = cluster_fitness < tabu_best_fitness

                            # Update best move if better and either not tabu OR aspiration criterion is satisfied
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                            if (not cluster_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and cluster_fitness < best_move_fitness:
                                # Deep copy the subset and assignments for later use
                                best_tasks_subset = tasks_subset.copy()
                                best_original_assignments = copy.deepcopy(original_assignments)

                                best_move = ('cluster', best_tasks_subset, target_resource, best_original_assignments)
                                best_move_fitness = cluster_fitness
                                best_move_details = {
                                    'task_type': task_type,
                                    'cluster_size': len(tasks_subset),
                                    'target_resource': target_resource.type,
                                    'fitness_before': current_fitness,
                                    'fitness_after': cluster_fitness,
                                    'move_type': 'cluster_reassign',
                                    'aspiration_used': cluster_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                                }

                                # Log if aspiration criterion was used
                                if cluster_tabu and aspiration_satisfied:
                                    logger.info(f"Aspiration criterion applied: Tabu cluster move allowed as it improves best solution")

                            # Restore original assignments
                            for task, resource in original_assignments.items():
                                current_solution[task] = resource

                # Apply best move if found
                if best_move:
                    move_type = best_move_details.get('move_type', 'unknown')

                    if move_type == 'high_impact':
                        task, new_resource = best_move
                        old_resource = current_solution[task]
                        current_solution[task] = new_resource

                        # Add to tabu list
                        tabu_list.append((task.id, new_resource.id))

                        logger.info(f"Applied high-impact move: Task {task.id} moved from {old_resource.type} to {new_resource.type}")

                    elif move_type == 'swap_pairs':
                        _, task1, task2, resource1, resource2 = best_move
                        current_solution[task1] = resource2
                        current_solution[task2] = resource1

                        # Add both moves to tabu list
                        tabu_list.append((task1.id, resource2.id))
                        tabu_list.append((task2.id, resource1.id))

                        logger.info(f"Applied swap move: Tasks {task1.id} and {task2.id} swapped resources")

                    elif move_type == 'cluster_reassign':
                        _, tasks_subset, target_resource, original_assignments = best_move

                        # Apply cluster reassignment
                        for task in tasks_subset:
                            current_solution[task] = target_resource
                            # Add to tabu list
                            tabu_list.append((task.id, target_resource.id))

                        logger.info(f"Applied cluster move: {len(tasks_subset)} tasks of type {best_move_details['task_type']} moved to {target_resource.type}")

                    # Update current fitness
                    old_fitness = current_fitness
                    current_fitness = best_move_fitness

                    # Calculate improvement percentage
                    improvement_pct = (old_fitness - current_fitness) / old_fitness * 100 if old_fitness > 0 else 0

                    # Write TS progress to CSV with move type info
                    with open(ts_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            iteration + 1,  # Iteration
                            current_fitness,  # Current fitness
                            tabu_best_fitness,  # Best fitness
                            move_type,  # Move type
                            best_move_details.get('task_id', best_move_details.get('task1_id', '')),  # Task ID
                            best_move_details.get('old_resource', best_move_details.get('resource1', '')),  # Old resource
                            best_move_details.get('new_resource', best_move_details.get('resource2', '')),  # New resource
                            len(tabu_list),  # Tabu list size
                            improvement_pct,  # Improvement percentage
                            hash(frozenset((t.id, r.id) for t, r in current_solution.items()))  # Solution hash
                        ])

                    # Update best solution if improved
                    if current_fitness < tabu_best_fitness:
                        tabu_best_solution = copy.deepcopy(current_solution)
                        tabu_best_fitness = current_fitness
                        stagnation_counter = 0
                    else:
                        stagnation_counter += 1
                else:
                    stagnation_counter += 1

                    # Log iteration with no move
                    with open(ts_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            iteration + 1,  # Iteration
                            current_fitness,  # Current fitness
                            tabu_best_fitness,  # Best fitness
                            "no_move",  # Move type
                            "",  # Task ID
                            "",  # Old resource
                            "",  # New resource
                            len(tabu_list),  # Tabu list size
                            0.0,  # Improvement percentage
                            hash(frozenset((t.id, r.id) for t, r in current_solution.items()))  # Solution hash
                        ])

                # Record overall solution every few iterations
                if iteration % 10 == 0 or iteration == tabu_iterations - 1:
                    # Calculate metrics similar to GA section
                    resource_distribution = _get_resource_distribution(tabu_best_solution)
                    task_type_distribution = _get_task_type_distribution(tabu_best_solution)
                    task_times = [fast_exec_time_estimate(task, resource.id)
                                for task, resource in tabu_best_solution.items()]
                    avg_time = sum(task_times) / len(task_times) if task_times else 0
                    worst_time = max(task_times) if task_times else 0
                    best_time = min(task_times) if task_times else 0
                    load_balance = np.std([len([t for t, r in tabu_best_solution.items() if r.id == res.id])
                                        for res in self.resources])

                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"TS_Iter_{iteration+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            tabu_best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= 5:
                    logger.info(f"Early stopping Tabu Search at iteration {iteration+1} (no improvement for 5 iterations)")
                    break

            # Record final TS solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_TS_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    tabu_best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(tabu_best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(tabu_best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # === STEP 7: Select final solution and apply to resources ===
            logger.info("Phase 5/5: Finalizing solution and distributing tasks...")

            # Validate solution before proceeding
            valid_assignments = 0
            for task, resource in tabu_best_solution.items():
                if resource is not None:
                    valid_assignments += 1

            logger.info(f"Validating tabu solution: {valid_assignments} valid assignments out of {len(tabu_best_solution)} tasks")

            # Check GA solution as well
            ga_valid_assignments = 0
            for task, resource in best_solution.items():
                if resource is not None:
                    ga_valid_assignments += 1

            logger.info(f"Validating GA solution: {ga_valid_assignments} valid assignments out of {len(best_solution)} tasks")
            tabu_cost = simplified_fitness(tabu_best_solution)

            final_solution = tabu_best_solution
            logger.info(f"Using Tabu-improved solution (cost: {tabu_cost}, valid assignments: {valid_assignments})")

            # Calculate execution time
            execution_time = time.time() - start_time

            # Reset resources - explicitly clear queues
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []
                logger.info(f"Reset {resource.type} queue to 0 tasks")

            # Apply solution to resources
            distributed_tasks = []
            assignment_data = []
            base_time = datetime.now()

            # Task distribution counts
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            # Temporary task assignment tracking
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process and apply all assignments, logging each step
            logger.info(f"Processing {len(final_solution)} task assignments...")
            assignment_count = 0

            # CRITICALLY IMPORTANT: Set reference to task's assigned resource
            # This helps maintain the connection between tasks and resources
            for task, resource in final_solution.items():
                # Verify task is valid
                if task is None:
                    logger.error("Found None task in solution!")
                    continue

                # Handle case where resource is None by assigning to least loaded resource
                if resource is None:
                    logger.warning(f"Task {task.id} has no resource assignment, assigning to least loaded resource")
                    continue

                # Set task status and add to queue
                task.status = 'CREATED'  # Ensure correct initial status
                task.assigned_resource = resource  # CRITICAL: Maintain reference to resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1
                assignment_count += 1

                # Debug log for each assignment
                logger.info(f"Assigned Task {task.id} ({task.type}) to {resource.type}, queue now has {len(resource.task_queue)} tasks")

                # Record assignment
                # Record assignment using direct timestamp conversion without offset
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type if resource else 'None',
                    'Estimated Time': fast_exec_time_estimate(task, resource.id) if resource else 'N/A'
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            logger.info(f"Completed {assignment_count} task assignments")

            # Write to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'optimized_hybrid_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as f:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node',
                    'Estimated Time'
                ]
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            logger.info(f"Final solution cost: {simplified_fitness(final_solution)}")
            logger.info(f"Task distribution:")
            for resource_type, count in resource_assignment_counts.items():
                if count > 0:
                    logger.info(f"  {resource_type}: {count} tasks")
            logger.info(f"Results saved to: {csv_filepath}")

            # Record final comparison
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_Applied_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    simplified_fitness(final_solution),  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(resource_assignment_counts),  # Resource distribution (actual applied counts)
                    json.dumps(_get_task_type_distribution(final_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Add detailed debug information to verify task distribution
            total_queued = 0
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]
                logger.info(f"Copied {len(resource.task_queue)} tasks to {resource.type} queue")
                queued = len(resource.task_queue)
                total_queued += queued
                logger.info(f"After distribution: {resource.type} has {queued} tasks in queue")

            logger.info(f"Total tasks queued: {total_queued} of {total_tasks} requested")

            # Ensure we always have the right number of tasks
            # If we don't have enough tasks, apply Tabu-based logic for additional tasks
            if total_queued < total_tasks:
                logger.warning(f"Distribution incomplete: only {total_queued} of {total_tasks} tasks distributed")

                # Calculate how many more tasks we need
                tasks_needed = total_tasks - total_queued
                logger.info(f"Adding {tasks_needed} additional tasks using Tabu optimization logic")

                # Create additional tasks
                additional_tasks = []
                for i in range(tasks_needed):
                    # Get task type
                    # task_type = list(TASK_TYPES.keys())[i % len(TASK_TYPES)]
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    # Create task
                    new_task_id = total_queued + i + 1
                    task = Task(
                        task_id=new_task_id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )

                    # Set arrival time
                    task.arrival_time = datetime.now().timestamp()
                    task.status = 'CREATED'
                    additional_tasks.append(task)

                # Apply Tabu-based assignment logic for the additional tasks
                # First calculate current resource loads from existing assignments
                current_loads = {r.id: len(r.task_queue) for r in self.resources}

                # For each additional task, assign based on the same criteria used in Tabu Search
                for task in additional_tasks:
                    # Find best resource based on type compatibility and load
                    best_resource = None
                    best_cost = float('inf')

                    for resource in self.resources:
                        # Calculate cost based on the same logic used in Tabu Search
                        exec_time = fast_exec_time_estimate(task, resource.id)

                        # Apply the same penalties from the original solution
                        if task.type == "WT3" and resource.type.startswith("Raspberry_"):
                            exec_time *= 3.0
                        elif task.type in ["RT1", "RT3"] and resource.type.startswith("Cloud_"):
                            exec_time *= 0.1

                        # Add load balancing factor
                        load_factor = current_loads[resource.id] * 500
                        total_cost = exec_time + load_factor

                        if total_cost < best_cost:
                            best_cost = total_cost
                            best_resource = resource

                    # Assign to the best resource
                    if best_resource:
                        task.assigned_resource = best_resource
                        best_resource.task_queue.append(task)
                        current_loads[best_resource.id] += 1
                        distributed_tasks.append(task)
                        logger.info(f"Added task {task.id} ({task.type}) to {best_resource.type} using Tabu logic")

                # Verify final count
                final_count = sum(len(r.task_queue) for r in self.resources)
                logger.info(f"Final distribution check: {final_count} of {total_tasks} tasks distributed")

            logger.info(f"Returning {len(distributed_tasks)} distributed tasks")

            # Return CSV paths alongside the distributed tasks
            return distributed_tasks, {
                'ga_csv': ga_csv_path,
                'ts_csv': ts_csv_path,
                'solution_csv': solution_csv_path,
                'final_assignments': csv_filepath
            }

        except Exception as e:
            logger.error(f"Error in Optimized Hybrid algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())

            # Just raise the exception instead of falling back to round-robin
            logger.error("Hybrid algorithm failed without fallback mechanism enabled")
            raise
    def _GA_GWO_hybrid_algorithm(self, total_tasks: int) -> List[Task]:
        """
        Optimized hybrid Genetic Algorithm and Grey Wolf Optimizer for task sets (2000-5000 tasks).
        This version is significantly faster while maintaining solution quality.

        Key optimizations:
        1. Faster initialization with smaller population
        2. Reduced generations with early convergence detection
        3. Grey Wolf Optimizer for refining the GA solution
        4. Simplified fitness function with multi-objective cost calculation
        5. Better progress tracking throughout execution

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        # Log start time and show progress indicator
        start_time = time.time()
        logger.info(f"\n⏳ Starting Optimized Hybrid GA-GWO Algorithm for {total_tasks} tasks...")
        logger.info("Phase 1/5: Task generation and initialization...")

        # Initialize CSV tracking for algorithm progress
        csv_folder = "/content/drive/My Drive/EdgeSimPy/algorithm_tracking"
        os.makedirs(csv_folder, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        ga_csv_path = os.path.join(csv_folder, f'ga_evolution_{timestamp}.csv')
        gwo_csv_path = os.path.join(csv_folder, f'gwo_search_{timestamp}.csv')
        solution_csv_path = os.path.join(csv_folder, f'solution_comparison_{timestamp}.csv')

        # Initialize CSV files with headers
        with open(ga_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Generation', 'Best_Fitness', 'Average_Fitness', 'Population_Diversity',
                            'Mutation_Rate', 'Crossover_Rate', 'Tasks_Per_Resource',
                            'Load_Balance_Score', 'Solution_Hash'])

        with open(gwo_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Iteration', 'Current_Fitness', 'Best_Fitness', 'Alpha_Fitness',
                            'Beta_Fitness', 'Delta_Fitness', 'Parameter_a', 'Wolf_Count',
                            'Solution_Hash'])

        with open(solution_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Algorithm_Phase', 'Timestamp', 'Solution_Cost', 'Load_Balance_Score',
                            'Resource_Distribution', 'Task_Type_Distribution', 'Average_Execution_Time',
                            'Worst_Task_Time', 'Best_Task_Time'])

        try:
            # === STEP 1: Generate tasks with batch processing for efficiency ===
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with cached requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                    'reliability': resource.reliability
                } for resource in self.resources
            }

            # Helper methods for tracking metrics
            def _calculate_population_diversity(population):
                """Calculate population diversity as percentage of different assignments"""
                if len(population) <= 1:
                    return 0

                # Take the first solution as reference
                reference = population[0]
                total_assignments = len(reference) * (len(population) - 1)
                different_assignments = 0

                # Compare each solution with the reference
                for solution in population[1:]:
                    for task in reference:
                        if task in solution and reference[task] != solution[task]:
                            different_assignments += 1

                return (different_assignments / total_assignments) * 100 if total_assignments > 0 else 0

            def _get_resource_distribution(solution):
                """Get distribution of tasks per resource"""
                distribution = {resource.type: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if resource:
                        distribution[resource.type] += 1

                return distribution

            def _get_task_type_distribution(solution):
                """Get distribution of task types per resource type"""
                distribution = {}

                for task, resource in solution.items():
                    if resource:
                        resource_type = resource.type.split('_')[0]  # Cloud, Smartphone, or Raspberry
                        task_type = task.type

                        if resource_type not in distribution:
                            distribution[resource_type] = {}

                        if task_type not in distribution[resource_type]:
                            distribution[resource_type][task_type] = 0

                        distribution[resource_type][task_type] += 1

                return distribution
        # === NEW MAKESPAN WEIGHTED SUM FUNCTION ===
            def makespan_weighted_sum_fitness(solution, weights=None):
                """
                Multi-objective weighted sum fitness function focusing on:
                1. Makespan (total completion time)
                2. Average resource utilization

                Args:
                    solution: Task-to-resource mapping
                    weights: Optional weights for different objectives

                Returns:
                    float: Weighted sum fitness value (lower is better)
                """
                if weights is None:
                    # Default weights - fixed for consistent optimization
                    weights = {
                        'makespan': 0.65,
                        'resource_utilization': 0.35
                    }

                # 1. Makespan calculation
                # Track earliest completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task arrival order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        continue

                    # Calculate task execution time
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Task starts at either its arrival time or when the resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + total_task_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                # 2. Average resource utilization calculation
                # Track total processing time for each resource
                resource_total_processing = {resource.id: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if not resource:
                        continue

                    # Calculate processing time for this task
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Add to resource's total processing time
                    resource_total_processing[resource.id] += total_task_time

                # Calculate average utilization across all resources
                if makespan > 0:
                    resource_utilizations = [processing_time / makespan for processing_time in resource_total_processing.values()]
                    avg_resource_utilization = sum(resource_utilizations) / len(self.resources)

                    # Convert to cost: higher utilization should have LOWER cost
                    utilization_cost = 1.0 / (avg_resource_utilization + 0.001)  # Add small constant to avoid division by zero
                else:
                    utilization_cost = 100000  # High penalty for zero makespan (shouldn't happen)

                # Calculate weighted sum of all objectives
                total_cost = (
                    weights['makespan'] * makespan +
                    weights['resource_utilization'] * utilization_cost
                )

                return total_cost

            # Faster task creation with larger batch size
            batch_size = 500
            initial_tasks = []
            resource_loads = {r.id: 0 for r in self.resources}

            # Show progress for task creation
            logger.info(f"Creating {total_tasks} tasks in batches of {batch_size}...")

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Instead of cycling through task types:
                    # task_type = list(TASK_TYPES.keys())[j % len(TASK_TYPES)]
                    # Use random selection:
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # Special handling for very small task counts (like 5)
            if total_tasks <= 10:
                logger.info(f"Small task count ({total_tasks}) detected - using specialized optimization parameters")
                # Smaller population, fewer generations for tiny problems
                population_size = max(5, min(10, total_tasks))
                generations = 3
                gwo_iterations = max(5, min(10, total_tasks))
                max_stagnation = 2
            else:
            # === STEP 2: Adjusted optimization parameters for larger task counts ===
                population_size = min(50, int(total_tasks * 0.1))  # Proportional to total tasks
                generations = min(20, int(total_tasks * 0.05))     # More generations for larger problems
                gwo_iterations = min(100, int(total_tasks * 0.05))  # Increased iterations
                max_stagnation = 10    # Increased early stopping patience

            # With these adaptive mutation parameters:
            base_mutation_rate = 0.15
            min_mutation_rate = 0.1
            max_mutation_rate = 0.4
            current_mutation_rate = base_mutation_rate
            diversity_threshold_low = 15  # Percentage below which to increase mutation
            diversity_threshold_high = 40  # Percentage above which to decrease mutation
            crossover_rate = 0.8
            elite_count = 2

            # === STEP 3: Define optimized utility functions ===
            def fast_exec_time_estimate(task, resource_id):
                """Simplified execution time estimation for speed"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Basic estimate combining transfer and processing time
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']
                process_time = task.total_cpu_required / resource['cpu_rating']

                # Simple reliability factor
                reliability_factor = 1.0 + (1.0 - resource['reliability']) * 0.5

                # Simplified total time calculation
                return (transfer_time + process_time) * reliability_factor

            def simplified_fitness(solution):
                """
                Multi-objective cost function focusing on:
                1. Task distribution efficiency
                2. Load balancing
                3. Resource type compatibility
                4. Task execution characteristics
                """
                # Calculate total task execution times with penalties
                total_execution_cost = 0
                task_counts = {}
                resource_loads = {}

                # Specific task type and resource compatibility coefficients
                type_resource_penalties = {
                    'WT3': {
                        'Raspberry_': 3.0,  # Heavy penalty for WT3 on Raspberry Pi
                        'Cloud_': 0.5,      # Slight bonus for WT3 on cloud
                        'Smartphone_': 1.5  # Moderate penalty on smartphones
                    },
                    'RT1': {
                        'Cloud_': 0.1,      # Preferred placement
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    },
                    'RT3': {
                        'Cloud_': 0.1,      # Preferred placement
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    }
                }

                # Process each task in the solution
                for task, resource in solution.items():
                    if not resource:
                        # Heavy penalty for unassigned tasks
                        total_execution_cost += 100000
                        continue

                    # Basic execution time estimation
                    exec_time = (task.input_size + task.output_size + task.total_cpu_required / resource.total_cpu_rating)

                    # Adjust cost based on task-resource compatibility
                    penalty_multiplier = type_resource_penalties.get(task.type, {}).get(resource.type.split('_')[0] + '_', 1.0)
                    exec_time *= penalty_multiplier

                    # Track task counts per resource
                    task_counts[resource.id] = task_counts.get(resource.id, 0) + 1
                    resource_loads[resource.id] = resource_loads.get(resource.id, 0) + exec_time

                    total_execution_cost += exec_time

                # Load balancing penalty
                if task_counts:
                    # Standard deviation of task counts
                    count_std = np.std(list(task_counts.values())) * 500
                    # Standard deviation of resource loads
                    load_std = np.std(list(resource_loads.values())) * 1000

                    total_execution_cost += count_std + load_std
                # Add the weighted sum component for makespan optimization (10% of total fitness)
                #weighted_sum_component = makespan_weighted_sum_fitness(solution)
                total_execution_cost = 1.0 * total_execution_cost # + 0.1 * weighted_sum_component
                # Scale down the total cost for manageable values
                total_execution_cost *= 0.5

                return total_execution_cost
            # Add these functions after simplified_fitness() and before the GA population initialization

            # === GWO HELPER FUNCTIONS ===
            def initialize_diverse_wolves(best_solution, gwo_population_size, resources, ga_population):
                """Create a more diverse initial wolf population by using:
                1. The best GA solution
                2. Some variations of the best solution
                3. Other solutions from the GA population
                4. Some entirely random solutions
                """
                gwo_population = [copy.deepcopy(best_solution)]  # Alpha wolf

                # Add some top solutions from GA population (if available)
                if ga_population and len(ga_population) > 3:
                    ga_sorted_indices = sorted(range(len(ga_population)),
                                            key=lambda i: simplified_fitness(ga_population[i]))
                    # Add top 3 solutions (that are different from best)
                    solutions_added = 0
                    for idx in ga_sorted_indices[1:min(7, len(ga_sorted_indices))]:  # Look at top 7
                        if solutions_added >= 3:  # Only add 3 max
                            break

                        # Check if this solution is sufficiently different from best
                        solution_diff = sum(1 for task in best_solution
                                        if task in ga_population[idx] and best_solution[task] != ga_population[idx][task])
                        if solution_diff > len(best_solution) * 0.2:  # At least 20% different
                            gwo_population.append(copy.deepcopy(ga_population[idx]))
                            solutions_added += 1

                # Fill the first half with variations of best solution
                half_point = max(2, gwo_population_size // 2)
                while len(gwo_population) < half_point:
                    wolf = copy.deepcopy(best_solution)
                    # Use more aggressive mutation to ensure diversity
                    mutation_pct = 0.1 + (0.3 * len(gwo_population) / half_point)  # 10-40% mutation
                    tasks_to_mutate = random.sample(
                        list(wolf.keys()),
                        k=max(1, int(len(wolf) * mutation_pct))
                    )

                    for task in tasks_to_mutate:
                        wolf[task] = random.choice(resources)

                    gwo_population.append(wolf)

                # Fill second half with more random solutions to ensure search space coverage
                while len(gwo_population) < gwo_population_size:
                    if len(gwo_population) < gwo_population_size * 0.8:  # For first 80% of this half
                        # Start with best solution but apply very heavy mutation
                        wolf = copy.deepcopy(best_solution)
                        mutation_pct = 0.4 + (0.4 * (len(gwo_population) - half_point) / (gwo_population_size - half_point))
                        tasks_to_mutate = random.sample(
                            list(wolf.keys()),
                            k=max(1, int(len(wolf) * mutation_pct))
                        )

                        for task in tasks_to_mutate:
                            wolf[task] = random.choice(resources)
                    else:
                        # For last 20%, create completely random solutions
                        wolf = {}
                        for task in best_solution.keys():
                            wolf[task] = random.choice(resources)

                    gwo_population.append(wolf)

                return gwo_population

            def adaptive_parameter_control(iteration, total_iterations, stagnation_counter, best_fitness, avg_fitness):
                """
                Adaptively control GWO parameters based on search progress

                Args:
                    iteration: Current iteration number
                    total_iterations: Total number of iterations
                    stagnation_counter: How many iterations without improvement
                    best_fitness: Current best fitness
                    avg_fitness: Average fitness of the population

                Returns:
                    Tuple of (a, C_adapt) parameters
                """
                # Base calculation for parameter 'a' (typically decreases from 2 to 0)
                progress = iteration / total_iterations

                # Standard linear decrease
                a_linear = 2.0 - 2.0 * progress

                # If we're seeing stagnation, increase 'a' to promote exploration
                stagnation_factor = min(0.5, stagnation_counter / 10)  # Cap at 0.5

                # Calculate convergence as ratio between best and average fitness
                # When they're close, population has converged
                if avg_fitness > 0:
                    convergence_ratio = best_fitness / avg_fitness
                else:
                    convergence_ratio = 0.9  # Assume partial convergence if avg_fitness is 0

                # If population is too converged, increase 'a' to break out
                convergence_factor = 0
                if convergence_ratio > 0.9:  # 90% convergence
                    convergence_factor = (convergence_ratio - 0.9) * 5  # Scale up to 0.5 max

                # Combine factors, ensuring 'a' is within reasonable range
                a = a_linear + stagnation_factor + convergence_factor

                # Constrain 'a' to reasonable range
                a = max(0.1, min(2.5, a))  # Never let it go below 0.1 or above 2.5

                # Adaptive C parameter - controls emphasis on leaders
                # Higher C means more emphasis on following alpha, beta, delta
                C_adapt = 1.0
                if stagnation_counter > 5:
                    # If stagnating, reduce emphasis on following leaders
                    C_adapt = max(0.5, 1.0 - (stagnation_counter - 5) / 20)

                return a, C_adapt

            def update_wolf_positions(gwo_population, alpha_wolf, beta_wolf, delta_wolf, a, C_adapt, resources, iteration, total_iterations, initial_tasks):
                """
                More sophisticated wolf position updating strategy that balances exploration and exploitation
                """
                new_wolves = []

                # For each wolf in the pack
                for wolf_idx, wolf in enumerate(gwo_population):
                    # Create a new wolf solution
                    new_wolf = {}

                    # Determine if this wolf should explore or exploit based on its rank
                    is_leader = wolf_idx <= 2  # Alpha, beta, or delta
                    is_explorer = wolf_idx >= len(gwo_population) * 0.8  # Last 20% are explorers

                    # For each task, update its assignment
                    for task in initial_tasks:
                        # Decide strategy based on wolf type and iteration progress
                        if is_leader:
                            # Leaders make smaller adjustments with higher probability of keeping their current assignment
                            # They focus on fine-tuning their solutions
                            if random.random() < 0.7:  # 70% chance to keep current assignment
                                new_wolf[task] = wolf.get(task, random.choice(resources))
                            else:
                                # When they do change, they typically adopt alpha's solution
                                new_wolf[task] = alpha_wolf.get(task, random.choice(resources))

                        elif is_explorer:
                            # Explorers maintain high diversity regardless of iteration
                            if random.random() < 0.5:  # 50% chance for random assignment
                                new_wolf[task] = random.choice(resources)
                            else:
                                # Otherwise follow standard GWO logic with high A values for exploration
                                A_explorer = a * (1.5 + random.random())  # Higher A value for exploration

                                # Weighted random selection between leader wolves and random exploration
                                r_val = random.random()
                                if r_val < 0.4:
                                    new_wolf[task] = alpha_wolf.get(task, random.choice(resources))
                                elif r_val < 0.6:
                                    new_wolf[task] = beta_wolf.get(task, random.choice(resources))
                                elif r_val < 0.8:
                                    new_wolf[task] = delta_wolf.get(task, random.choice(resources))
                                else:
                                    new_wolf[task] = random.choice(resources)

                        else:
                            # Standard wolves - implement core GWO position updating with leader influence
                            # Components of GWO formula - randomized coefficients
                            A1 = (random.random() * 2 * a) - a
                            A2 = (random.random() * 2 * a) - a
                            A3 = (random.random() * 2 * a) - a

                            C1 = random.random() * 2 * C_adapt
                            C2 = random.random() * 2 * C_adapt
                            C3 = random.random() * 2 * C_adapt

                            # Calculate influence factors based on A and C values
                            # These determine how much each leader's position affects this wolf
                            infl_alpha = abs(C1 - abs(A1))
                            infl_beta = abs(C2 - abs(A2))
                            infl_delta = abs(C3 - abs(A3))

                            # Calculate total influence
                            total_infl = infl_alpha + infl_beta + infl_delta

                            # Normalize influences
                            if total_infl > 0:
                                infl_alpha /= total_infl
                                infl_beta /= total_infl
                                infl_delta /= total_infl
                            else:
                                # Default values if all influences are 0
                                infl_alpha, infl_beta, infl_delta = 0.6, 0.3, 0.1

                            # Probabilistic assignment based on normalized influences
                            r_val = random.random()
                            if r_val < infl_alpha:
                                new_wolf[task] = alpha_wolf.get(task, random.choice(resources))
                            elif r_val < infl_alpha + infl_beta:
                                new_wolf[task] = beta_wolf.get(task, random.choice(resources))
                            elif r_val < infl_alpha + infl_beta + infl_delta:
                                new_wolf[task] = delta_wolf.get(task, random.choice(resources))
                            else:
                                # Small chance for random exploration
                                new_wolf[task] = random.choice(resources)

                    # Special handling for middle-rank wolves (between leaders and explorers)
                    # Occasionally perform targeted mutation to optimize specific task types
                    if not is_leader and not is_explorer and random.random() < 0.3:
                        # Select a random task type to focus on
                        task_types = list(set(task.type for task in initial_tasks))
                        target_type = random.choice(task_types)

                        # Find all tasks of this type
                        type_tasks = [task for task in initial_tasks if task.type == target_type]

                        # If we have tasks of this type, mutate some of them
                        if type_tasks:
                            # Mutate a percentage of tasks of this type
                            mutation_count = max(1, int(len(type_tasks) * 0.5))  # Mutate up to 50%
                            tasks_to_mutate = random.sample(type_tasks, k=min(mutation_count, len(type_tasks)))

                            for task in tasks_to_mutate:
                                # For these targeted mutations, try to use the best resource for this task type
                                # from the alpha wolf's solution
                                new_wolf[task] = alpha_wolf.get(task, random.choice(resources))

                    new_wolves.append(new_wolf)

                return new_wolves

            def handle_stagnation(gwo_population, gwo_fitnesses, stagnation_counter, total_iterations, iteration, gwo_best_solution, resources, initial_tasks):
                """
                Advanced stagnation handling to help GWO escape local optima
                """
                # If we're stagnating for too long
                if stagnation_counter >= 5:
                    # Calculate how severe the intervention should be
                    severity = min(1.0, stagnation_counter / 15)  # Scale from 0 to 1

                    # Also consider remaining iterations
                    remaining_iterations = total_iterations - iteration
                    # If we're very stagnant but still have many iterations left, intervention should be more aggressive
                    if remaining_iterations > total_iterations * 0.3:  # More than 30% iterations left
                        severity += 0.2

                    logger.info(f"Stagnation detected for {stagnation_counter} iterations - applying disruption (severity: {severity:.2f})")

                    # Store best solution before disruption
                    preserved_best = copy.deepcopy(gwo_best_solution)

                    # Modify solutions based on severity
                    for i, wolf in enumerate(gwo_population):
                        if i == 0 and severity < 0.7:
                            # Protect alpha wolf in mild interventions (low severity)
                            continue

                        # Calculate mutation percentage based on severity and wolf rank
                        mutation_pct = severity * (0.2 + (0.6 * i / len(gwo_population)))

                        # Select tasks to mutate
                        tasks_to_mutate = random.sample(
                            list(wolf.keys()),
                            k=max(1, int(len(wolf) * mutation_pct))
                        )

                        # Apply different mutation strategies
                        for task in tasks_to_mutate:
                            # Strategy selection
                            strategy = random.random()

                            if strategy < 0.2:
                                # Random reassignment
                                wolf[task] = random.choice(resources)
                            elif strategy < 0.5:
                                # Targeted mutation based on task type
                                task_type = task.type
                                # Select resource based on typical preferences for this task type
                                # (simplified version - you could expand with task-specific logic)
                                if task_type.startswith("RT"):
                                    # RT tasks might prefer cloud resources
                                    cloud_resources = [r for r in resources if r.type.startswith("Cloud_")]
                                    if cloud_resources:
                                        wolf[task] = random.choice(cloud_resources)
                                    else:
                                        wolf[task] = random.choice(resources)
                                else:
                                    wolf[task] = random.choice(resources)
                            else:
                                # Swap with another task's resource
                                other_task = random.choice(list(wolf.keys()))
                                if other_task != task:
                                    wolf[task], wolf[other_task] = wolf[other_task], wolf[task]

                    # Always preserve best solution by replacing worst wolf
                    # This ensures we don't lose progress entirely
                    if severity < 0.9:  # Unless intervention is very severe
                        worst_idx = gwo_fitnesses.index(max(gwo_fitnesses))
                        gwo_population[worst_idx] = copy.deepcopy(preserved_best)

                    # Reset stagnation counter if using this technique
                    return 0

                # No intervention needed
                return stagnation_counter

            class GWOTracker:
                """Utility class to track GWO optimization progress"""

                def __init__(self, total_iterations):
                    self.iteration_history = []
                    self.best_fitness_history = []
                    self.avg_fitness_history = []
                    self.wolf_diversity_history = []
                    self.a_param_history = []
                    self.total_iterations = total_iterations

                def update(self, iteration, gwo_population, gwo_fitnesses, a_param):
                    """Record metrics for current iteration"""
                    self.iteration_history.append(iteration)

                    # Best and average fitness
                    best_fitness = min(gwo_fitnesses)
                    avg_fitness = sum(gwo_fitnesses) / len(gwo_fitnesses)
                    self.best_fitness_history.append(best_fitness)
                    self.avg_fitness_history.append(avg_fitness)

                    # Calculate wolf diversity (percentage of different assignments)
                    diversity = self._calculate_wolf_diversity(gwo_population)
                    self.wolf_diversity_history.append(diversity)

                    # Record a parameter
                    self.a_param_history.append(a_param)

                    # Log status periodically
                    if iteration % 5 == 0 or iteration == self.total_iterations - 1:
                        logger.info(f"GWO Iteration {iteration+1}/{self.total_iterations}: "
                                f"Best={best_fitness:.2f}, Avg={avg_fitness:.2f}, "
                                f"Diversity={diversity:.1f}%, a={a_param:.2f}")

                def _calculate_wolf_diversity(self, population):
                    """Calculate diversity as percentage of different assignments between wolves"""
                    if len(population) <= 1:
                        return 0

                    # Take alpha wolf as reference
                    reference = population[0]
                    total_comparisons = 0
                    different_assignments = 0

                    # Compare each wolf with the reference
                    for wolf in population[1:]:
                        for task in reference:
                            if task in wolf:
                                total_comparisons += 1
                                if reference[task] != wolf[task]:
                                    different_assignments += 1

                    if total_comparisons > 0:
                        return (different_assignments / total_comparisons) * 100
                    return 0

            # === STEP 4: Create smart initial solution ===
            logger.info("Phase 2/5: Creating initial population...")

            def create_bandwidth_aware_solution():
                """Generate initial solution with data size awareness"""
                solution = {}
                high_bw_resources = [r for r in self.resources if r.total_bandwidth >= 800]
                med_bw_resources = [r for r in self.resources if 400 <= r.total_bandwidth < 800]
                low_bw_resources = [r for r in self.resources if r.total_bandwidth < 400]

                # Track loads for balancing
                local_loads = {r.id: 0 for r in self.resources}

                # Sort tasks by arrival time
                for task in sorted(initial_tasks, key=lambda t: t.arrival_time):
                    # Assign based on data size needs
                    if task.input_size > 2.0 or task.output_size > 2.0:
                        candidates = high_bw_resources if high_bw_resources else self.resources
                    elif task.input_size > 0.5 or task.output_size > 0.5:
                        candidates = med_bw_resources + high_bw_resources if med_bw_resources or high_bw_resources else self.resources
                    else:
                        candidates = self.resources

                    # Select least loaded resource
                    resource = min(candidates, key=lambda r: local_loads[r.id])
                    solution[task] = resource
                    local_loads[resource.id] += 1

                return solution

            # Create initial population
            population = []
            smart_solution = create_bandwidth_aware_solution()
            # Log the smart solution to verify it contains valid assignments
            logger.info(f"Created smart solution with {len(smart_solution)} assignments")
            population.append(smart_solution)

            # Generate diverse initial population
            for i in range(1, population_size):
                # Create variation of smart solution
                solution = copy.deepcopy(smart_solution)

                # Mutate a percentage of assignments
                mutation_pct = 0.1 + (0.2 * i / population_size)  # More mutation for later solutions
                tasks_to_mutate = random.sample(
                    list(solution.keys()),
                    k=max(1, int(len(solution) * mutation_pct))
                )

                for task in tasks_to_mutate:
                    solution[task] = random.choice(self.resources)

                population.append(solution)

            # === STEP 5: Run Optimized Genetic Algorithm ===
            logger.info("Phase 3/5: Running genetic algorithm optimization...")

            # Evaluate initial population
            fitnesses = [simplified_fitness(sol) for sol in population]

            # Track best solution
            best_idx = fitnesses.index(min(fitnesses))
            best_solution = copy.deepcopy(population[best_idx])
            best_fitness = fitnesses[best_idx]
            best_cost = simplified_fitness(best_solution)

            # Record initial state for the solution comparison CSV
            initial_resource_distribution = json.dumps(_get_resource_distribution(best_solution))
            initial_task_type_distribution = json.dumps(_get_task_type_distribution(best_solution))
            initial_task_times = [fast_exec_time_estimate(task, resource.id) for task, resource in best_solution.items()]
            avg_time = sum(initial_task_times) / len(initial_task_times) if initial_task_times else 0
            worst_time = max(initial_task_times) if initial_task_times else 0
            best_time = min(initial_task_times) if initial_task_times else 0

            # Calculate initial load balance
            load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id]) for res in self.resources])

            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Initial_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    initial_resource_distribution,  # Resource distribution
                    initial_task_type_distribution,  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Run genetic algorithm with progress updates
            stagnation_counter = 0

            for gen in range(generations):
                # Progress indicator
                current_cost = simplified_fitness(best_solution)
                logger.info(f"Generation {gen+1}/{generations} - Current best cost: {current_cost}")

                # Sort by fitness and retain elites
                sorted_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i])
                new_population = [copy.deepcopy(population[i]) for i in sorted_indices[:elite_count]]

                # Fill remaining population with crossover and mutation
                while len(new_population) < population_size:
                    # Tournament selection
                    tournament_size = min(3, len(population))
                    parent1_idx = min(random.sample(range(len(population)), tournament_size),
                                    key=lambda i: fitnesses[i])
                    parent2_idx = min(random.sample(range(len(population)), tournament_size),
                                    key=lambda i: fitnesses[i])

                    parent1 = population[parent1_idx]
                    parent2 = population[parent2_idx]

                    # Simplified crossover
                    if random.random() < crossover_rate:
                        child = {}

                        # Create a random crossover point
                        tasks_ordered = sorted(parent1.keys(), key=lambda t: t.arrival_time)
                        crossover_point = random.randint(1, len(tasks_ordered) - 1)

                        # Take first part from parent1, second from parent2
                        for i, task in enumerate(tasks_ordered):
                            if i < crossover_point:
                                child[task] = parent1[task]
                            else:
                                # Make sure task exists in parent2
                                if task in parent2:
                                    child[task] = parent2[task]
                                else:
                                    child[task] = parent1[task]
                    else:
                        # No crossover, just clone parent1
                        child = copy.deepcopy(parent1)

                    # Simple mutation
                    if random.random() < current_mutation_rate:
                        mutation_count = max(1, int(len(child) * 0.1))  # Mutate about 10%
                        tasks_to_mutate = random.sample(list(child.keys()), k=mutation_count)

                        for task in tasks_to_mutate:
                            child[task] = random.choice(self.resources)

                    new_population.append(child)

                # Update population
                population = new_population

                # Calculate fitness - evaluate each solution
                fitnesses = [simplified_fitness(sol) for sol in population]

                # Check for improvement
                current_best_idx = fitnesses.index(min(fitnesses))
                current_best_fitness = fitnesses[current_best_idx]

                if current_best_fitness < best_fitness:
                    best_solution = copy.deepcopy(population[current_best_idx])
                    best_fitness = current_best_fitness
                    best_cost = simplified_fitness(best_solution)
                    logger.info(f"✓ New best solution found! Fitness: {best_fitness}, Cost: {best_cost}")
                    stagnation_counter = 0
                else:
                    stagnation_counter += 1

                # Calculate diversity (percentage of different assignments between solutions)
                diversity = _calculate_population_diversity(population)

                if diversity < diversity_threshold_low:
                    # Low diversity - increase mutation rate to encourage exploration
                    current_mutation_rate = min(max_mutation_rate, current_mutation_rate * 1.5)
                    logger.info(f"Low diversity ({diversity:.1f}%) - increasing mutation rate to {current_mutation_rate:.3f}")
                elif diversity > diversity_threshold_high:
                    # High diversity - decrease mutation rate to encourage exploitation
                    current_mutation_rate = max(min_mutation_rate, current_mutation_rate * 0.8)
                    logger.info(f"High diversity ({diversity:.1f}%) - decreasing mutation rate to {current_mutation_rate:.3f}")

                # Calculate resource distribution
                resource_distribution = _get_resource_distribution(best_solution)

                # Calculate task type distribution
                task_type_distribution = _get_task_type_distribution(best_solution)

                # Calculate task execution times
                task_times = [fast_exec_time_estimate(task, resource.id)
                            for task, resource in best_solution.items()]
                avg_time = sum(task_times) / len(task_times) if task_times else 0
                worst_time = max(task_times) if task_times else 0
                best_time = min(task_times) if task_times else 0

                # Create a hash to track solution differences
                solution_hash = hash(frozenset((task.id, resource.id)
                                            for task, resource in best_solution.items()))

                # Calculate load balance score
                load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id])
                                    for res in self.resources])

                # Write GA progress to CSV
                with open(ga_csv_path, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        gen + 1,  # Generation number
                        best_fitness,  # Best fitness
                        sum(fitnesses) / len(fitnesses),  # Average fitness
                        diversity,  # Population diversity
                        current_mutation_rate,  # Mutation rate
                        crossover_rate,  # Crossover rate
                        json.dumps(resource_distribution),  # Tasks per resource
                        load_balance,  # Load balance score
                        solution_hash  # Solution hash
                    ])

                # Record overall solution at end of each generation
                if gen % 5 == 0 or gen == generations - 1:  # Every 5 generations and the last one
                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"GA_Gen_{gen+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping at generation {gen+1} (no improvement for {max_stagnation} generations)")
                    break

            # Record final GA solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_GA_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # === STEP 6: Apply Enhanced GWO to the best GA solution ===
            logger.info("Phase 4/5: Applying Enhanced Grey Wolf Optimization to refine GA solution...")

            # Parameters for improved GWO
            gwo_population_size = min(40, int(total_tasks * 0.015))  # Increased population size
            gwo_iterations = min(150, int(total_tasks * 0.07))  # More iterations to avoid premature convergence
            max_stagnation = 15  # Increased patience for stagnation

            # Create initial GWO population with much more diversity
            gwo_population = initialize_diverse_wolves(best_solution, gwo_population_size,
                                                    self.resources, population)
            current_fitness = simplified_fitness(best_solution)
            gwo_best_solution = copy.deepcopy(best_solution)
            gwo_best_fitness = current_fitness

            # Initialize tracker for visualization and debugging
            tracker = GWOTracker(gwo_iterations)

            # Setup for tracking
            stagnation_counter = 0
            best_fitness_history = []
            fitness_improvement = 0
            last_improvement_iter = 0

            # Evaluate initial wolves
            gwo_fitnesses = [simplified_fitness(wolf) for wolf in gwo_population]

            # Run enhanced GWO optimization
            for iteration in range(gwo_iterations):
                # Sort wolves by fitness to identify alpha, beta, and delta
                sorted_indices = sorted(range(len(gwo_fitnesses)), key=lambda i: gwo_fitnesses[i])
                alpha_wolf = gwo_population[sorted_indices[0]]
                beta_wolf = gwo_population[sorted_indices[1]] if len(sorted_indices) > 1 else alpha_wolf
                delta_wolf = gwo_population[sorted_indices[2]] if len(sorted_indices) > 2 else beta_wolf

                # Calculate adaptive parameters based on search progress
                avg_fitness = sum(gwo_fitnesses) / len(gwo_fitnesses)
                a, C_adapt = adaptive_parameter_control(
                    iteration, gwo_iterations, stagnation_counter,
                    gwo_fitnesses[sorted_indices[0]], avg_fitness
                )

                # Update tracker
                tracker.update(iteration, gwo_population, gwo_fitnesses, a)

                # Update wolf positions with improved strategy
                new_wolves = update_wolf_positions(
                    gwo_population, alpha_wolf, beta_wolf, delta_wolf,
                    a, C_adapt, self.resources, iteration, gwo_iterations, initial_tasks
                )

                # Evaluate new wolves
                new_fitnesses = [simplified_fitness(wolf) for wolf in new_wolves]

                # Update best solution if improved
                best_idx = new_fitnesses.index(min(new_fitnesses))
                current_best_fitness = new_fitnesses[best_idx]

                if current_best_fitness < gwo_best_fitness:
                    # Calculate improvement percentage
                    improvement_pct = (gwo_best_fitness - current_best_fitness) / gwo_best_fitness * 100

                    # Only consider significant improvements (avoid numerical noise)
                    if improvement_pct > 0.01:  # 0.01% threshold
                        previous_best = gwo_best_fitness
                        gwo_best_solution = copy.deepcopy(new_wolves[best_idx])
                        gwo_best_fitness = current_best_fitness

                        # Log improvement
                        logger.info(f"✓ GWO found improved solution at iteration {iteration+1}: "
                                f"{previous_best:.2f} → {gwo_best_fitness:.2f} "
                                f"(improvement: {improvement_pct:.2f}%)")

                        # Reset stagnation counter
                        stagnation_counter = 0
                        last_improvement_iter = iteration
                        fitness_improvement += improvement_pct
                    else:
                        stagnation_counter += 1
                else:
                    stagnation_counter += 1

                # Record best fitness for history
                best_fitness_history.append(gwo_best_fitness)

                # Handle stagnation if needed
                if stagnation_counter >= 5:
                    stagnation_counter = handle_stagnation(
                        new_wolves, new_fitnesses, stagnation_counter,
                        gwo_iterations, iteration, gwo_best_solution,
                        self.resources, initial_tasks
                    )

                # Early stopping condition - if we've made significant improvement and
                # haven't improved for many iterations
                significant_improvement = fitness_improvement > 5.0  # 5% total improvement
                long_stagnation = stagnation_counter > max_stagnation
                near_end = iteration > gwo_iterations * 0.7  # 70% through iterations

                if significant_improvement and long_stagnation and near_end:
                    logger.info(f"Early stopping at iteration {iteration+1}: Significant improvement "
                            f"({fitness_improvement:.2f}%) achieved and no improvement for "
                            f"{stagnation_counter} iterations")
                    break

                # Update wolves for next iteration
                gwo_population = new_wolves
                gwo_fitnesses = new_fitnesses

                # Record progress to CSV
                if iteration % 10 == 0 or iteration == gwo_iterations - 1:
                    # Calculate metrics for logging
                    resource_distribution = _get_resource_distribution(gwo_best_solution)
                    task_type_distribution = _get_task_type_distribution(gwo_best_solution)
                    task_times = [fast_exec_time_estimate(task, resource.id)
                                for task, resource in gwo_best_solution.items()]
                    avg_time = sum(task_times) / len(task_times) if task_times else 0
                    worst_time = max(task_times) if task_times else 0
                    best_time = min(task_times) if task_times else 0
                    load_balance = np.std([len([t for t, r in gwo_best_solution.items() if r.id == res.id])
                                        for res in self.resources])

                    # Record solution metrics
                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"GWO_Iter_{iteration+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            gwo_best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                    # Log GWO iteration details
                    solution_hash = hash(frozenset((t.id, r.id) for t, r in gwo_best_solution.items()))
                    with open(gwo_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            iteration + 1,  # Iteration
                            gwo_best_fitness,  # Current fitness
                            gwo_best_fitness,  # Best fitness
                            gwo_fitnesses[sorted_indices[0]] if sorted_indices else 'N/A',  # Alpha fitness
                            gwo_fitnesses[sorted_indices[1]] if len(sorted_indices) > 1 else 'N/A',  # Beta fitness
                            gwo_fitnesses[sorted_indices[2]] if len(sorted_indices) > 2 else 'N/A',  # Delta fitness
                            a,  # Parameter a
                            gwo_population_size,  # Wolf count
                            solution_hash  # Solution hash
                        ])

            # Final refinement phase - apply makespan optimization to best solution
            logger.info("Running final refinement using makespan optimization...")

            # Save original best solution
            original_gwo_best = copy.deepcopy(gwo_best_solution)
            original_gwo_fitness = simplified_fitness(original_gwo_best)

            # Create variations of best solution to improve makespan
            refinement_population = [gwo_best_solution]

            # Create 10 variations with targeted modifications
            for i in range(10):
                solution = copy.deepcopy(gwo_best_solution)

                # Focus on tasks with highest execution times
                task_exec_times = [(task, fast_exec_time_estimate(task, resource.id))
                                for task, resource in solution.items()]
                task_exec_times.sort(key=lambda x: x[1], reverse=True)

                # Select top 10-20% of tasks with highest execution times
                top_tasks = [task for task, _ in task_exec_times[:int(len(task_exec_times) * (0.1 + (i * 0.01)))]]

                # Try to find better assignments for these tasks
                for task in top_tasks:
                    current_resource = solution[task]
                    best_resource = current_resource
                    best_time = fast_exec_time_estimate(task, current_resource.id)

                    # Try each resource to find the best one for this task
                    for resource in self.resources:
                        if resource != current_resource:
                            exec_time = fast_exec_time_estimate(task, resource.id)
                            if exec_time < best_time:
                                best_time = exec_time
                                best_resource = resource

                    # Update assignment if better resource found
                    if best_resource != current_resource:
                        solution[task] = best_resource

                # Add to refinement population
                refinement_population.append(solution)

            # Evaluate all refinement solutions with makespan_weighted_sum_fitness
            refinement_fitnesses = [makespan_weighted_sum_fitness(sol) for sol in refinement_population]
            best_idx = refinement_fitnesses.index(min(refinement_fitnesses))
            makespan_best_solution = refinement_population[best_idx]

            # Compare with original GWO solution
            makespan_fitness = simplified_fitness(makespan_best_solution)
            if makespan_fitness < original_gwo_fitness:
                # If makespan optimization improved the solution, use it
                gwo_best_solution = makespan_best_solution
                gwo_best_fitness = makespan_fitness
                logger.info(f"✓ Final refinement improved solution: {original_gwo_fitness:.2f} → {makespan_fitness:.2f}")
            else:
                logger.info("Final refinement did not improve the solution, keeping original GWO result")

            # Record final GWO solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_GWO_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    gwo_best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(gwo_best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(gwo_best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Calculate improvement over GA solution
            if best_fitness > 0:
                improvement_percentage = ((best_fitness - gwo_best_fitness) / best_fitness) * 100
                logger.info(f"GWO optimization improved the GA solution by {improvement_percentage:.2f}%")
            else:
                logger.info("Could not calculate improvement percentage (invalid fitness values)")
            # === STEP 7: Select final solution and apply to resources ===
            logger.info("Phase 5/5: Finalizing solution and distributing tasks...")

            # Validate solution before proceeding
            valid_assignments = 0
            for task, resource in final_solution.items():
                if resource is not None:
                    valid_assignments += 1

            logger.info(f"Validating final solution: {valid_assignments} valid assignments out of {len(final_solution)} tasks")

            # Calculate execution time
            execution_time = time.time() - start_time

            # Reset resources - explicitly clear queues
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []
                logger.info(f"Reset {resource.type} queue to 0 tasks")

            # Apply solution to resources
            distributed_tasks = []
            assignment_data = []
            base_time = datetime.now()

            # Task distribution counts
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            # Temporary task assignment tracking
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process and apply all assignments, logging each step
            logger.info(f"Processing {len(final_solution)} task assignments...")
            assignment_count = 0

            # CRITICALLY IMPORTANT: Set reference to task's assigned resource
            # This helps maintain the connection between tasks and resources
            for task, resource in final_solution.items():
                # Verify task is valid
                if task is None:
                    logger.error("Found None task in solution!")
                    continue

                # Handle case where resource is None by assigning to least loaded resource
                if resource is None:
                    logger.warning(f"Task {task.id} has no resource assignment, assigning to least loaded resource")
                    continue

                # Set task status and add to queue
                task.status = 'CREATED'  # Ensure correct initial status
                task.assigned_resource = resource  # CRITICAL: Maintain reference to resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1
                assignment_count += 1

                # Debug log for each assignment
                logger.info(f"Assigned Task {task.id} ({task.type}) to {resource.type}, queue now has {len(resource.task_queue)} tasks")

                # Record assignment
                # Record assignment using direct timestamp conversion without offset
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type if resource else 'None',
                    'Estimated Time': fast_exec_time_estimate(task, resource.id) if resource else 'N/A'
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            logger.info(f"Completed {assignment_count} task assignments")

            # Write to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'optimized_hybrid_ga_gwo_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as f:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node',
                    'Estimated Time'
                ]
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            logger.info(f"Final solution cost: {simplified_fitness(final_solution)}")
            logger.info(f"Task distribution:")
            for resource_type, count in resource_assignment_counts.items():
                if count > 0:
                    logger.info(f"  {resource_type}: {count} tasks")
            logger.info(f"Results saved to: {csv_filepath}")

            # Record final comparison
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_Applied_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    simplified_fitness(final_solution),  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(resource_assignment_counts),  # Resource distribution (actual applied counts)
                    json.dumps(_get_task_type_distribution(final_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Add detailed debug information to verify task distribution
            total_queued = 0
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]
                logger.info(f"Copied {len(resource.task_queue)} tasks to {resource.type} queue")
                queued = len(resource.task_queue)
                total_queued += queued
                logger.info(f"After distribution: {resource.type} has {queued} tasks in queue")

            logger.info(f"Total tasks queued: {total_queued} of {total_tasks} requested")

            # Ensure we always have the right number of tasks
            # If we don't have enough tasks, apply GWO-based logic for additional tasks
            if total_queued < total_tasks:
                logger.warning(f"Distribution incomplete: only {total_queued} of {total_tasks} tasks distributed")

                # Calculate how many more tasks we need
                tasks_needed = total_tasks - total_queued
                logger.info(f"Adding {tasks_needed} additional tasks using GWO optimization logic")

                # Create additional tasks
                additional_tasks = []
                for i in range(tasks_needed):
                    # Get task type
                    # task_type = list(TASK_TYPES.keys())[i % len(TASK_TYPES)]
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    # Create task
                    new_task_id = total_queued + i + 1
                    task = Task(
                        task_id=new_task_id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )

                    # Set arrival time
                    task.arrival_time = datetime.now().timestamp()
                    task.status = 'CREATED'
                    additional_tasks.append(task)

                # Apply GWO-based assignment logic for the additional tasks
                # First calculate current resource loads from existing assignments
                current_loads = {r.id: len(r.task_queue) for r in self.resources}

                # For each additional task, assign based on the same criteria used
                for task in additional_tasks:
                    # Find best resource based on type compatibility and load
                    best_resource = None
                    best_cost = float('inf')

                    for resource in self.resources:
                        # Calculate cost based on the same logic used in fitness function
                        exec_time = fast_exec_time_estimate(task, resource.id)

                        # Apply the same penalties from the original solution
                        if task.type == "WT3" and resource.type.startswith("Raspberry_"):
                            exec_time *= 3.0
                        elif task.type in ["RT1", "RT3"] and resource.type.startswith("Cloud_"):
                            exec_time *= 0.1

                        # Add load balancing factor
                        load_factor = current_loads[resource.id] * 500
                        total_cost = exec_time + load_factor

                        if total_cost < best_cost:
                            best_cost = total_cost
                            best_resource = resource

                    # Assign to the best resource
                    if best_resource:
                        task.assigned_resource = best_resource
                        best_resource.task_queue.append(task)
                        current_loads[best_resource.id] += 1
                        distributed_tasks.append(task)
                        logger.info(f"Added task {task.id} ({task.type}) to {best_resource.type} using GWO logic")

                # Verify final count
                final_count = sum(len(r.task_queue) for r in self.resources)
                logger.info(f"Final distribution check: {final_count} of {total_tasks} tasks distributed")

            logger.info(f"Returning {len(distributed_tasks)} distributed tasks")

            # Return CSV paths alongside the distributed tasks
            return distributed_tasks, {
                'ga_csv': ga_csv_path,
                'gwo_csv': gwo_csv_path,
                'solution_csv': solution_csv_path,
                'final_assignments': csv_filepath
            }

        except Exception as e:
            logger.error(f"Error in Optimized Hybrid GA-GWO algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())

            # Just raise the exception instead of falling back to round-robin
            logger.error("Hybrid algorithm failed without fallback mechanism enabled")
            raise
    def _hybrid_gwo_tabu_distribution(self, total_tasks: int) -> List[Task]:
        """
        Hybrid Grey Wolf Optimizer and Tabu Search for task scheduling in edge-cloud environments.

        This algorithm combines the global exploration capabilities of Grey Wolf Optimization with
        the local exploitation strengths of Tabu Search to find an efficient task distribution.

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        start_time = time.time()
        logger.info(f"\n⏳ Starting Hybrid GWO-Tabu Search Algorithm for {total_tasks} tasks...")

        try:
            # === STEP 1: Generate tasks and initialize task types ===
            logger.info("Phase 1/6: Task generation and initialization...")
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                    'reliability': resource.reliability
                } for resource in self.resources
            }

            # Create tasks with batch processing
            batch_size = 500
            initial_tasks = []

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Instead of cycling through task types:
                    # task_type = list(TASK_TYPES.keys())[j % len(TASK_TYPES)]

                    # Use random selection:
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # === STEP 2: Define optimization utility functions ===
            logger.info("Phase 2/6: Setting up optimization parameters...")

            def calculate_execution_time(task, resource_id):
                """Calculate task execution time on a given resource"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Calculate transfer time (input + output)
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']

                # Calculate processing time
                process_time = task.total_cpu_required / resource['cpu_rating']

                # Apply reliability factor
                reliability_factor = 1.0 + (1.0 - resource['reliability']) * 0.5

                # Total execution time
                return (transfer_time + process_time) * reliability_factor

            def calculate_objective_function(solution):
                """
                Multi-objective function that considers:
                1. Task execution time
                2. Load balancing
                3. Task-resource compatibility
                4. Resource utilization
                """
                total_cost = 0
                task_counts = {}
                resource_loads = {}

                # Task-resource compatibility weights
                compatibility_weights = {
                    'WT3': {
                        'Raspberry_': 3.0,  # Heavy penalty for WT3 on Raspberry Pi
                        'Cloud_': 0.5,      # Bonus for WT3 on cloud
                        'Smartphone_': 1.5  # Moderate penalty on smartphones
                    },
                    'RT1': {
                        'Cloud_': 0.1,      # Strong preference for RT1 on cloud
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    },
                    'RT3': {
                        'Cloud_': 0.1,      # Strong preference for RT3 on cloud
                        'Smartphone_': 2.0, # Less preferred
                        'Raspberry_': 3.0   # Least preferred
                    }
                }

                # Calculate execution cost for each task-resource assignment
                for task, resource in solution.items():
                    if not resource:
                        # Penalize unassigned tasks heavily
                        total_cost += 100000
                        continue

                    # Calculate base execution time
                    exec_time = calculate_execution_time(task, resource.id)

                    # Adjust cost based on task-resource compatibility
                    task_type = task.type
                    resource_type = resource.type.split('_')[0] + '_'

                    # Apply compatibility weights if defined
                    if task_type in compatibility_weights and resource_type in compatibility_weights[task_type]:
                        exec_time *= compatibility_weights[task_type][resource_type]

                    # Track resource load
                    task_counts[resource.id] = task_counts.get(resource.id, 0) + 1
                    resource_loads[resource.id] = resource_loads.get(resource.id, 0) + exec_time

                    # Add to total cost
                    total_cost += exec_time

                # Load balancing penalty based on standard deviation
                if task_counts:
                    # Penalize uneven task distribution
                    count_std = np.std(list(task_counts.values())) * 500
                    # Penalize uneven resource load
                    load_std = np.std(list(resource_loads.values())) * 1000

                    total_cost += count_std + load_std

                return total_cost

            # === STEP 3: Initialize Grey Wolf Optimizer parameters ===
            logger.info("Phase 3/6: Initializing Grey Wolf Optimizer...")

            # GWO parameters
            num_wolves = min(30, int(total_tasks * 0.05))  # Number of search agents
            max_iterations = min(25, int(total_tasks * 0.02))  # Maximum iterations

            # Function to create an initial solution
            def create_initial_solution():
                """Create a balanced initial solution for wolves"""
                solution = {}
                resource_loads = {r.id: 0 for r in self.resources}

                # Group resources by type for better assignment
                cloud_resources = [r for r in self.resources if r.type.startswith("Cloud_")]
                smartphone_resources = [r for r in self.resources if r.type.startswith("Smartphone_")]
                raspberry_resources = [r for r in self.resources if r.type.startswith("Raspberry_")]

                for task in initial_tasks:
                    # Match task types to preferred resources
                    if task.type in ['RT1', 'RT3']:
                        # Prefer cloud for these types
                        preferred_resources = cloud_resources if cloud_resources else self.resources
                    elif task.type == 'WT3':
                        # Avoid Raspberry Pi for WT3
                        preferred_resources = smartphone_resources + cloud_resources if smartphone_resources or cloud_resources else self.resources
                    else:
                        # Use all resources with preference to less loaded ones
                        preferred_resources = self.resources

                    # Assign to least loaded resource among preferred ones
                    if preferred_resources:
                        resource = min(preferred_resources, key=lambda r: resource_loads[r.id])
                        solution[task] = resource
                        resource_loads[resource.id] += 1
                    else:
                        # Fallback to any resource
                        resource = min(self.resources, key=lambda r: resource_loads[r.id])
                        solution[task] = resource
                        resource_loads[resource.id] += 1

                return solution

            # Initialize the wolf pack with solutions
            wolf_pack = []
            for i in range(num_wolves):
                solution = create_initial_solution()
                # Add some randomization for diversity
                if i > 0:  # Keep the first solution unchanged
                    # Randomly reassign some tasks for diversity
                    mutation_rate = 0.1 + (0.3 * i / num_wolves)  # Increase mutation for later wolves
                    tasks_to_mutate = random.sample(
                        list(solution.keys()),
                        k=max(1, int(len(solution) * mutation_rate))
                    )
                    for task in tasks_to_mutate:
                        solution[task] = random.choice(self.resources)

                # Evaluate the solution
                fitness = calculate_objective_function(solution)
                wolf_pack.append({"solution": solution, "fitness": fitness})

            # Sort wolves by fitness (ascending - lower is better)
            wolf_pack.sort(key=lambda w: w["fitness"])

            # Initialize alpha, beta, and delta wolves (best three solutions)
            alpha = wolf_pack[0]
            beta = wolf_pack[1] if len(wolf_pack) > 1 else alpha
            delta = wolf_pack[2] if len(wolf_pack) > 2 else beta

            logger.info(f"Initial alpha wolf fitness: {alpha['fitness']:.2f}")

            # === STEP 4: Run Grey Wolf Optimizer ===
            logger.info("Phase 4/6: Running Grey Wolf Optimization...")

            for iteration in range(max_iterations):
                # Update parameter a (linearly decreases from 2 to 0)
                a = 2 - iteration * (2 / max_iterations)

                # Update each wolf's position (solution)
                for i in range(num_wolves):
                    # Get current wolf
                    wolf = wolf_pack[i]
                    new_solution = {}

                    # For each task, update its assignment based on alpha, beta, delta
                    for task in initial_tasks:
                        # Calculate the influence of each leader wolf

                        # Randomly select some positions from alpha, beta, delta solutions
                        # to create diversification and influence the current wolf's solution
                        r1, r2, r3 = random.random(), random.random(), random.random()

                        # Determine which leader wolves to follow for this task
                        if r1 < 0.6:  # 60% chance to follow alpha
                            new_solution[task] = alpha["solution"].get(task, random.choice(self.resources))
                        elif r2 < 0.5:  # 30% chance to follow beta (0.6 + 0.4*0.5 = 0.8)
                            new_solution[task] = beta["solution"].get(task, random.choice(self.resources))
                        elif r3 < 0.5:  # 10% chance to follow delta (0.8 + 0.2*0.5 = 0.9)
                            new_solution[task] = delta["solution"].get(task, random.choice(self.resources))
                        else:  # 10% chance to explore randomly
                            new_solution[task] = random.choice(self.resources)

                    # Evaluate the new solution
                    new_fitness = calculate_objective_function(new_solution)

                    # Update the wolf if the new solution is better
                    if new_fitness < wolf["fitness"]:
                        wolf_pack[i] = {"solution": new_solution, "fitness": new_fitness}

                # Re-sort the wolf pack
                wolf_pack.sort(key=lambda w: w["fitness"])

                # Update alpha, beta, delta wolves
                new_alpha = wolf_pack[0]
                new_beta = wolf_pack[1] if len(wolf_pack) > 1 else new_alpha
                new_delta = wolf_pack[2] if len(wolf_pack) > 2 else new_beta

                # Log progress if alpha improved
                if new_alpha["fitness"] < alpha["fitness"]:
                    logger.info(f"Iteration {iteration+1}/{max_iterations}: New alpha fitness: {new_alpha['fitness']:.2f}")

                alpha, beta, delta = new_alpha, new_beta, new_delta

            # Get the best solution from GWO (alpha wolf)
            best_gwo_solution = alpha["solution"]
            best_gwo_fitness = alpha["fitness"]

            logger.info(f"GWO completed. Best fitness: {best_gwo_fitness:.2f}")

            # === STEP 5: Enhance with Tabu Search ===
            logger.info("Phase 5/6: Enhancing solution with Tabu Search...")

            # Tabu Search parameters
            tabu_tenure = min(50, int(total_tasks * 0.03))
            tabu_iterations = min(40, int(total_tasks * 0.05))
            tabu_list = collections.deque(maxlen=tabu_tenure)

            # Start with the best GWO solution
            current_solution = copy.deepcopy(best_gwo_solution)
            current_fitness = best_gwo_fitness

            # Initialize best Tabu solution
            tabu_best_solution = copy.deepcopy(current_solution)
            tabu_best_fitness = current_fitness

            # Function to identify high-impact tasks
            def identify_high_impact_tasks(solution, limit=20):
                # Calculate execution times for each task
                task_times = []
                for task, resource in solution.items():
                    if resource:
                        exec_time = calculate_execution_time(task, resource.id)
                        task_times.append((task, exec_time))

                # Sort by execution time and return top tasks
                return [task for task, _ in sorted(task_times, key=lambda x: x[1], reverse=True)[:limit]]

            # Run Tabu Search
            stagnation_counter = 0
            max_stagnation = 10

            for iteration in range(tabu_iterations):
                # Focus on high-impact tasks
                task_limit = max(20, int(len(current_solution) * 0.15))
                high_impact_tasks = identify_high_impact_tasks(current_solution, limit=task_limit)

                # Generate and evaluate moves
                best_move = None
                best_move_fitness = float('inf')

                # For each high-impact task, try moving to different resources
                for task in high_impact_tasks:
                    current_resource = current_solution[task]

                    # Try moving to each resource
                    for resource in self.resources:
                        # Skip current assignment and tabu moves
                        if resource == current_resource or (task.id, resource.id) in tabu_list:
                            continue

                        # Make the move
                        current_solution[task] = resource

                        # Evaluate the move
                        move_fitness = calculate_objective_function(current_solution)

                        # Update best move if better
                        if move_fitness < best_move_fitness:
                            best_move = (task, resource)
                            best_move_fitness = move_fitness

                        # Restore the solution
                        current_solution[task] = current_resource

                # Apply best move if found
                if best_move:
                    task, resource = best_move
                    current_solution[task] = resource
                    current_fitness = best_move_fitness

                    # Add move to tabu list
                    tabu_list.append((task.id, resource.id))

                    # Update best tabu solution if improved
                    if current_fitness < tabu_best_fitness:
                        tabu_best_solution = copy.deepcopy(current_solution)
                        tabu_best_fitness = current_fitness
                        stagnation_counter = 0
                        logger.info(f"Tabu iteration {iteration+1}/{tabu_iterations}: New best fitness: {tabu_best_fitness:.2f}")
                    else:
                        stagnation_counter += 1
                else:
                    stagnation_counter += 1

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping Tabu Search at iteration {iteration+1} (no improvement for {max_stagnation} iterations)")
                    break

            # === STEP 6: Finalize solution and apply to resources ===
            logger.info("Phase 6/6: Finalizing and distributing tasks...")

            # Compare GWO and Tabu solutions
            logger.info(f"GWO solution fitness: {best_gwo_fitness:.2f}")
            logger.info(f"Tabu enhanced solution fitness: {tabu_best_fitness:.2f}")

            # Select the better solution
            final_solution = tabu_best_solution if tabu_best_fitness < best_gwo_fitness else best_gwo_solution
            final_fitness = min(tabu_best_fitness, best_gwo_fitness)

            # Calculate execution time
            execution_time = time.time() - start_time
            logger.info(f"Total algorithm execution time: {execution_time:.2f} seconds")

            # Reset resources
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []

            # Apply the final solution
            distributed_tasks = []
            assignment_data = []
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process all assignments
            for task, resource in final_solution.items():
                if task is None or resource is None:
                    continue

                # Set task status and maintain reference
                task.status = 'CREATED'
                task.assigned_resource = resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1

                # Record the assignment
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type,
                    'Estimated Time': calculate_execution_time(task, resource.id)
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            # Update resource queues
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]

            # Save assignments to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'hybrid_gwo_tabu_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as csv_file:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time'
                ]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Log distribution summary
            total_tasks_assigned = sum(resource_assignment_counts.values())
            logger.info("\n=== Task Distribution Summary ===")
            logger.info(f"Total tasks assigned: {total_tasks_assigned}/{total_tasks}")

            for resource_type, count in sorted(resource_assignment_counts.items()):
                percent = (count / total_tasks_assigned) * 100 if total_tasks_assigned > 0 else 0
                logger.info(f"  {resource_type}: {count} tasks ({percent:.1f}%)")

            logger.info(f"Final solution fitness: {final_fitness:.2f}")
            logger.info(f"Results saved to: {csv_filepath}")

            return distributed_tasks

        except Exception as e:
            logger.error(f"Error in Hybrid GWO-Tabu Search algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())
            raise
    def _tabu_search_distribution(self, total_tasks: int) -> List[Task]:
        """
        Optimized Tabu Search for large task sets (up to 8000 tasks) - Sequential Version
        """
        # Step 1: Generate tasks with optimized batch processing
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Define task types with cached requirements
        TASK_TYPES = {
            "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        }

        # Pre-compute resource capabilities with additional metrics
        RESOURCE_CAPABILITIES = {
            resource.id: {
                'type': resource.type,
                'cpu_rating': resource.total_cpu_rating,
                'bandwidth': resource.total_bandwidth,
                'is_cloud': resource.type.startswith("Cloud_"),
                'memory': resource.total_memory,
                'efficiency_score': resource.total_cpu_rating / resource.total_bandwidth
            } for resource in self.resources
        }
        def calculate_adaptive_tabu_tenure(current_solution, recent_improvements):
            """
            Calculate adaptive tabu tenure based on recent improvement rates

            Args:
                current_solution: Current task-to-resource assignments
                recent_improvements: Rate of improvement over recent iterations

            Returns:
                int: Adjusted tabu tenure value
            """
            base_tenure = min(int(len(current_solution) * 0.025), 800)
            if recent_improvements < 0.001:  # Stuck in local optimum
                return int(base_tenure * 1.5)  # Increase tenure to force exploration
            return base_tenure
        # Optimize task object creation with batch processing
        initial_tasks = []
        task_lookup = {}
        batch_size = 500  # Process tasks in larger batches

        for i in range(0, total_tasks, batch_size):
            batch_end = min(i + batch_size, total_tasks)
            batch_tasks = []

            for j in range(i, batch_end):
                task_record = tasks[j]
                # Instead of cycling through task types:
                # task_type = list(TASK_TYPES.keys())[j % len(TASK_TYPES)]

                # Use random selection:
                task_type = random.choice(list(TASK_TYPES.keys()))
                specs = TASK_TYPES[task_type]

                task = Task(
                    task_id=task_record.id,
                    task_type=task_type,
                    input_size=specs["input_size"],
                    output_size=specs["output_size"],
                    cpu_required=specs["cpu_required"]
                )
                task.arrival_time = task_record.arrival_time
                batch_tasks.append(task)
                task_lookup[task.id] = task

            initial_tasks.extend(batch_tasks)

        # Adjust parameters for large-scale optimization
        tabu_tenure = min(int(total_tasks * 0.025), 800)  # Dynamic scaling

        # With this adaptive implementation:
        improvement_history = []  # Track improvement rates
        current_tenure = min(int(total_tasks * 0.025), 800)  # Initial tenure
        max_iterations = min(int(total_tasks * 0.0125), 800)
        max_stagnation = 60
        convergence_window = 100
        convergence_threshold = 0.0005

        tabu_list = collections.deque(maxlen=current_tenure)
        resource_loads = {r.id: 0 for r in self.resources}
        cost_history = []

        def quick_exec_time_estimate(task, resource_id):
            """Enhanced execution time estimation"""
            resource = RESOURCE_CAPABILITIES[resource_id]

            # Fast path for cloud resources
            if resource['is_cloud'] and task.type in ['RT1', 'RT3']:
                return task.total_cpu_required / resource['cpu_rating']

            # Basic estimate combining transfer and processing time
            transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']
            process_time = task.total_cpu_required / resource['cpu_rating']

            return transfer_time + process_time

        def calculate_batch_cost(assignments):
            """Sequential batch cost calculation"""
            total_cost = 0
            resource_times = {r.id: 0 for r in self.resources}


            # Process assignments sequentially
            for task, resource in assignments.items():
                if not resource:
                    total_cost += 1000000  # Penalty for unassigned tasks
                    continue

                exec_time = quick_exec_time_estimate(task, resource.id)
                current_time = resource_times[resource.id]
                completion_time = max(current_time, task.arrival_time) + exec_time
                resource_times[resource.id] = completion_time
                total_cost += exec_time

                # Added Specific penalty so that WT3 will be discouraged to be assigned to Raspberry nodes
                if task.type == "WT3" and resource.type.startswith("Raspberry_"):
                    total_cost += exec_time * 3 # 5x penalty makes Raspberry Pi cost higher than other alternatives
                # Cloud penalty
                elif resource.type.startswith("Cloud_") and task.type not in ['RT1', 'RT3']:
                    total_cost += exec_time * 0.5

            # Load balancing penalty
            loads = {r.id: 0 for r in self.resources}
            for _, resource in assignments.items():
                if resource:
                    loads[resource.id] += 1

            std_dev = np.std(list(loads.values()))
            balance_penalty = std_dev * 1000

            return total_cost + balance_penalty

        def calculate_dynamic_moves(total_tasks):
            base_moves = 100
            # Scale with task count but cap at reasonable maximum
            additional_moves = min(int(total_tasks * 0.05), 900)
            return base_moves + additional_moves

        def generate_efficient_moves(current_solution):
            """
            Generate selective neighborhood moves based on task execution times and resource loads.

            This version implements dynamic move scaling based on problem size while
            maintaining efficient memory usage and move diversity.

            Args:
                current_solution: Dictionary mapping tasks to their currently assigned resources

            Returns:
                List of (task, new_resource) tuples representing possible moves
            """
            # Calculate dynamic move limit
            total_moves = calculate_dynamic_moves(len(current_solution))
            moves = []

            # Select tasks with highest execution times
            task_times = [
                (task, quick_exec_time_estimate(task, current_solution[task].id))
                for task in current_solution
                if current_solution[task]
            ]

            # Calculate 40% of total tasks for consideration
            num_candidates = max(int(len(task_times) * 0.40), 1)
            candidate_tasks = [
                task for task, _ in sorted(task_times,
                key=lambda x: x[1], reverse=True)[:num_candidates]
            ]

            logger.info(f"Selected {len(candidate_tasks)} candidate tasks (40% of {len(task_times)} total tasks)")

            # Generate moves up to the dynamic limit
            while len(moves) < total_moves:
                # Randomly select a candidate task
                task = random.choice(candidate_tasks)
                current_resource = current_solution[task]

                # Select a potential resource
                potential_resources = [
                    r for r in sorted(self.resources, key=lambda r: resource_loads[r.id])
                    if r != current_resource and (task, r) not in tabu_list
                ]

                if potential_resources:
                    new_resource = random.choice(potential_resources)
                    move = (task, new_resource)
                    if move not in moves:  # Avoid duplicate moves
                        moves.append(move)

            return moves
        def generate_smart_initial_solution():
            """Generate initial solution with improved distribution"""
            solution = {}
            cloud_resources = [r for r in self.resources if r.type.startswith("Cloud_")]
            edge_resources = [r for r in self.resources if not r.type.startswith("Cloud_")]

            # Sort tasks by requirements
            sorted_tasks = sorted(
                initial_tasks,
                key=lambda t: (t.total_cpu_required + (t.input_size + t.output_size) * 1024),
                reverse=True
            )

            for task in sorted_tasks:
                if task.type in ['RT1', 'RT3']:
                    resource = min(cloud_resources, key=lambda r: resource_loads[r.id])
                else:
                    candidates = edge_resources if edge_resources else cloud_resources
                    resource = min(candidates, key=lambda r: resource_loads[r.id])

                solution[task] = resource
                resource_loads[resource.id] += 1

            return solution

        # Main optimization loop
        logger.info(f"Starting Tabu Search with {total_tasks} tasks")
        current_solution = generate_smart_initial_solution()
        best_solution = current_solution.copy()
        best_cost = calculate_batch_cost(current_solution)

        stagnation_counter = 0

        for iteration in range(max_iterations):
    # Calculate recent improvement rate
            if len(cost_history) >= convergence_window:
                recent_improvements = (cost_history[-convergence_window] - cost_history[-1]) / cost_history[-convergence_window]
                improvement_history.append(recent_improvements)

                # Update tabu tenure adaptively
                current_tenure = calculate_adaptive_tabu_tenure(
                    current_solution,
                    sum(improvement_history[-10:]) / min(10, len(improvement_history))  # Average recent improvements
                )
                tabu_list = collections.deque(list(tabu_list), maxlen=current_tenure)

            moves = generate_efficient_moves(current_solution)
            if not moves:
                break

            # Sequential move evaluation
            best_move = None
            best_move_cost = float('inf')

            for move in moves:
                task, new_resource = move
                temp_solution = current_solution.copy()
                temp_solution[task] = new_resource
                move_cost = calculate_batch_cost(temp_solution)

                if move_cost < best_move_cost:
                    best_move = move
                    best_move_cost = move_cost

            # Apply best move
            if best_move:
                task, new_resource = best_move
                current_solution[task] = new_resource
                tabu_list.append(best_move)

                if best_move_cost < best_cost:
                    best_solution = current_solution.copy()
                    best_cost = best_move_cost
                    stagnation_counter = 0
                else:
                    stagnation_counter += 1

            # Check convergence
            cost_history.append(best_cost)
            if len(cost_history) >= convergence_window:
                improvement = (cost_history[-convergence_window] - cost_history[-1]) / cost_history[-convergence_window]
                if improvement < convergence_threshold:
                    logger.info(f"Convergence reached at iteration {iteration}")
                    break

            # Early stopping check
            if stagnation_counter >= max_stagnation:
                logger.info(f"Early stopping at iteration {iteration} due to stagnation")
                break

            if iteration % 10 == 0:
                logger.info(f"Iteration {iteration}: Current best cost = {best_cost}")

        # Convert solution to task assignments
        distributed_tasks = []
        assignment_data = []
        base_time = datetime.now()

        # Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Apply final solution
        for task, resource in best_solution.items():
            if resource:
                task.status = 'READY'
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'
                task.failure_reason = "No compatible resource found"
                least_loaded = min(self.resources, key=lambda r: len(r.task_queue))
                least_loaded.failed_tasks.append(task)

            arrival_time = datetime.fromtimestamp(task.arrival_time)
            record = {
                'Task ID': task.id,
                'Type': task.type,
                'Input Size (GB)': task.input_size,
                'Output Size (GB)': task.output_size,
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': resource.type if resource else 'None',
                'Estimated Time': quick_exec_time_estimate(task, resource.id) if resource else 'N/A'
            }
            assignment_data.append(record)
            distributed_tasks.append(task)

        # Write final assignments to CSV
        self._write_tabu_assignments_to_csv(assignment_data)

        logger.info(f"Tabu Search completed. Distributed {len(distributed_tasks)} tasks")
        return distributed_tasks
    def _shortest_job_first_distribution(self, total_tasks: int) -> List[Task]:
        """
        Distribute tasks using Shortest Job First approach. Tasks are sorted by estimated
        execution time before distribution, while maintaining original task validation logic.
        """
        # Step 1: Generate tasks and arrival times
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Create temporary tasks with execution time estimates
        task_estimates = []
        for task_record in tasks:
            task_type = task_types[task_record.id % len(task_types)]

            # Create temporary task for estimation
            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            # Find minimum execution time across all compatible resources
            min_execution_time = float('inf')
            for resource in self.resources:
                can_process, _ = resource.can_process_task(temp_task)
                if can_process:
                    estimated_time = temp_task.estimate_execution_time(resource)
                    min_execution_time = min(min_execution_time, estimated_time)

            task_estimates.append({
                'task_record': task_record,
                'task_type': task_type,
                'estimated_time': min_execution_time
            })

        # Step 3: Sort tasks by estimated execution time
        sorted_tasks = sorted(task_estimates, key=lambda x: x['estimated_time'])

        # Step 4: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 5: Pre-validate tasks (keeping original validation logic)
        for task_info in sorted_tasks:
            task_type = task_info['task_type']
            task_record = task_info['task_record']

            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 6: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 7: Distribute sorted tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_info in sorted_tasks:
            task_record = task_info['task_record']
            task_type = task_info['task_type']
            resource = self.resources[resource_index]

            # Create the actual task
            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time

            arrival_time = base_time + timedelta(seconds=task_record.arrival_time)
            assigned_node = resource.type

            # Check if resource can process this task
            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            # Record task assignment
            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_type["input_size"],
                'Output Size': task_type["output_size"],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node,
                'Estimated Time': task_info['estimated_time']
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)

        # Logging and verification steps
        logger.info("\n=== Task Distribution Summary (SJF) ===")
        logger.info(f"Total tasks requested: {total_tasks}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_sjf_csv(csv_data)

        return distributed_tasks
    def _round_robin_distribution(self, total_tasks: int) -> List[Task]:
        """
        Distribute tasks using Round Robin approach
        """
        # Step 1: Generate tasks and arrival times
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Step 1: Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 3: Pre-validate tasks
        for task_record in tasks:
            # task_type = task_types[task_record.id % len(task_types)]
            task_type = random.choice(task_types)
            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            # Create temporary task for estimation
            temp_task = Task(
                task_id=task_record.id,
                task_type=random.choice(task_types)["type"],  # Randomly select task type
                input_size=random.choice(task_types)["input_size"],
                output_size=random.choice(task_types)["output_size"],
                cpu_required=random.choice(task_types)["cpu_required"]
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 4: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 5: Distribute tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_record in tasks:
            resource = self.resources[resource_index]

            # Determine task type
            # task_type = task_types[task_record.id % len(task_types)]
            task_type = random.choice(task_types)

            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time

            arrival_time = base_time + timedelta(seconds=task_record.arrival_time)
            assigned_node = resource.type

            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_type["input_size"],
                'Output Size': task_type["output_size"],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)

        # Logging and verification steps
        logger.info("\n=== Task Distribution Summary ===")
        logger.info(f"Total tasks requested: {total_tasks}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_csv(csv_data)

        return distributed_tasks
    def _Default_distribute_tasks(self, total_tasks: int) -> List[Task]:
        """
        Guaranteed distribution of exactly total_tasks tasks.
        Uses pre-validation, strict counting, and saves distribution data to CSV.
        """
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Step 1: Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Generate task records
        task_records = []
        for i, arrival_time in enumerate(cumulative_times):
            task_type = task_types[i % len(task_types)]
            task_records.append({
                'id': i + 1,
                'type': task_type["type"],
                'input_size': task_type["input_size"],
                'output_size': task_type["output_size"],
                'cpu_required': task_type["cpu_required"],
                'arrival_time': arrival_time
            })

        # Step 3: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 4: Pre-validate tasks
        for task_record in task_records:
            temp_task = Task(
                task_id=task_record['id'],
                task_type=task_record['type'],
                input_size=task_record['input_size'],
                output_size=task_record['output_size'],
                cpu_required=task_record['cpu_required']
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 5: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 6: Distribute tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_record in task_records:
            resource = self.resources[resource_index]

            task = Task(
                task_id=task_record['id'],
                task_type=task_record['type'],
                input_size=task_record['input_size'],
                output_size=task_record['output_size'],
                cpu_required=task_record['cpu_required']
            )
            task.arrival_time = task_record['arrival_time']

            arrival_time = base_time + timedelta(seconds=task_record['arrival_time'])
            assigned_node = resource.type

            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'  # Changed from 'queued' to match Task class states
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'  # Changed from 'failed' to match Task class states
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_record['input_size'],
                'Output Size': task_record['output_size'],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)

        # Step 7: Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        # Log summary
        logger.info("\n=== Task Distribution Summary ===")
        logger.info(f"Total tasks requested: {total_tasks}")
        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_csv(csv_data)

        return distributed_tasks

    def write_tasks_to_sjf_csv(self, task_data):
        """
        Write task distribution data to CSV with updated filename for SJF algorithm
        """
        # Create CSV folder if it doesn't exist
        csv_folder = "/content/drive/My Drive/CSV_dump"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        # Generate filename with timestamp and 'sjf' identifier
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'sjf_task_distribution_{timestamp}.csv')

        # Define CSV fields
        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Start Time', 'Input Transfer Time',
            'Processing Time', 'Output Transfer Time', 'Total Time',
            'Status', 'Assigned Node', 'Estimated Time'
        ]

        # Write data to CSV
        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for task in task_data:
                writer.writerow({
                    'Task ID': task['Task ID'],
                    'Type': task['Type'],
                    'Input Size (GB)': task.get('Input Size', 0),
                    'Output Size (GB)': task.get('Output Size', 0),
                    'Time of Arrival': task['Time of Arrival'],
                    'Start Time': task.get('Start Time', ''),
                    'Input Transfer Time': task.get('Input Transfer Time', ''),
                    'Processing Time': task.get('Processing Time', ''),
                    'Output Transfer Time': task.get('Output Transfer Time', ''),
                    'Total Time': task.get('Total Time', ''),
                    'Status': task['Status'],
                    'Assigned Node': task['Assigned Node'],
                    'Estimated Time': task.get('Estimated Time', 'N/A')
                })

        logger.info(f"Task distribution data written to {csv_filepath}")
        return csv_filepath

    def write_tasks_to_csv(self, task_data):
        """
        Updated CSV output to include transfer phases and data sizes
        """
        csv_folder = "/content/drive/My Drive/CSV_dump"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'round_robin_distribution_{timestamp}.csv')

        # Updated fieldnames to include transfer information
        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Start Time', 'Input Transfer Time',
            'Processing Time', 'Output Transfer Time', 'Total Time',
            'Status', 'Assigned Node'
        ]

        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for task in task_data:
                writer.writerow({
                    'Task ID': task['Task ID'],
                    'Type': task['Type'],
                    'Input Size (GB)': task.get('input_size', 0),
                    'Output Size (GB)': task.get('output_size', 0),
                    'Time of Arrival': task['Time of Arrival'],
                    'Start Time': task.get('start_time', ''),
                    'Input Transfer Time': task.get('input_transfer_time', ''),
                    'Processing Time': task.get('processing_time', ''),
                    'Output Transfer Time': task.get('output_transfer_time', ''),
                    'Total Time': task.get('total_time', ''),
                    'Status': task['Status'],
                    'Assigned Node': task['Assigned Node']
                })

        logger.info(f"Task distribution data written to {csv_filepath}")

        # Flush and sync to ensure all data is written to the drive
        logger.info("Google Drive has been unmounted. You can now access the CSV file in your Google Drive.")
    def distribute_tasks(self, total_tasks: int, distribution_type: str = 'default'):
        """
        Enhanced distribution method with verification and emergency fallback
        """
        # Mapping of distribution strategies
        distribution_strategies = {
            'default': self._Default_distribute_tasks,
            'round_robin': self._round_robin_distribution,
            'shortest_job_first': self._shortest_job_first_distribution,
            'tabu_search': self._tabu_search_distribution,
            'hybrid': self._optimized_hybrid_algorithm,
            'gwo_tabu': self._hybrid_gwo_tabu_distribution,
            'ga_gwo': self._GA_GWO_hybrid_algorithm
        }

        # Select and execute the appropriate distribution strategy
        if distribution_type in distribution_strategies:
            try:
                logger.info(f"Using {distribution_type} distribution strategy for {total_tasks} tasks")

                # First attempt with selected algorithm
                result = distribution_strategies[distribution_type](total_tasks)

                # Handle different return types
                if isinstance(result, tuple) and len(result) == 2:
                    distributed_tasks, csv_paths = result
                else:
                    distributed_tasks = result

                # Verify tasks were actually queued and fix if needed
                total_queued = sum(len(resource.task_queue) for resource in self.resources)
                if total_queued < total_tasks:
                    logger.warning(f"Verification failed: Only {total_queued} of {total_tasks} queued after {distribution_type}")
                    fixed_tasks = self.verify_and_fix_task_distribution(total_tasks)

                    # If we got CSV paths, package them with the fixed tasks
                    if 'csv_paths' in locals():
                        return fixed_tasks, csv_paths
                    else:
                        return fixed_tasks

                # Return the original result
                return result

            except Exception as e:
                logger.error(f"Error in {distribution_type} distribution: {e}")
                logger.info("Falling back to emergency distribution due to error")
                return self.verify_and_fix_task_distribution(total_tasks)
        else:
            raise ValueError(f"Unsupported distribution type: {distribution_type}. "
                            f"Supported types are: {list(distribution_strategies.keys())}")
    def _write_tabu_assignments_to_csv(self, assignment_data):
        """Helper method to write Tabu Search assignments to CSV with total cost."""
        csv_folder = "/content/drive/My Drive/CSV_dump"
        os.makedirs(csv_folder, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'tabu_task_distribution_{timestamp}.csv')

        # Calculate total cost for each task when written
        for task_record in assignment_data:
            # Skip if no resource assigned
            if task_record['Assigned Node'] == 'None':
                task_record['Total Cost'] = 'N/A'
                continue

            # Base execution time
            base_cost = float(task_record['Estimated Time'])
            total_cost = base_cost

            # Add WT3-Raspberry penalty if applicable
            if task_record['Type'] == 'WT3' and task_record['Assigned Node'].startswith('Raspberry_'):
                total_cost += base_cost * 3  # 5x penalty

            # Add cloud penalty if applicable
            elif task_record['Assigned Node'].startswith('Cloud_') and task_record['Type'] not in ['RT1', 'RT3']:
                total_cost += base_cost * 0.5

            task_record['Total Cost'] = f"{total_cost:.2f}"

        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time',
            'Total Cost'  # Added Total Cost field
        ]

        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(assignment_data)

        logger.info(f"Task assignments with costs written to {csv_filepath}")
        return csv_filepath

    def generate_live_feed(self):
        """
        Updated live visualization to show transfer phases
        """
        table = Table(
            title="Live Resource Processing Status",
            box=box.MINIMAL_DOUBLE_HEAD,
            show_header=True,
            header_style="bold magenta",
            border_style="bold green",
            show_lines=True,
            padding=(0, 1)
        )

        # Updated columns to show transfer phases
        table.add_column("Resource", style="bold blue", width=15)
        table.add_column("Type", style="bold cyan", width=20)
        table.add_column("Current Task", style="bold yellow", width=15)
        table.add_column("Phase", style="bold green", width=26)
        table.add_column("Progress", style="bold red", width=18)
        table.add_column("Data Size", style="bold yellow", width=18)
        table.add_column("Queue", style="bold green", width=15)
        table.add_column("Completed", style="bold magenta", width=15)
        table.add_column("Failed", style="bold red", width=15)

        for resource in self.resources:
            current_task = "None"
            phase = "Idle"
            progress = "N/A"
            data_size = "N/A"

            if resource.current_task:
                task = resource.current_task
                current_task = f"{task.id} ({task.type})"

                # Phase and progress based on task status
                phase_colors = {
                    'TRANSFERRING_INPUT': "yellow",
                    'PROCESSING': "blue",
                    'TRANSFERRING_OUTPUT': "cyan",
                    'COMPLETED': "green"
                }

                phase = task.status
                progress = f"{task.completion_percentage:.1f}%"

                # Show relevant data size based on phase
                if task.status == 'TRANSFERRING_INPUT':
                    data_size = f"↑{task.input_size:.1f}GB"
                elif task.status == 'TRANSFERRING_OUTPUT':
                    data_size = f"↓{task.output_size:.1f}GB"
                else:
                    data_size = f"↕{max(task.input_size, task.output_size):.1f}GB"

            # Add row with updated information
            table.add_row(
                f"{resource.id:<8}",
                f"{resource.type:<11}",
                current_task,
                phase,
                progress,
                data_size,
                f"{len(resource.task_queue):<5}",
                f"{len(resource.completed_tasks):<9}",
                str(len(resource.failed_tasks))
            )

        return table
    def write_simulation_results(self, metrics, start_time, distribution_type):
        """
        Write simulation results to CSV file

        Args:
            metrics (dict): Simulation metrics
            start_time (float): Simulation start timestamp
            distribution_type (str): Type of distribution algorithm used
        """
        csv_folder = "/content/drive/My Drive/EdgeSimPy/results"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'run_simulation_results_{distribution_type}_{timestamp}.csv')

        # Prepare simulation results
        results = {
            'Distribution_Algorithm': distribution_type,
            'Simulation_Start_Time': datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S'),
            'Simulation_End_Time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Total_Tasks': metrics['total_tasks'],
            'Completed_Tasks': metrics['completed_tasks'],
            'Failed_Tasks': metrics['failed_tasks'],
            'Makespan': f"{metrics['makespan']:.2f}s",
            'Throughput': f"{metrics['throughput']:.2f} tasks/s",
        }

        # Add load balancing metrics if available
        # Add utilization variability metrics if available
        if 'load_balancing_metrics' in metrics:
            # Map to the expected 'load_balancing' key format in the output
            lb_metrics = metrics['load_balancing_metrics']
            results.update({
                'Average_Resource_Utilization': f"{lb_metrics.get('average_utilization', 0):.4f}",
                'Coefficient_of_Variation': f"{lb_metrics.get('load_balance_score', 0):.4f}",
                'CoV_Standard_Deviation': f"{lb_metrics.get('utilization_std_dev', 0):.4f}"
            })

            # Add resource type specific metrics
            if 'resource_type_metrics' in lb_metrics:
                for resource_type, type_metrics in lb_metrics['resource_type_metrics'].items():
                    if resource_type != 'inter_type_balance':
                        results.update({
                            f'{resource_type}_avg_utilization': f"{type_metrics.get('average_utilization', 0):.4f}",
                            f'{resource_type}_CoV': f"{type_metrics.get('load_balance_score', 0):.4f}"
                        })

                # Add inter-type balance if available
                if 'inter_type_balance' in lb_metrics['resource_type_metrics']:
                    inter_balance = lb_metrics['resource_type_metrics']['inter_type_balance']
                    results.update({
                        'Inter_Type_CoV': f"{inter_balance.get('balance_score', 0):.4f}"
                    })

        # Add timing metrics if available
        if 'average_turnaround_time' in metrics:
            results['Average_Turnaround_Time'] = f"{metrics['average_turnaround_time']:.2f}s"
        if 'average_waiting_time' in metrics:
            results['Average_Waiting_Time'] = f"{metrics['average_waiting_time']:.2f}s"
        if 'average_execution_time' in metrics:
            results['Average_Execution_Time'] = f"{metrics['average_execution_time']:.2f}s"

        # Add failed tasks breakdown
        for task_type, count in metrics['failed_tasks_by_type'].items():
            results[f'Failed_{task_type}'] = count

        for resource_type, count in metrics['failed_tasks_by_resource'].items():
            results[f'Failed_On_{resource_type}'] = count

        # Write to CSV
        with open(csv_filepath, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Metric', 'Value'])
            for key, value in results.items():
                writer.writerow([key, value])

        logger.info(f"Simulation results written to: {csv_filepath}")
        return csv_filepath
    def run_simulation(self, total_tasks: int, distribution_type: str = 'default') -> Dict:
        """
        Run the simulation with enhanced logging and timing metrics.

        This method handles the simulation execution, including:
        1. Task distribution via selected algorithm
        2. Real-time processing and visualization
        3. Timing metrics calculation
        4. Resource utilization tracking
        5. Comprehensive result gathering

        Args:
            total_tasks (int): Number of tasks to simulate
            distribution_type (str): The distribution algorithm to use

        Returns:
            Dict: Comprehensive metrics about the simulation run
        """
        # Distribute tasks using the chosen algorithm
        distribution_result = self.distribute_tasks(total_tasks, distribution_type)

        # Check if result contains CSV paths (from hybrid algorithm)
        if isinstance(distribution_result, tuple) and len(distribution_result) == 2:
            tasks, csv_paths = distribution_result
            # Store CSV paths for later analysis
            self.algorithm_tracking_paths = csv_paths
            logger.info(f"Algorithm tracking data saved to: {csv_paths}")
        else:
            tasks = distribution_result
            self.algorithm_tracking_paths = {}

        simulation_start_time = datetime.now().timestamp()

        logger.info(f"Starting simulation with {total_tasks} total tasks")
        logger.info(f"Distributed tasks count: {len(tasks)}")

        # Start with original task distribution logging
        logger.info("\n=== Initial Task Distribution ===")
        for resource in self.resources:
            logger.debug(f"\nProcessing Resource: {resource.type}")
            logger.debug(f"  Current Task: {resource.current_task.type if resource.current_task else 'None'}")
            logger.debug(f"  Queue Length: {len(resource.task_queue)}")
            if resource.task_queue:
                for task in resource.task_queue[:5]:
                    logger.info(f"""
                    Task {task.id}:
                    - Type: {task.type}
                    - CPU Required: {task.total_cpu_required}
                    - Arrival Time: {task.arrival_time}
                    """)

        resource_utilization = {}
        datacenter_utilization_snapshots = []

        # Main simulation loop
        with Live(self.generate_live_feed(), refresh_per_second=1) as live:
            while True:
                self.current_time = datetime.now().timestamp() - simulation_start_time
                all_completed = True

                for resource in self.resources:
                    logger.debug(f"\nProcessing Resource: {resource.type}")
                    logger.debug(f"  Current Task: {resource.current_task.type if resource.current_task else 'None'}")
                    logger.debug(f"  Queue Length: {len(resource.task_queue)}")
                    # Add a debug log right here to check task states
                    for i, task in enumerate(resource.task_queue[:3]):  # Just check first 3 tasks
                        logger.info(f"Queue {i}: Task {task.id} - Status: {task.status}")

                    # Process the current queue for this resource
                    queue_status = resource.process_queue(self.current_time)
                    utilization = resource.calculate_resource_utilization()
                    resource_utilization[resource.type] = utilization

                    # Check completion status
                    if resource.current_task:
                        if (resource.current_task.status != 'COMPLETED' or
                            resource.current_task.completion_percentage < 100):
                            all_completed = False
                    if resource.task_queue:
                        all_completed = False

                # Update datacenter utilization
                datacenter_utilization_snapshots.append({
                    'timestamp': self.current_time,
                    'utilization': self.calculate_datacenter_utilization(
                        simulation_start_time,
                        datetime.now().timestamp() - simulation_start_time
                    )
                })

                # Update live visualization
                live.update(self.generate_live_feed())

                # Check if all tasks are complete
                total_completed = sum(len(r.completed_tasks) for r in self.resources)
                total_failed = sum(len(r.failed_tasks) for r in self.resources) + len(self.metrics['globally_failed_tasks'])

                if all_completed and (total_completed + total_failed >= total_tasks):
                    all_tasks_truly_complete = True

                    # Double-check all tasks are truly complete
                    for resource in self.resources:
                        for task in resource.completed_tasks:
                            if task.completion_percentage < 100 or task.status != 'COMPLETED':
                                all_tasks_truly_complete = False
                                break
                        if not all_tasks_truly_complete:
                            break

                    if all_tasks_truly_complete:
                        break

                time.sleep(0.1)

            # Calculate final metrics
            completed_tasks = []
            for resource in self.resources:
                completed_tasks.extend(resource.completed_tasks)
                logger.info(f"""
                Resource {resource.type} Final Status:
                - Completed Tasks: {len(resource.completed_tasks)}
                - Failed Tasks: {len(resource.failed_tasks)}
                """)

            # Calculate timing metrics with detailed debugging
            total_turnaround_time = 0.0
            total_waiting_time = 0.0
            valid_tasks = 0

            logger.info(f"\nCalculating timing metrics for {len(completed_tasks)} tasks:")

            for task in completed_tasks:
                # Debug log all timing values
                logger.info(f"""
                Task {task.id} Raw Timing Values Debug:
                - Status: {task.status}
                - Arrival Time Present: {task.arrival_time is not None}
                - Arrival Time: {task.arrival_time if task.arrival_time is not None else 'None'}
                - Start Time Present: {task.start_time is not None}
                - Start Time: {task.start_time if task.start_time is not None else 'None'}
                - Completion Time Present: {task.completion_time is not None}
                - Completion Time: {task.completion_time if task.completion_time is not None else 'None'}
                - Actual Exec Time: {task.actual_exec_time}
                """)

                # Check each condition separately for debugging
                has_completion = task.completion_time is not None
                has_arrival = task.arrival_time is not None
                has_exec_time = task.actual_exec_time > 0
                completion_after_arrival = (task.completion_time >= task.arrival_time) if (has_completion and has_arrival) else False

                logger.info(f"""
                Task {task.id} Validation Checks:
                - Has Completion Time: {has_completion}
                - Has Arrival Time: {has_arrival}
                - Has Exec Time > 0: {has_exec_time}
                - Completion After Arrival: {completion_after_arrival}
                """)

                if has_completion and has_arrival and has_exec_time and completion_after_arrival:
                    # Calculate turnaround time
                    task.turnaround_time = self.calculate_turnaround_time(task, simulation_start_time)
                    task.waiting_time = self.calculate_waiting_time(task)

                    # Log task timing details
                    logger.info(f"""
                    Task {task.id} Timing Details:
                    - Arrival Time: {task.arrival_time}
                    - Completion Time: {task.completion_time}
                    - Actual Execution Time: {task.actual_exec_time}
                    - Turnaround Time: {task.turnaround_time}
                    - Waiting Time: {task.waiting_time}
                    """)

                    # Additional validation for calculated times
                    if task.turnaround_time > 0:
                        total_turnaround_time += task.turnaround_time
                        total_waiting_time += task.waiting_time
                        valid_tasks += 1

                        logger.info(f"Task {task.id} ACCEPTED for metrics calculation")
                    else:
                        logger.warning(f"""
                        Task {task.id} has invalid calculated times:
                        - Turnaround Time: {task.turnaround_time:.2f}
                        - Waiting Time: {task.waiting_time:.2f}
                        """)
                else:
                    logger.warning(f"""
                    Task {task.id} failed validation:
                    - Missing completion time: {not has_completion}
                    - Missing arrival time: {not has_arrival}
                    - No execution time: {not has_exec_time}
                    - Completion before arrival: {not completion_after_arrival}
                    """)

            # After processing all tasks, log summary
            logger.info(f"""
            Timing Metrics Summary:
            Total Completed Tasks: {len(completed_tasks)}
            Valid Tasks for Timing: {valid_tasks}
            Total Turnaround Time: {total_turnaround_time:.2f}
            Total Waiting Time: {total_waiting_time:.2f}
            """)

            # Calculate averages
            if valid_tasks > 0:
                avg_turnaround = total_turnaround_time / valid_tasks
                avg_waiting = total_waiting_time / valid_tasks
                logger.info(f"""
                Final Timing Metrics:
                - Valid Tasks: {valid_tasks}
                - Average Turnaround Time: {avg_turnaround:.2f}s
                - Average Waiting Time: {avg_waiting:.2f}s
                """)
            else:
                logger.warning("No valid tasks found for timing calculation!")
                avg_turnaround = 0
                avg_waiting = 0

            # Calculate load balancing metrics
            load_balancing_metrics = self.calculate_load_balancing_metrics()

            # Update metrics dictionary
            self.metrics.update({
                'total_tasks': total_tasks,
                'completed_tasks': total_completed,
                'failed_tasks': total_failed,
                'makespan': self.current_time,
                'throughput': total_completed / self.current_time if self.current_time > 0 else 0,
                'load_balancing_metrics': load_balancing_metrics,  # Add load balancing metrics here
                'average_turnaround_time': avg_turnaround,
                'average_waiting_time': avg_waiting,
                'valid_tasks': valid_tasks
            })

            # If we have algorithm tracking data, add it to metrics
            if hasattr(self, 'algorithm_tracking_paths') and self.algorithm_tracking_paths:
                self.metrics['algorithm_tracking_paths'] = self.algorithm_tracking_paths

            # Process failures
            detailed_failed_tasks = []
            failed_tasks_by_type = {}
            failed_tasks_by_resource = {}

            for resource in self.resources:
                for task in resource.failed_tasks:
                    failure_info = {
                        'task_id': task.id,
                        'task_type': task.type,
                        'resource_type': resource.type,
                        'reason': getattr(task, 'failure_reason', 'Unknown reason')
                    }
                    detailed_failed_tasks.append(failure_info)
                    failed_tasks_by_type[task.type] = failed_tasks_by_type.get(task.type, 0) + 1
                    failed_tasks_by_resource[resource.type] = failed_tasks_by_resource.get(resource.type, 0) + 1

            # Add globally failed tasks
            for task in self.metrics['globally_failed_tasks']:
                detailed_failed_tasks.append({
                    'task_id': task.id,
                    'task_type': task.type,
                    'resource_type': "None",
                    'reason': getattr(task, 'failure_reason', 'Unknown reason')
                })

            # Update failure metrics
            self.metrics.update({
                'detailed_failed_tasks': detailed_failed_tasks,
                'failed_tasks_by_type': failed_tasks_by_type,
                'failed_tasks_by_resource': failed_tasks_by_resource
            })

            # Store final datacenter utilization
            self.metrics['final_datacenter_utilization'] = datacenter_utilization_snapshots[-1]['utilization'] if datacenter_utilization_snapshots else {}

            # Write results and finish
            results_file = self.write_simulation_results(self.metrics, simulation_start_time, distribution_type)
            logger.info(f"Simulation results saved to: {results_file}")

            # If we have algorithm tracking data, generate the visualizations
            if 'algorithm_tracking_paths' in self.metrics:
                try:
                    visualization_files = self.analyze_algorithm_progress(self.metrics['algorithm_tracking_paths'])
                    logger.info(f"Algorithm progress visualizations saved to: {visualization_files}")
                    self.metrics['algorithm_visualizations'] = visualization_files
                except Exception as e:
                    logger.error(f"Error generating algorithm visualizations: {e}")

            return self.metrics
def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Smartphone_{i}",  # Changed from Edge_ to Smartphone_
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=400         # 20 MB/s
            )
        )

    # Create 5 Raspberry Pi Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=100          # 5 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=1200         # 80 MB/s
            )
        )

    # Log created resources
    logger.info(f"Created {len(resources)} resources: {[r.type for r in resources]}")

    return resources
def main():

    # Disable all logging at the beginning
    #import logging
    logging.disable(logging.CRITICAL)
    # Create resources
    resources = create_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    try:
        # Define the total number of tasks - use a very small number for debugging
        total_tasks = 500  # Just 5 tasks for easier debugging

        # Increase logging detail
        #logger.setLevel(logging.DEBUG)

        logger.info("=== STARTING SIMULATION WITH DETAILED DEBUGGING ===")

        # Run simulation
        metrics = scheduler.run_simulation(total_tasks, distribution_type='round_robin')

        logger.info("=== SIMULATION COMPLETED ===")

        # Print final metrics
        print("\nFinal Simulation Metrics:")
        for key, value in metrics.items():
            if key not in ['resource_utilization', 'datacenter_utilization_snapshots', 'detailed_failed_tasks']:
                print(f"{key}: {value}")

    except Exception as e:
        logger.error(f"Unhandled error in main: {e}")
        import traceback
        traceback.print_exc()
if __name__ == "__main__":
    main()

Mounted at /content/drive

📂 Available log files in EdgeSimPy/logs:
GA_Simulation.log.20250305_203847
GA_Simulation.log.20250305_203910
GA_Simulation.log.20250305_203934
GA_Simulation.log.20250305_204035
GA_Simulation.log.20250305_204135
TS_Simulation_20250305_214058.log
Hybrid_GA_TS_Simulation_20250305_214437.log
TS_Simulation.log.20250305_214552
Hybrid_Simulation.log.20250305_215956
Hybrid_Simulation.log.20250305_220302
Hybrid_Simulation.log.20250305_220606
GA_Simulation.log.20250305_221222
TS_Simulation.log.20250305_221222
Hybrid_GA_TS.log.20250305_221642
Hybrid_GA_TS.log.20250305_221756
Hybrid_Simulation.log.20250306_174611
Hybrid_GA_TS.log.20250306_174650
GA_Simulation.log.20250306_174729
Hybrid_GA_TS.log.20250306_174746
simulation_2025-03-07_20-43-55.log
simulation_2025-03-07_20-45-25.log
simulation_2025-03-09_08-06-19.log
simulation_2025-03-10_23-10-11.log
simulation_2025-03-18_22-42-38.log
simulation_2025-03-19_13-30-23.log
simulation_2025-03-19_13-31-17.log
simulation_2025-03-

Output()

Files Categories

In [ ]:
import json
from typing import List, Dict, Any
import time
from collections import defaultdict
from rich.console import Console
from rich.table import Table

# Task Specifications
TASK_SPECS = {
    # Read Tasks
    'RT1': {
        'instructions': 2_000_000,
        'data_size': 5,
        'description': 'CPU-intensive, memory-intensive',
        'example': 'Financial modeling based on large historical dataset'
    },
    'RT2': {
        'instructions': 4_000_000,
        'data_size': 0.2,
        'description': 'CPU-intensive, memory-light',
        'example': 'Computation of NP-hard optimization problem'
    },
    'RT3': {
        'instructions': 200_000,
        'data_size': 5,
        'description': 'CPU-light, memory-intensive',
        'example': 'Light database queries on large in-memory dataset'
    },
    'RT4': {
        'instructions': 500_000,
        'data_size': 0.5,
        'description': 'CPU-light, memory-light',
        'example': 'Light video editing'
    },
    # Write Tasks
    'WT1': {
        'instructions': 2_000_000,
        'data_size': 2,
        'description': 'CPU-intensive, I/O-intensive',
        'example': 'Complex data write operations'
    },
    'WT2': {
        'instructions': 1_000_000,
        'data_size': 0.5,
        'description': 'CPU-intensive, I/O-light',
        'example': 'Streamlined data writing'
    },
    'WT3': {
        'instructions': 500_000,
        'data_size': 5,
        'description': 'CPU-light, I/O-intensive',
        'example': 'Bulk data transfer'
    },
    'WT4': {
        'instructions': 200_000,
        'data_size': 0.2,
        'description': 'CPU-light, I/O-light',
        'example': 'Simple data logging'
    }
}

class Task:
    """
    Enhanced Task class with precise categorization
    """
    def __init__(self,
                 task_id: int,
                 data_size: float,     # in MB
                 cpu_required: float,  # in MI (Million Instructions)
                 task_details: Dict[str, Any] = None):
        self.id = task_id
        self.data_size = data_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Detailed metadata
        self.details = task_details or {}
        self.task_name = self.details.get('task_name', f'Task_{task_id}')
        self.size = self.details.get('size', 'unspecified')
        self.type = self.details.get('type', 'unknown')

        # Precise task category identification
        self.task_category = self._identify_precise_category()

        # Task lifecycle tracking
        self.arrival_time = 0
        self.start_time = 0
        self.completion_time = 0
        self.status = 'pending'

    def _identify_precise_category(self) -> str:
        """
        Identify precise task category based on specifications
        """
        for category, spec in TASK_SPECS.items():
            if (abs(self.total_cpu_required - spec['instructions']) < 1000 and
                abs(self.data_size - spec['data_size']) < 0.1):
                return category
        return 'Uncategorized'

    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = time.time()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }

class Resource:
    """
    Resource class with advanced task tracking
    """
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # in MI/s (Million Instructions per Second)
                 memory: int,        # in GB
                 bandwidth: int):    # in MB/s
        self.id = resource_id
        self.type = resource_type
        self.cpu_rating = cpu_rating
        self.memory = memory
        self.bandwidth = bandwidth

        # Advanced task tracking
        self.task_queue: List[Task] = []
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []

        # Detailed task categorization tracking
        self.task_category_counts = defaultdict(int)
        self.queue_category_counts = defaultdict(int)
        self.task_details = defaultdict(lambda: {
            'total_instr': 0,
            'total_data_size': 0.0,
            'description': '',
            'example': ''
        })

    def enqueue_task(self, task: Task):
        """
        Add task to resource's queue with detailed categorization
        """
        task.queue_position = len(self.task_queue)
        self.task_queue.append(task)
        self.queue_category_counts[task.task_category] += 1

        # Track task details
        category = task.task_category
        if category in TASK_SPECS:
            spec = TASK_SPECS[category]
            self.task_details[category]['total_instr'] += task.total_cpu_required
            self.task_details[category]['total_data_size'] += task.data_size
            self.task_details[category]['description'] = spec['description']
            self.task_details[category]['example'] = spec['example']

    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks with detailed categorization
        """
        # Process current tasks
        for task in self.current_tasks[:]:
            processing_result = task.process(self.cpu_rating)

            if processing_result['status'] == 'completed':
                self.current_tasks.remove(task)
                self.completed_tasks.append(task)

                # Track completed task category
                self.task_category_counts[task.task_category] += 1

        # Move tasks from queue to current tasks
        while self.task_queue and len(self.current_tasks) < 5:
            next_task = self.task_queue.pop(0)

            # Update task timing
            next_task.start_time = current_time

            # Decrement queue category count
            self.queue_category_counts[next_task.task_category] -= 1

            self.current_tasks.append(next_task)

        return {
            'completed_tasks': len(self.completed_tasks),
            'current_tasks': len(self.current_tasks),
            'queue_length': len(self.task_queue),
            'completed_categories': dict(self.task_category_counts),
            'queue_categories': {k: v for k, v in self.queue_category_counts.items() if v > 0}
        }

class DetailedTaskScheduler:
    """
    Scheduler with comprehensive task categorization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.console = Console()
        self.metrics = {
            'total_tasks': 0,
            'task_distribution': {},
        }

    def load_tasks_from_json(self, json_path: str) -> List[Task]:
        """
        Load tasks with comprehensive parsing
        """
        with open(json_path, 'r') as f:
            task_data = json.load(f)

        tasks_list = task_data.get('tasks', [])

        tasks = []
        for task_dict in tasks_list:
            task = Task(
                task_id=task_dict.get('id', len(tasks) + 1),
                data_size=task_dict.get('data_size', 10),
                cpu_required=task_dict.get('instructions', 50000),
                task_details=task_dict
            )
            tasks.append(task)

        return tasks

    def distribute_tasks(self):
        """
        Distribute tasks across resources with categorization
        """
        # Load tasks
        tasks = self.load_tasks_from_json(
            '/content/drive/My Drive/FCFS_Task_Sets/fcfs_task_set_20250201_201915.json'
        )
        self.metrics['total_tasks'] = len(tasks)

        # Track task distribution
        task_distribution = {resource.type: 0 for resource in self.resources}

        # Round-robin distribution
        resource_index = 0
        for task in tasks:
            resource = self.resources[resource_index]
            resource.enqueue_task(task)
            task_distribution[resource.type] += 1
            resource_index = (resource_index + 1) % len(self.resources)

        self.metrics['task_distribution'] = task_distribution

        return tasks

    def run_simulation(self, max_iterations: int = 1000):
        """
        Run simulation and track task categorization
        """
        # Distribute tasks
        self.distribute_tasks()

        # Process tasks
        for _ in range(max_iterations):
            all_completed = True

            for resource in self.resources:
                resource.process_queue(time.time())

                # Check if resource still has tasks
                if (len(resource.task_queue) > 0 or
                    len(resource.current_tasks) > 0):
                    all_completed = False

            if all_completed:
                break

        # Generate comprehensive report
        self.generate_final_report()

    def generate_final_report(self):
        """
        Generate detailed report of task processing with comprehensive information
        """
        self.console.rule("[bold blue]Task Processing Report[/bold blue]")

        for resource in self.resources:
            # Create table for resource
            resource_table = Table(title=f"Resource {resource.id}: {resource.type}")
            resource_table.add_column("Task Category", style="cyan")
            resource_table.add_column("Completed Tasks", style="green")
            resource_table.add_column("Remaining in Queue", style="red")
            resource_table.add_column("Total Instructions (MI)", style="magenta")
            resource_table.add_column("Total Data Size (GB)", style="yellow")
            resource_table.add_column("Description", style="blue")

            # Combine all task categories
            all_categories = sorted(set(list(resource.task_category_counts.keys()) +
                                 list(resource.queue_category_counts.keys())))

            # Populate table
            for category in all_categories:
                completed = resource.task_category_counts.get(category, 0)
                queued = resource.queue_category_counts.get(category, 0)

                # Get task details
                details = resource.task_details.get(category, {
                    'total_instr': 0,
                    'total_data_size': 0.0,
                    'description': 'N/A',
                    'example': ''
                })

                resource_table.add_row(
                    category,
                    str(completed),
                    str(queued),
                    f"{details['total_instr']:,}",
                    f"{details['total_data_size']:.2f}",
                    details['description']
                )

            # Print resource-specific table
            self.console.print(resource_table)
            self.console.print("\n")

def create_original_resources():
    """
    Create resources exactly matching the original configuration table
    """
    return [
        # Raspberry Pi Edge Node
        Resource(
            resource_id=1,
            resource_type="Edge_Raspberry_Pi",
            cpu_rating=80000,    # 80,000 MI/s
            memory=1,            # 1 GB
            bandwidth=5          # 5 MB/s
        ),

        # Smartphone Edge Node
        Resource(
            resource_id=2,
            resource_type="Edge_Smartphone",
            cpu_rating=400000,   # 400,000 MI/s
            memory=4,            # 4 GB
            bandwidth=20         # 20 MB/s
        ),

        # Cloud Host
        Resource(
            resource_id=3,
            resource_type="Cloud_Host",
            cpu_rating=1000000,  # 1,000,000 MI/s
            memory=32,           # 32 GB
            bandwidth=80         # 80 MB/s
        )
    ]

def main():
    # Create resources
    resources = create_original_resources()

    # Initialize scheduler
    scheduler = DetailedTaskScheduler(resources)

    # Run simulation
    scheduler.run_simulation()

if __name__ == "__main__":
    main()


Task Generator mixed large and small files


In [ ]:
import random
import json
from typing import List, Dict, Any
import os
from collections import defaultdict

class TaskGenerator:
    def __init__(self):
        # Detailed task configurations with more specific characteristics
        self.task_configs = {
            # Large Read Tasks
            'large_read_tasks': {
                'RT1': {
                    'instr': 2_000_000,  # Million Instructions
                    'data': 5,           # GB
                    'cpu_intensity': 'high',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-intensive, memory-intensive'
                },
                'RT2': {
                    'instr': 4_000_000,
                    'data': 0.2,
                    'cpu_intensity': 'high',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-intensive, memory-light'
                }
            },
            # Large Write Tasks
            'large_write_tasks': {
                'WT1': {
                    'instr': 2_000_000,
                    'data': 2,
                    'cpu_intensity': 'high',
                    'io_intensity': 'high',
                    'task_class': 'CPU-intensive, I/O-intensive'
                },
                'WT2': {
                    'instr': 1_000_000,
                    'data': 0.5,
                    'cpu_intensity': 'high',
                    'io_intensity': 'low',
                    'task_class': 'CPU-intensive, I/O-light'
                }
            },
            # Small Read Tasks
            'small_read_tasks': {
                'RT3': {
                    'instr': 200_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'high',
                    'task_class': 'CPU-light, memory-intensive'
                },
                'RT4': {
                    'instr': 500_000,
                    'data': 0.5,
                    'cpu_intensity': 'low',
                    'memory_intensity': 'low',
                    'task_class': 'CPU-light, memory-light'
                }
            },
            # Small Write Tasks
            'small_write_tasks': {
                'WT3': {
                    'instr': 500_000,
                    'data': 5,
                    'cpu_intensity': 'low',
                    'io_intensity': 'high',
                    'task_class': 'CPU-light, I/O-intensive'
                },
                'WT4': {
                    'instr': 200_000,
                    'data': 0.2,
                    'cpu_intensity': 'low',
                    'io_intensity': 'low',
                    'task_class': 'CPU-light, I/O-light'
                }
            }
        }

    def generate_large_task_set(self, total_tasks: int, large_task_percentage: float = 0.7) -> Dict[str, Any]:
        """
        Generate a comprehensive set of tasks with detailed categorization
        """
        # Calculate number of each type
        num_large_tasks = int(total_tasks * large_task_percentage)
        num_small_tasks = total_tasks - num_large_tasks

        # Generate tasks
        tasks = []
        task_id = 1

        # Generate large tasks
        large_tasks = self._generate_tasks(num_large_tasks, "large", task_id)
        tasks.extend(large_tasks)
        task_id += num_large_tasks

        # Generate small tasks
        small_tasks = self._generate_tasks(num_small_tasks, "small", task_id)
        tasks.extend(small_tasks)

        # Shuffle tasks to randomize their order
        random.shuffle(tasks)

        # Categorize tasks
        categorized_tasks = self._categorize_tasks(tasks)

        # Prepare task set metadata
        task_set_metadata = {
            "total_tasks": total_tasks,
            "large_task_percentage": large_task_percentage,
            "large_tasks": num_large_tasks,
            "small_tasks": num_small_tasks,
            "task_distribution": {
                "read_tasks": sum(1 for task in tasks if task.get('type') == 'read'),
                "write_tasks": sum(1 for task in tasks if task.get('type') == 'write')
            }
        }

        return {
            "metadata": task_set_metadata,
            "categorized_tasks": categorized_tasks,
            "raw_tasks": tasks
        }

    def _generate_tasks(self, num_tasks: int, size: str, start_id: int) -> List[Dict]:
        """Generate tasks of a specific size"""
        tasks = []

        for i in range(num_tasks):
            # Randomly choose between read and write tasks (50-50 distribution)
            task_type = random.choice(["read", "write"])

            # Get appropriate config based on size and type
            config_key = f"{size}_{task_type}_tasks"
            possible_tasks = self.task_configs[config_key]

            # Randomly select a task configuration
            task_name = random.choice(list(possible_tasks.keys()))
            task_config = possible_tasks[task_name]

            # Create task dictionary
            task = {
                "id": start_id + i,
                "task_name": task_name,
                "size": size,
                "type": task_type,
                "instructions": task_config['instr'],
                "data_size": task_config['data'],
                "arrival_time": random.randint(0, 1000),  # Random arrival time
                "status": "pending",
                **{k: v for k, v in task_config.items() if k not in ['instr', 'data']}
            }
            tasks.append(task)

        return tasks

    def _categorize_tasks(self, tasks: List[Dict]) -> Dict[str, List[Dict]]:
        """
        Categorize tasks by their specific characteristics
        """
        categorized = defaultdict(list)

        # Categorize by task names (RT1, RT2, etc.)
        for task in tasks:
            categorized[task['task_name']].append(task)

        return dict(categorized)

def generate_and_save_task_set(total_tasks: int = 1500, large_task_percentage: float = 0.7):
    """
    Generate task set, save to file, and print detailed categorization
    """
    # Create task generator
    generator = TaskGenerator()

    # Generate tasks
    task_set = generator.generate_large_task_set(
        total_tasks=total_tasks,
        large_task_percentage=large_task_percentage
    )

    # Print detailed categorization
    print("\n--- DETAILED TASK CATEGORIZATION ---")
    for task_category, tasks in task_set['categorized_tasks'].items():
        print(f"\n{task_category} Tasks:")
        print(f"Total {task_category} Tasks: {len(tasks)}")
        print("Sample Task Details:")
        for task in tasks[:3]:  # Print first 3 tasks of each category
            print("\nTask Details:")
            for key, value in task.items():
                print(f"{key}: {value}")
        print("-" * 50)

    # Print overall metadata
    print("\n--- TASK SET METADATA ---")
    print(json.dumps(task_set['metadata'], indent=2))

    # Save to JSON
    save_path = 'fcfs_task_set.json'
    with open(save_path, 'w') as f:
        json.dump(task_set, f, indent=2)

    print(f"\nFull task set saved to: {save_path}")

    return task_set

# Run the task generation
if __name__ == "__main__":
    generate_and_save_task_set()



--- DETAILED TASK CATEGORIZATION ---

WT1 Tasks:
Total WT1 Tasks: 270
Sample Task Details:

Task Details:
id: 156
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 401
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 951
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 637
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive

Task Details:
id: 123
task_name: WT1
size: large
type: write
instructions: 2000000
data_size: 2
arrival_time: 52
status: pending
cpu_intensity: high
io_intensity: high
task_class: CPU-intensive, I/O-intensive
--------------------------------------------------

RT2 Tasks:
Total RT2 Tasks: 253
Sample Task Details:

Task Details:
id: 462
task_name: RT2
size: large
type: read
instructions: 4000000
data_size: 0.2
arrival_time: 656
status: pending
cpu_intensity: high
memory_intensity: l

Once we have our stopping criterion, we can finally run our simulation by creating an instance of the `Simulator` class, loading a dataset, and calling the `run_model()` method.

In [ ]:

# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

# Checking the placement output
for service in Service.all():
    print(f"{service}. Host: {service.server}")

Service_1. Host: EdgeServer_1
Service_2. Host: EdgeServer_1
Service_3. Host: EdgeServer_1
Service_4. Host: EdgeServer_1
Service_5. Host: EdgeServer_1
Service_6. Host: EdgeServer_1
